<a href="https://colab.research.google.com/github/RussAbbott/Wordle/blob/main/Copy_of_Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
from functools import reduce
from itertools import product
from math import ceil, floor, log2
from random import choice
from re import sub

# fmean (float mean) is faster than mean
from statistics import fmean, median
from time import perf_counter
from typing import Any, Callable, Dict, ForwardRef, List, Optional, Set, Sized, Tuple, TypeVar, Union
# Guess = ForwardRef('Guess')
G = TypeVar("G", bound="Guess")

from typing_extensions import Self
from warnings import warn


In [ ]:
## Tests


# alpha_segments() tests
def test_alpha_segments(verbose=False):
    print(f'1. {alpha_segments([])}\n')
    print(f'2. {alpha_segments(wordle_word_list[:150])}\n')
    print(f'3. {alpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=True)}\n')
    print(f'4. {alpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=False)}\n')
    print(f'5. {alpha_segments(wordle_word_list[2200:])}\n')


# Secret_Candidates test
def test_Secret_Candidates(verbose=False):
    test_words = ['arise', 'crane', 'crate', 'irate', 'least', 'plate', 'raise',
                  'sauce', 'sauce', 'saute', 'shine', 'slate', 'trace']

    # Test Words_Profile on both words and test_words
    for wrds in [test_words]:
        candidates = Secret_Candidates(current_candidates=wordle_word_list, guess_words=wordle_word_list,
                                       filter=Filter(5))
        # print(f'\n{candidates.best_guesses = }')
        print(f'{candidates.best_position_guesses = }\n\n')



# This data is used in the following two tests.
secret_guess_exp_list = [('hpaax', 'aaapz', "y#gy#"), ('aabbb', 'ccaaa', "##yy#"),
                         ('godly', 'dolly', "yg#gg"), ('madam', 'mamma', 'ggy#y'),
                         ('madam', 'magma', 'gg#yy'), ('moose', 'onion', 'y##y#')
                         ]

# Scoring test
def test_Scoring(verbose=False):
    """
    Scoring: g (green)  = Letter in correct position;
             y (yellow) = Letter in incorrect position;
             # (gray)   = Letter not in word.

    In the following example:  guess  = 'aaapz'
                               secret = 'hpaay'
                            -> score  = 'y#gy#'

    First check the matching letters.

    The third letter in both the guess and the secret is "a". So the third letter in the score must be "g".

    The first letter in the guess, 'a', is associated with the fourth letter in the secret, 'a'. (This is
    why the matching letters must be determined first.) So the first letter in the score is "y".

    The second letter in the guess, 'a', has nothing in secret to be associated with. So the second letter
    in the score is "#".

    The fourth letter in the guess, 'p', is associated with the second letter in guess. So the fourth letter
    in the score is "y".

    The fifth letter in the guess, 'z', is not in secret. So the fifth letter in the score is "#".
    """

    for secret, guess, exp_score in secret_guess_exp_list:
        score = Utils.score_a_guess(secret=secret, guess=guess)
        print(f' {secret = } :: {guess = } -> {score = } {exp_score = } => {"Pass" if score == exp_score else "Fail" }')


# Filter Test
def test_Filter(verbose=False):
    filter = Filter(5)
    print(f'1.\n{filter}\n')
    filter.greens = list("_x___")
    filter.edit_filter('abxcd', '##y##', verbose=True)
    print(f'2.\n{filter}\n')

    print(f'{Utils.score_a_guess("wxyza", "wxyza")} -> ggggg\n')
    filter.edit_filter('wxyza', 'ggggg', verbose=True)
    print(f'3.\n{filter}')

    print(f'\n{filter.word_passes("wxyza") = } -> True')

    filter.yellows['x'] = 1
    print(f'\n{filter.yellows = }')
    print(f'{filter.word_passes("wxyza") = } -> False')

    filter.yellows.pop('x')
    print(f'\n{filter.yellows = }')
    print(f'{filter.word_passes("wxyza") = } -> True')
    filter.grays.add('a')
    print(f'\n{filter.grays = }')
    print(f'{filter.word_passes("wxyza") =  } -> False')


# Test nbr_remaining_candidates guess selection
def test_fwst_rem(verbose=False):
    scp = Secret_Candidates(wordle_word_list, wordle_word_list, Filter(5), False)
    scp.max_candidates_reduction(current_candidates=wordle_word_list, guess_words=['arise', 'crane'])


#######################################################################################################
# Run tests
tests = [(False, test_alpha_segments),      (False, test_Scoring),
         (False, test_Filter),              (False, test_Secret_Candidates),
         (False, test_fwst_rem),           # (False, Utils.is_long_test),
         ]

for run, test in tests:
    if run:
        print(f'\tRunning: {test.__name__}\n')
        test()
        print(f'{"="*100}', flush=True)


In [ ]:

guess_words_long = [
    'aahed', 'aalii', 'aapas', 'aargh', 'aarti', 'abaca', 'abaci', 'aback', 'abacs', 'abaft', 'abaht', 'abaka', 'abamp', 'aband', 'abase', 'abash', 'abask', 'abate', 'abaya', 'abbas', 'abbed', 'abbes', 'abbey', 'abbot', 'abcee', 'abeam', 'abear', 'abeat', 'abeer', 'abele', 'abeng', 'abers', 'abets', 'abeys', 'abhor', 'abide', 'abies', 'abius', 'abjad', 'abjud', 'abled', 'abler', 'ables', 'ablet', 'ablow', 'abmho', 'abnet', 'abode', 'abohm', 'aboil', 'aboma', 'aboon', 'abord', 'abore', 'aborn', 'abort', 'about', 'above', 'abram', 'abray', 'abrim', 'abrin', 'abris', 'absey', 'absit', 'abuna', 'abune', 'abura', 'aburn', 'abuse', 'abuts', 'abuzz', 'abyes', 'abysm', 'abyss', 'acais', 'acara', 'acari', 'accas', 'accha', 'accoy', 'accra', 'acedy', 'acene', 'acerb', 'acers', 'aceta', 'achar', 'ached', 'acher', 'aches', 'achey', 'achoo', 'acids', 'acidy', 'acies', 'acing', 'acini', 'ackee', 'acker', 'acmes', 'acmic', 'acned', 'acnes', 'acock', 'acoel', 'acold', 'acone', 'acorn', 'acral', 'acred', 'acres', 'acrid', 'acron', 'acros', 'acryl', 'actas', 'acted', 'actin', 'acton', 'actor', 'actus', 'acute', 'acyls', 'adage', 'adapt', 'adats', 'adawn', 'adaws', 'adays', 'adbot', 'addas', 'addax', 'added', 'adder', 'addin', 'addio', 'addle', 'addra', 'adead', 'adeem', 'adept', 'adhan', 'adhoc', 'adieu', 'adios', 'adits', 'adlib', 'adman', 'admen', 'admin', 'admit', 'admix', 'adnex', 'adobe', 'adobo', 'adoon', 'adopt', 'adorb', 'adore', 'adorn', 'adown', 'adoze', 'adrad', 'adraw', 'adred', 'adret', 'adrip', 'adsum', 'aduki', 'adult', 'adunc', 'adust', 'advew', 'advts', 'adyta', 'adyts', 'adzed', 'adzes', 'aecia', 'aedes', 'aeger', 'aegis', 'aeons', 'aerie', 'aeros', 'aesir', 'aevum', 'afald', 'afanc', 'afara', 'afars', 'afear', 'affix', 'affly', 'afion', 'afire', 'afizz', 'aflaj', 'aflap', 'aflow', 'afoam', 'afoot', 'afore', 'afoul', 'afret', 'afrit', 'afros', 'after', 'aftos', 'again', 'agals', 'agama', 'agami', 'agamy', 'agape', 'agars', 'agasp', 'agast', 'agate', 'agaty', 'agave', 'agaze', 'agbas', 'agene', 'agent', 'agers', 'aggag', 'agger', 'aggie', 'aggri', 'aggro', 'aggry', 'aghas', 'agidi', 'agila', 'agile', 'aging', 'agios', 'agism', 'agist', 'agita', 'aglee', 'aglet', 'agley', 'agloo', 'aglow', 'aglus', 'agmas', 'agoge', 'agogo', 'agone', 'agons', 'agony', 'agood', 'agora', 'agree', 'agria', 'agrin', 'agros', 'agrum', 'agued', 'agues', 'aguey', 'aguna', 'agush', 'aguti', 'ahead', 'aheap', 'ahent', 'ahigh', 'ahind', 'ahing', 'ahint', 'ahold', 'ahole', 'ahull', 'ahuru', 'aidas', 'aided', 'aider', 'aides', 'aidoi', 'aidos', 'aiery', 'aigas', 'aight', 'ailed', 'aimag', 'aimak', 'aimed', 'aimer', 'ainee', 'ainga', 'aioli', 'aired', 'airer', 'airns', 'airth', 'airts', 'aisle', 'aitch', 'aitus', 'aiver', 'aixes', 'aiyah', 'aiyee', 'aiyoh', 'aiyoo', 'aizle', 'ajies', 'ajiva', 'ajuga', 'ajupa', 'ajwan', 'akara', 'akees', 'akela', 'akene', 'aking', 'akita', 'akkas', 'akker', 'akoia', 'akoja', 'akoya', 'aksed', 'akses', 'alaap', 'alack', 'alala', 'alamo', 'aland', 'alane', 'alang', 'alans', 'alant', 'alapa', 'alaps', 'alarm', 'alary', 'alata', 'alate', 'alays', 'albas', 'albee', 'albid', 'album', 'alcea', 'alces', 'alcid', 'alcos', 'aldea', 'alder', 'aldol', 'aleak', 'aleck', 'alecs', 'aleem', 'alefs', 'aleft', 'aleph', 'alert', 'alews', 'aleye', 'alfas', 'algae', 'algal', 'algas', 'algid', 'algin', 'algor', 'algos', 'algum', 'alias', 'alibi', 'alick', 'alien', 'alifs', 'align', 'alike', 'alims', 'aline', 'alios', 'alist', 'alive', 'aliya', 'alkie', 'alkin', 'alkos', 'alkyd', 'alkyl', 'allan', 'allay', 'allee', 'allel', 'allen', 'aller', 'alley', 'allin', 'allis', 'allod', 'allot', 'allow', 'alloy', 'allus', 'allyl', 'almah', 'almas', 'almeh', 'almes', 'almud', 'almug', 'alods', 'aloed', 'aloes', 'aloft', 'aloha', 'aloin', 'alone', 'along', 'aloof', 'aloos', 'alose', 'aloud', 'alowe', 'alpha', 'altar', 'alter', 'altho', 'altos', 'alula', 'alums', 'alumy', 'alure', 'alurk', 'alvar', 'alway', 'amahs', 'amain', 'amari', 'amaro', 'amass', 'amate', 'amaut', 'amaze', 'amban', 'amber', 'ambit', 'amble', 'ambos', 'ambry', 'ameba', 'ameer', 'amend', 'amene', 'amens', 'ament', 'amias', 'amice', 'amici', 'amide', 'amido', 'amids', 'amies', 'amiga', 'amigo', 'amine', 'amino', 'amins', 'amirs', 'amiss', 'amity', 'amlas', 'amman', 'ammas', 'ammon', 'ammos', 'amnia', 'amnic', 'amnio', 'amoks', 'amole', 'among', 'amore', 'amort', 'amour', 'amove', 'amowt', 'amped', 'ample', 'amply', 'ampul', 'amrit', 'amuck', 'amuse', 'amyls', 'anana', 'anata', 'ancho', 'ancle', 'ancon', 'andic', 'andro', 'anear', 'anele', 'anent', 'angas', 'angel', 'anger', 'angle', 'anglo', 'angry', 'angst', 'anigh', 'anile', 'anils', 'anima', 'anime', 'animi', 'anion', 'anise', 'anker', 'ankhs', 'ankle', 'ankus', 'anlas', 'annal', 'annan', 'annas', 'annat', 'annex', 'annoy', 'annul', 'annum', 'annus', 'anoas', 'anode', 'anole', 'anomy', 'ansae', 'ansas', 'antae', 'antar', 'antas', 'anted', 'antes', 'antic', 'antis', 'antra', 'antre', 'antsy', 'anura', 'anvil', 'anyon', 'aorta', 'apace', 'apage', 'apaid', 'apart', 'apayd', 'apays', 'apeak', 'apeek', 'apers', 'apert', 'apery', 'apgar', 'aphid', 'aphis', 'apian', 'aping', 'apiol', 'apish', 'apism', 'apnea', 'apode', 'apods', 'apols', 'apoop', 'aport', 'appal', 'appam', 'appay', 'appel', 'apple', 'apply', 'appro', 'appts', 'appui', 'appuy', 'apres', 'apron', 'apses', 'apsis', 'apsos', 'apted', 'apter', 'aptly', 'aquae', 'aquas', 'araba', 'araks', 'arame', 'arars', 'arbah', 'arbas', 'arbor', 'arced', 'archi', 'arcos', 'arcus', 'ardeb', 'ardor', 'ardri', 'aread', 'areae', 'areal', 'arear', 'areas', 'areca', 'aredd', 'arede', 'arefy', 'areic', 'arena', 'arene', 'arepa', 'arere', 'arete', 'arets', 'arett', 'argal', 'argan', 'argil', 'argle', 'argol', 'argon', 'argot', 'argue', 'argus', 'arhat', 'arias', 'ariel', 'ariki', 'arils', 'ariot', 'arise', 'arish', 'arith', 'arked', 'arled', 'arles', 'armed', 'armer', 'armet', 'armil', 'armor', 'arnas', 'arnis', 'arnut', 'aroba', 'aroha', 'aroid', 'aroma', 'arose', 'arpas', 'arpen', 'arrah', 'arras', 'array', 'arret', 'arris', 'arrow', 'arroz', 'arsed', 'arses', 'arsey', 'arsis', 'arson', 'artal', 'artel', 'arter', 'artic', 'artis', 'artly', 'artsy', 'aruhe', 'arums', 'arval', 'arvee', 'arvos', 'aryls', 'asada', 'asana', 'ascon', 'ascot', 'ascus', 'asdic', 'ashed', 'ashen', 'ashes', 'ashet', 'aside', 'asity', 'askar', 'asked', 'asker', 'askew', 'askoi', 'askos', 'aspen', 'asper', 'aspic', 'aspie', 'aspis', 'aspro', 'assai', 'assam', 'assay', 'assed', 'asses', 'asset', 'assez', 'assot', 'aster', 'astir', 'astun', 'asura', 'asway', 'aswim', 'asyla', 'ataps', 'ataxy', 'atigi', 'atilt', 'atimy', 'atlas', 'atman', 'atmas', 'atmos', 'atocs', 'atoke', 'atoks', 'atoll', 'atoms', 'atomy', 'atone', 'atony', 'atopy', 'atria', 'atrip', 'attap', 'attar', 'attas', 'atter', 'attic', 'atuas', 'aucht', 'audad', 'audax', 'audio', 'audit', 'augen', 'auger', 'auges', 'aught', 'augur', 'aulas', 'aulic', 'auloi', 'aulos', 'aumil', 'aunes', 'aunts', 'aunty', 'aurae', 'aural', 'aurar', 'auras', 'aurei', 'aures', 'auric', 'auris', 'aurum', 'autos', 'auxin', 'avail', 'avale', 'avant', 'avast', 'avels', 'avens', 'avers', 'avert', 'avgas', 'avian', 'avine', 'avion', 'avise', 'aviso', 'avize', 'avoid', 'avows', 'avyze', 'await', 'awake', 'award', 'aware', 'awari', 'awarn', 'awash', 'awato', 'awave', 'aways', 'awdls', 'aweel', 'aweto', 'awful', 'awing', 'awkin', 'awmry', 'awned', 'awner', 'awoke', 'awols', 'awork', 'axels', 'axial', 'axile', 'axils', 'axing', 'axiom', 'axion', 'axite', 'axled', 'axles', 'axman', 'axmen', 'axoid', 'axone', 'axons', 'ayahs', 'ayaya', 'ayelp', 'aygre', 'ayins', 'aymag', 'ayont', 'ayres', 'ayrie', 'azans', 'azide', 'azido', 'azine', 'azlon', 'azoic', 'azole', 'azons', 'azote', 'azoth', 'azuki', 'azure', 'azurn', 'azury', 'azygy', 'azyme', 'azyms', 'baaed', 'baals', 'baaps', 'babas', 'babby', 'babel', 'babes', 'babka', 'baboo', 'babul', 'babus', 'bacca', 'bacco', 'baccy', 'bacha', 'bachs', 'backs', 'backy', 'bacne', 'bacon', 'badam', 'baddy', 'badge', 'badly', 'baels', 'baffs', 'baffy', 'bafta', 'bafts', 'bagel', 'baggy', 'baghs', 'bagie', 'bagsy', 'bagua', 'bahts', 'bahus', 'bahut', 'baiks', 'baile', 'bails', 'bairn', 'baisa', 'baith', 'baits', 'baiza', 'baize', 'bajan', 'bajra', 'bajri', 'bajus', 'baked', 'baken', 'baker', 'bakes', 'bakra', 'balas', 'balds', 'baldy', 'baled', 'baler', 'bales', 'balks', 'balky', 'ballo', 'balls', 'bally', 'balms', 'balmy', 'baloi', 'balon', 'baloo', 'balot', 'balsa', 'balti', 'balun', 'balus', 'balut', 'bamas', 'bambi', 'bamma', 'bammy', 'banak', 'banal', 'banco', 'bancs', 'banda', 'bandh', 'bands', 'bandy', 'baned', 'banes', 'bangs', 'bania', 'banjo', 'banks', 'banky', 'banns', 'bants', 'bantu', 'banty', 'bantz', 'banya', 'baons', 'baozi', 'bappu', 'bapus', 'barbe', 'barbs', 'barby', 'barca', 'barde', 'bardo', 'bards', 'bardy', 'bared', 'barer', 'bares', 'barfi', 'barfs', 'barfy', 'barge', 'baric', 'barks', 'barky', 'barms', 'barmy', 'barns', 'barny', 'baron', 'barps', 'barra', 'barre', 'barro', 'barry', 'barye', 'basal', 'basan', 'basas', 'based', 'basen', 'baser', 'bases', 'basha', 'basho', 'basic', 'basij', 'basil', 'basin', 'basis', 'basks', 'bason', 'basse', 'bassi', 'basso', 'bassy', 'basta', 'baste', 'basti', 'basto', 'basts', 'batch', 'bated', 'bates', 'bathe', 'baths', 'batik', 'baton', 'batos', 'batta', 'batts', 'battu', 'batty', 'bauds', 'bauks', 'baulk', 'baurs', 'bavin', 'bawds', 'bawdy', 'bawks', 'bawls', 'bawns', 'bawrs', 'bawty', 'bayas', 'bayed', 'bayer', 'bayes', 'bayle', 'bayou', 'bayts', 'bazar', 'bazas', 'bazoo', 'bball', 'bdays', 'beach', 'beads', 'beady', 'beaks', 'beaky', 'beals', 'beams', 'beamy', 'beano', 'beans', 'beany', 'beard', 'beare', 'bears', 'beast', 'beath', 'beats', 'beaty', 'beaus', 'beaut', 'beaux', 'bebop', 'becap', 'becke', 'becks', 'bedad', 'bedel', 'bedes', 'bedew', 'bedim', 'bedye', 'beech', 'beedi', 'beefs', 'beefy', 'beeps', 'beers', 'beery', 'beets', 'befit', 'befog', 'begad', 'began', 'begar', 'begat', 'begem', 'beget', 'begin', 'begob', 'begot', 'begum', 'begun', 'beige', 'beigy', 'being', 'beins', 'beira', 'beisa', 'bekah', 'belah', 'belar', 'belay', 'belch', 'belee', 'belga', 'belie', 'belit', 'belle', 'belli', 'bello', 'bells', 'belly', 'belon', 'below', 'belts', 'belve', 'bemad', 'bemas', 'bemix', 'bemud', 'bench', 'bends', 'bendy', 'benes', 'benet', 'benga', 'benis', 'benji', 'benne', 'benni', 'benny', 'bento', 'bents', 'benty', 'bepat', 'beray', 'beres', 'beret', 'bergs', 'berko', 'berks', 'berme', 'berms', 'berob', 'berry', 'berth', 'beryl', 'besat', 'besaw', 'besee', 'beses', 'beset', 'besit', 'besom', 'besot', 'besti', 'bests', 'betas', 'beted', 'betel', 'betes', 'beths', 'betid', 'beton', 'betta', 'betty', 'bevan', 'bevel', 'bever', 'bevor', 'bevue', 'bevvy', 'bewdy', 'bewet', 'bewig', 'bezel', 'bezes', 'bezil', 'bezzy', 'bhais', 'bhaji', 'bhang', 'bhats', 'bhava', 'bhels', 'bhoot', 'bhuna', 'bhuts', 'biach', 'biali', 'bialy', 'bibbs', 'bibes', 'bibis', 'bible', 'biccy', 'bicep', 'bices', 'bicky', 'biddy', 'bided', 'bider', 'bides', 'bidet', 'bidis', 'bidon', 'bidri', 'bield', 'biers', 'biffo', 'biffs', 'biffy', 'bifid', 'bigae', 'biggs', 'biggy', 'bigha', 'bight', 'bigly', 'bigos', 'bigot', 'bihon', 'bijou', 'biked', 'biker', 'bikes', 'bikie', 'bikky', 'bilal', 'bilat', 'bilbo', 'bilby', 'biled', 'biles', 'bilge', 'bilgy', 'bilks', 'bills', 'billy', 'bimah', 'bimas', 'bimbo', 'binal', 'bindi', 'binds', 'biner', 'bines', 'binge', 'bingo', 'bings', 'bingy', 'binit', 'binks', 'binky', 'bints', 'biogs', 'biome', 'bions', 'biont', 'biose', 'biota', 'biped', 'bipod', 'bippy', 'birch', 'birdo', 'birds', 'biris', 'birks', 'birle', 'birls', 'biros', 'birrs', 'birse', 'birsy', 'birth', 'birze', 'birzz', 'bises', 'bisks', 'bisom', 'bison', 'bitch', 'biter', 'bites', 'bitey', 'bitos', 'bitou', 'bitsy', 'bitte', 'bitts', 'bitty', 'bivia', 'bivvy', 'bizes', 'bizzo', 'bizzy', 'blabs', 'black', 'blade', 'blads', 'blady', 'blaer', 'blaes', 'blaff', 'blags', 'blahs', 'blain', 'blame', 'blams', 'blanc', 'bland', 'blank', 'blare', 'blart', 'blase', 'blash', 'blast', 'blate', 'blats', 'blatt', 'blaud', 'blawn', 'blaws', 'blays', 'blaze', 'bleah', 'bleak', 'blear', 'bleat', 'blebs', 'blech', 'bleed', 'bleep', 'blees', 'blend', 'blent', 'blert', 'bless', 'blest', 'blets', 'bleys', 'blimp', 'blimy', 'blind', 'bling', 'blini', 'blink', 'blins', 'bliny', 'blips', 'bliss', 'blist', 'blite', 'blits', 'blitz', 'blive', 'bloat', 'blobs', 'block', 'blocs', 'blogs', 'bloke', 'blond', 'blonx', 'blood', 'blook', 'bloom', 'bloop', 'blore', 'blots', 'blown', 'blows', 'blowy', 'blubs', 'blude', 'bluds', 'bludy', 'blued', 'bluer', 'blues', 'bluet', 'bluey', 'bluff', 'bluid', 'blume', 'blunk', 'blunt', 'blurb', 'blurs', 'blurt', 'blush', 'blype', 'boabs', 'boaks', 'board', 'boars', 'boart', 'boast', 'boats', 'boaty', 'bobac', 'bobak', 'bobas', 'bobby', 'bobol', 'bobos', 'bocca', 'bocce', 'bocci', 'boche', 'bocks', 'boded', 'bodes', 'bodge', 'bodgy', 'bodhi', 'bodle', 'bodoh', 'boeps', 'boers', 'boeti', 'boets', 'boeuf', 'boffo', 'boffs', 'bogan', 'bogey', 'boggy', 'bogie', 'bogle', 'bogue', 'bogus', 'bohea', 'bohos', 'boils', 'boing', 'boink', 'boite', 'boked', 'bokeh', 'bokes', 'bokos', 'bolar', 'bolas', 'boldo', 'bolds', 'boles', 'bolet', 'bolix', 'bolks', 'bolls', 'bolos', 'bolts', 'bolus', 'bomas', 'bombe', 'bombo', 'bombs', 'bomoh', 'bomor', 'bonce', 'bonds', 'boned', 'boner', 'bones', 'boney', 'bongo', 'bongs', 'bonie', 'bonks', 'bonne', 'bonny', 'bonum', 'bonus', 'bonza', 'bonze', 'booai', 'booay', 'boobs', 'booby', 'boody', 'booed', 'boofy', 'boogy', 'boohs', 'books', 'booky', 'bools', 'booms', 'boomy', 'boong', 'boons', 'boord', 'boors', 'boose', 'boost', 'booth', 'boots', 'booty', 'booze', 'boozy', 'boppy', 'borak', 'boral', 'boras', 'borax', 'borde', 'bords', 'bored', 'boree', 'borek', 'borel', 'borer', 'bores', 'borgo', 'boric', 'borks', 'borms', 'borna', 'borne', 'boron', 'borts', 'borty', 'bortz', 'bosey', 'bosie', 'bosks', 'bosky', 'bosom', 'boson', 'bossa', 'bossy', 'bosun', 'botas', 'botch', 'boteh', 'botel', 'botes', 'botew', 'bothy', 'botos', 'botte', 'botts', 'botty', 'bouge', 'bough', 'bouks', 'boule', 'boult', 'bound', 'bouns', 'bourd', 'bourg', 'bourn', 'bouse', 'bousy', 'bouts', 'boutu', 'bovid', 'bowat', 'bowed', 'bowel', 'bower', 'bowes', 'bowet', 'bowie', 'bowls', 'bowne', 'bowrs', 'bowse', 'boxed', 'boxen', 'boxer', 'boxes', 'boxla', 'boxty', 'boyar', 'boyau', 'boyed', 'boyey', 'boyfs', 'boygs', 'boyla', 'boyly', 'boyos', 'boysy', 'bozos', 'braai', 'brace', 'brach', 'brack', 'bract', 'brads', 'braes', 'brags', 'brahs', 'braid', 'brail', 'brain', 'brake', 'braks', 'braky', 'brame', 'brand', 'brane', 'brank', 'brans', 'brant', 'brash', 'brass', 'brast', 'brats', 'brava', 'brave', 'bravi', 'bravo', 'brawl', 'brawn', 'braws', 'braxy', 'brays', 'braza', 'braze', 'bread', 'break', 'bream', 'brede', 'breds', 'breed', 'breem', 'breer', 'brees', 'breid', 'breis', 'breme', 'brens', 'brent', 'brere', 'brers', 'breve', 'brews', 'breys', 'briar', 'bribe', 'brick', 'bride', 'brief', 'brier', 'bries', 'brigs', 'briki', 'briks', 'brill', 'brims', 'brine', 'bring', 'brink', 'brins', 'briny', 'brios', 'brise', 'brisk', 'briss', 'brith', 'brits', 'britt', 'brize', 'broad', 'broch', 'brock', 'brods', 'brogh', 'brogs', 'broil', 'broke', 'brome', 'bromo', 'bronc', 'brond', 'brood', 'brook', 'brool', 'broom', 'broos', 'brose', 'brosy', 'broth', 'brown', 'brows', 'bruck', 'brugh', 'bruhs', 'bruin', 'bruit', 'bruja', 'brujo', 'brule', 'brume', 'brung', 'brunt', 'brush', 'brusk', 'brust', 'brute', 'bruts', 'bruvs', 'buats', 'buaze', 'bubal', 'bubas', 'bubba', 'bubbe', 'bubby', 'bubus', 'buchu', 'bucko', 'bucks', 'bucku', 'budas', 'buddy', 'buded', 'budes', 'budge', 'budis', 'budos', 'buena', 'buffa', 'buffe', 'buffi', 'buffo', 'buffs', 'buffy', 'bufos', 'bufty', 'bugan', 'buggy', 'bugle', 'buhls', 'buhrs', 'buiks', 'build', 'built', 'buist', 'bukes', 'bukos', 'bulbs', 'bulge', 'bulgy', 'bulks', 'bulky', 'bulla', 'bulls', 'bully', 'bulse', 'bumbo', 'bumfs', 'bumph', 'bumps', 'bumpy', 'bunas', 'bunce', 'bunch', 'bunco', 'bunde', 'bundh', 'bunds', 'bundt', 'bundu', 'bundy', 'bungs', 'bungy', 'bunia', 'bunje', 'bunjy', 'bunko', 'bunks', 'bunns', 'bunny', 'bunts', 'bunty', 'bunya', 'buoys', 'buppy', 'buran', 'buras', 'burbs', 'burds', 'buret', 'burfi', 'burgh', 'burgs', 'burin', 'burka', 'burke', 'burks', 'burls', 'burly', 'burns', 'burnt', 'buroo', 'burps', 'burqa', 'burra', 'burro', 'burrs', 'burry', 'bursa', 'burse', 'burst', 'busby', 'bused', 'buses', 'bushy', 'busks', 'busky', 'bussu', 'busti', 'busts', 'busty', 'butch', 'buteo', 'butes', 'butle', 'butoh', 'butte', 'butts', 'butty', 'butut', 'butyl', 'buxom', 'buyer', 'buyin', 'buzzy', 'bwana', 'bwazi', 'byded', 'bydes', 'byked', 'bykes', 'bylaw', 'byres', 'byrls', 'byssi', 'bytes', 'byway', 'caaed', 'cabal', 'cabas', 'cabby', 'caber', 'cabin', 'cable', 'cabob', 'caboc', 'cabre', 'cacao', 'cacas', 'cache', 'cacks', 'cacky', 'cacti', 'caddy', 'cadee', 'cades', 'cadet', 'cadge', 'cadgy', 'cadie', 'cadis', 'cadre', 'caeca', 'caese', 'cafes', 'caffe', 'caffs', 'caged', 'cager', 'cages', 'cagey', 'cagot', 'cahow', 'caids', 'cains', 'caird', 'cairn', 'cajon', 'cajun', 'caked', 'cakes', 'cakey', 'calfs', 'calid', 'calif', 'calix', 'calks', 'calla', 'calle', 'calls', 'calms', 'calmy', 'calos', 'calpa', 'calps', 'calve', 'calyx', 'caman', 'camas', 'camel', 'cameo', 'cames', 'camis', 'camos', 'campi', 'campo', 'camps', 'campy', 'camus', 'canal', 'cando', 'candy', 'caned', 'caneh', 'caner', 'canes', 'cangs', 'canid', 'canna', 'canns', 'canny', 'canoe', 'canon', 'canso', 'canst', 'canti', 'canto', 'cants', 'canty', 'capas', 'capax', 'caped', 'caper', 'capes', 'capex', 'caphs', 'capiz', 'caple', 'capon', 'capos', 'capot', 'capri', 'capul', 'caput', 'carap', 'carat', 'carbo', 'carbs', 'carby', 'cardi', 'cards', 'cardy', 'cared', 'carer', 'cares', 'caret', 'carex', 'cargo', 'carks', 'carle', 'carls', 'carne', 'carns', 'carny', 'carob', 'carol', 'carom', 'caron', 'carpe', 'carpi', 'carps', 'carrs', 'carry', 'carse', 'carta', 'carte', 'carts', 'carve', 'carvy', 'casas', 'casco', 'cased', 'caser', 'cases', 'casks', 'casky', 'caste', 'casts', 'casus', 'catch', 'cater', 'cates', 'catty', 'cauda', 'cauks', 'cauld', 'caulk', 'cauls', 'caums', 'caups', 'cauri', 'causa', 'cause', 'cavas', 'caved', 'cavel', 'caver', 'caves', 'cavie', 'cavil', 'cavus', 'cawed', 'cawks', 'caxon', 'cease', 'ceaze', 'cebid', 'cecal', 'cecum', 'cedar', 'ceded', 'ceder', 'cedes', 'cedis', 'ceiba', 'ceili', 'ceils', 'celeb', 'cella', 'celli', 'cello', 'cells', 'celly', 'celom', 'celts', 'cense', 'cento', 'cents', 'centu', 'ceorl', 'cepes', 'cerci', 'cered', 'ceres', 'cerge', 'ceria', 'ceric', 'cerne', 'ceroc', 'ceros', 'certs', 'certy', 'cesse', 'cesta', 'cesti', 'cetes', 'cetyl', 'cezve', 'chaap', 'chaat', 'chace', 'chack', 'chaco', 'chado', 'chads', 'chafe', 'chaff', 'chaft', 'chain', 'chair', 'chais', 'chalk', 'chals', 'champ', 'chams', 'chana', 'chang', 'chank', 'chant', 'chaos', 'chape', 'chaps', 'chapt', 'chara', 'chard', 'chare', 'chark', 'charm', 'charr', 'chars', 'chart', 'chary', 'chase', 'chasm', 'chats', 'chava', 'chave', 'chavs', 'chawk', 'chawl', 'chaws', 'chaya', 'chays', 'cheap', 'cheat', 'cheba', 'check', 'chedi', 'cheeb', 'cheek', 'cheep', 'cheer', 'cheet', 'chefs', 'cheka', 'chela', 'chelp', 'chemo', 'chems', 'chere', 'chert', 'chess', 'chest', 'cheth', 'chevy', 'chews', 'chewy', 'chiao', 'chias', 'chiba', 'chibs', 'chica', 'chich', 'chick', 'chico', 'chics', 'chide', 'chief', 'chiel', 'chiko', 'chiks', 'child', 'chile', 'chili', 'chill', 'chimb', 'chime', 'chimo', 'chimp', 'china', 'chine', 'ching', 'chink', 'chino', 'chins', 'chips', 'chirk', 'chirl', 'chirm', 'chiro', 'chirp', 'chirr', 'chirt', 'chiru', 'chiti', 'chits', 'chiva', 'chive', 'chivs', 'chivy', 'chizz', 'chock', 'choco', 'chocs', 'chode', 'chogs', 'choil', 'choir', 'choke', 'choko', 'choky', 'chola', 'choli', 'cholo', 'chomp', 'chons', 'choof', 'chook', 'choom', 'choon', 'chops', 'chord', 'chore', 'chose', 'choss', 'chota', 'chott', 'chout', 'choux', 'chowk', 'chows', 'chubs', 'chuck', 'chufa', 'chuff', 'chugs', 'chump', 'chums', 'chunk', 'churl', 'churn', 'churr', 'chuse', 'chute', 'chuts', 'chyle', 'chyme', 'chynd', 'cibol', 'cided', 'cider', 'cides', 'ciels', 'cigar', 'ciggy', 'cilia', 'cills', 'cimar', 'cimex', 'cinch', 'cinct', 'cines', 'cinqs', 'cions', 'cippi', 'circa', 'circs', 'cires', 'cirls', 'cirri', 'cisco', 'cissy', 'cists', 'cital', 'cited', 'citee', 'citer', 'cites', 'cives', 'civet', 'civic', 'civie', 'civil', 'civvy', 'clach', 'clack', 'clade', 'clads', 'claes', 'clags', 'claim', 'clair', 'clame', 'clamp', 'clams', 'clang', 'clank', 'clans', 'claps', 'clapt', 'claro', 'clart', 'clary', 'clash', 'clasp', 'class', 'clast', 'clats', 'claut', 'clave', 'clavi', 'claws', 'clays', 'clean', 'clear', 'cleat', 'cleck', 'cleek', 'cleep', 'clefs', 'cleft', 'clegs', 'cleik', 'clems', 'clepe', 'clept', 'clerk', 'cleve', 'clews', 'click', 'clied', 'clies', 'cliff', 'clift', 'climb', 'clime', 'cline', 'cling', 'clink', 'clint', 'clipe', 'clips', 'clipt', 'clits', 'cloak', 'cloam', 'clock', 'clods', 'cloff', 'clogs', 'cloke', 'clomb', 'clomp', 'clone', 'clonk', 'clons', 'cloop', 'cloot', 'clops', 'close', 'clote', 'cloth', 'clots', 'cloud', 'clour', 'clous', 'clout', 'clove', 'clown', 'clows', 'cloye', 'cloys', 'cloze', 'clubs', 'cluck', 'clued', 'clues', 'cluey', 'clump', 'clung', 'clunk', 'clype', 'cnida', 'coach', 'coact', 'coady', 'coala', 'coals', 'coaly', 'coapt', 'coarb', 'coast', 'coate', 'coati', 'coats', 'cobbs', 'cobby', 'cobia', 'coble', 'cobot', 'cobra', 'cobza', 'cocas', 'cocci', 'cocco', 'cocks', 'cocky', 'cocoa', 'cocos', 'cocus', 'codas', 'codec', 'coded', 'coden', 'coder', 'codes', 'codex', 'codon', 'coeds', 'coffs', 'cogie', 'cogon', 'cogue', 'cohab', 'cohen', 'cohoe', 'cohog', 'cohos', 'coifs', 'coign', 'coils', 'coins', 'coirs', 'coits', 'coked', 'cokes', 'cokey', 'colas', 'colby', 'colds', 'coled', 'coles', 'coley', 'colic', 'colin', 'colle', 'colls', 'colly', 'colog', 'colon', 'color', 'colts', 'colza', 'comae', 'comal', 'comas', 'combe', 'combi', 'combo', 'combs', 'comby', 'comer', 'comes', 'comet', 'comfy', 'comic', 'comix', 'comma', 'comme', 'commo', 'comms', 'commy', 'compo', 'comps', 'compt', 'comte', 'comus', 'conch', 'condo', 'coned', 'cones', 'conex', 'coney', 'confs', 'conga', 'conge', 'congo', 'conia', 'conic', 'conin', 'conks', 'conky', 'conne', 'conns', 'conte', 'conto', 'conus', 'convo', 'cooch', 'cooed', 'cooee', 'cooer', 'cooey', 'coofs', 'cooks', 'cooky', 'cools', 'cooly', 'coomb', 'cooms', 'coomy', 'coons', 'coops', 'coopt', 'coost', 'coots', 'cooty', 'cooze', 'copal', 'copay', 'coped', 'copen', 'coper', 'copes', 'copha', 'coppy', 'copra', 'copse', 'copsy', 'coqui', 'coral', 'coram', 'corbe', 'corby', 'corda', 'cords', 'cored', 'corer', 'cores', 'corey', 'corgi', 'coria', 'corks', 'corky', 'corms', 'corni', 'corno', 'corns', 'cornu', 'corny', 'corps', 'corse', 'corso', 'cosec', 'cosed', 'coses', 'coset', 'cosey', 'cosie', 'costa', 'coste', 'costs', 'cotan', 'cotch', 'coted', 'cotes', 'coths', 'cotta', 'cotts', 'couch', 'coude', 'cough', 'could', 'count', 'coupe', 'coups', 'courb', 'courd', 'coure', 'cours', 'court', 'couta', 'couth', 'coved', 'coven', 'cover', 'coves', 'covet', 'covey', 'covin', 'cowal', 'cowan', 'cowed', 'cower', 'cowks', 'cowls', 'cowps', 'cowry', 'coxae', 'coxal', 'coxed', 'coxes', 'coxib', 'coyau', 'coyed', 'coyer', 'coyly', 'coypu', 'cozed', 'cozen', 'cozes', 'cozey', 'cozie', 'craal', 'crabs', 'crack', 'craft', 'crags', 'craic', 'craig', 'crake', 'crame', 'cramp', 'crams', 'crane', 'crank', 'crans', 'crape', 'craps', 'crapy', 'crare', 'crash', 'crass', 'crate', 'crave', 'crawl', 'craws', 'crays', 'craze', 'crazy', 'creak', 'cream', 'credo', 'creds', 'creed', 'creek', 'creel', 'creep', 'crees', 'crein', 'crema', 'creme', 'crems', 'crena', 'crepe', 'creps', 'crept', 'crepy', 'cress', 'crest', 'crewe', 'crews', 'crias', 'cribo', 'cribs', 'crick', 'cried', 'crier', 'cries', 'crime', 'crimp', 'crims', 'crine', 'crink', 'crins', 'crios', 'cripe', 'crips', 'crise', 'crisp', 'criss', 'crith', 'crits', 'croak', 'croci', 'crock', 'crocs', 'croft', 'crogs', 'cromb', 'crome', 'crone', 'cronk', 'crons', 'crony', 'crook', 'crool', 'croon', 'crops', 'crore', 'cross', 'crost', 'croup', 'crout', 'crowd', 'crowl', 'crown', 'crows', 'croze', 'cruck', 'crude', 'crudo', 'cruds', 'crudy', 'cruel', 'crues', 'cruet', 'cruft', 'crumb', 'crump', 'crunk', 'cruor', 'crura', 'cruse', 'crush', 'crust', 'crusy', 'cruve', 'crwth', 'cryer', 'cryne', 'crypt', 'ctene', 'cubby', 'cubeb', 'cubed', 'cuber', 'cubes', 'cubic', 'cubit', 'cucks', 'cudda', 'cuddy', 'cueca', 'cuffo', 'cuffs', 'cuifs', 'cuing', 'cuish', 'cuits', 'cukes', 'culch', 'culet', 'culex', 'culls', 'cully', 'culms', 'culpa', 'culti', 'cults', 'culty', 'cumec', 'cumin', 'cundy', 'cunei', 'cunit', 'cunny', 'cunts', 'cupel', 'cupid', 'cuppa', 'cuppy', 'cupro', 'curat', 'curbs', 'curch', 'curds', 'curdy', 'cured', 'curer', 'cures', 'curet', 'curfs', 'curia', 'curie', 'curio', 'curli', 'curls', 'curly', 'curns', 'curny', 'currs', 'curry', 'curse', 'cursi', 'curst', 'curve', 'curvy', 'cusec', 'cushy', 'cusks', 'cusps', 'cuspy', 'cusso', 'cusum', 'cutch', 'cuter', 'cutes', 'cutey', 'cutie', 'cutin', 'cutis', 'cutto', 'cutty', 'cutup', 'cuvee', 'cuzes', 'cwtch', 'cyano', 'cyans', 'cyber', 'cycad', 'cycas', 'cycle', 'cyclo', 'cyder', 'cylix', 'cymae', 'cymar', 'cymas', 'cymes', 'cymol', 'cynic', 'cysts', 'cytes', 'cyton', 'czars', 'daals', 'dabba', 'daces', 'dacha', 'dacks', 'dadah', 'dadas', 'daddy', 'dadis', 'dadla', 'dados', 'daffs', 'daffy', 'dagga', 'daggy', 'dagos', 'dahis', 'dahls', 'daiko', 'daily', 'daine', 'daint', 'dairy', 'daisy', 'daker', 'daled', 'dalek', 'dales', 'dalis', 'dalle', 'dally', 'dalts', 'daman', 'damar', 'dames', 'damme', 'damna', 'damns', 'damps', 'dampy', 'dance', 'dancy', 'danda', 'dandy', 'dangs', 'danio', 'danks', 'danny', 'danse', 'dants', 'dappy', 'daraf', 'darbs', 'darcy', 'dared', 'darer', 'dares', 'darga', 'dargs', 'daric', 'daris', 'darks', 'darky', 'darls', 'darns', 'darre', 'darts', 'darzi', 'dashi', 'dashy', 'datal', 'dated', 'dater', 'dates', 'datil', 'datos', 'datto', 'datum', 'daube', 'daubs', 'dauby', 'dauds', 'dault', 'daunt', 'daurs', 'dauts', 'daven', 'davit', 'dawah', 'dawds', 'dawed', 'dawen', 'dawgs', 'dawks', 'dawns', 'dawts', 'dayal', 'dayan', 'daych', 'daynt', 'dazed', 'dazer', 'dazes', 'dbags', 'deads', 'deair', 'deals', 'dealt', 'deans', 'deare', 'dearn', 'dears', 'deary', 'deash', 'death', 'deave', 'deaws', 'deawy', 'debag', 'debar', 'debby', 'debel', 'debes', 'debit', 'debts', 'debud', 'debug', 'debur', 'debus', 'debut', 'debye', 'decad', 'decaf', 'decal', 'decan', 'decay', 'decim', 'decko', 'decks', 'decor', 'decos', 'decoy', 'decry', 'decyl', 'dedal', 'deeds', 'deedy', 'deely', 'deems', 'deens', 'deeps', 'deere', 'deers', 'deets', 'deeve', 'deevs', 'defat', 'defer', 'deffo', 'defis', 'defog', 'degas', 'degum', 'degus', 'deice', 'deids', 'deify', 'deign', 'deils', 'deink', 'deism', 'deist', 'deity', 'deked', 'dekes', 'dekko', 'delay', 'deled', 'deles', 'delfs', 'delft', 'delis', 'della', 'dells', 'delly', 'delos', 'delph', 'delta', 'delts', 'delve', 'deman', 'demes', 'demic', 'demit', 'demob', 'demoi', 'demon', 'demos', 'demot', 'dempt', 'demur', 'denar', 'denay', 'dench', 'denes', 'denet', 'denim', 'denis', 'dense', 'dente', 'dents', 'deoch', 'deoxy', 'depot', 'depth', 'derat', 'deray', 'derby', 'dered', 'deres', 'derig', 'derma', 'derms', 'derns', 'derny', 'deros', 'derpy', 'derro', 'derry', 'derth', 'dervs', 'desex', 'deshi', 'desis', 'desks', 'desse', 'detag', 'deter', 'detox', 'deuce', 'devas', 'devel', 'devil', 'devis', 'devon', 'devos', 'devot', 'dewan', 'dewar', 'dewax', 'dewed', 'dexes', 'dexie', 'dexys', 'dhaba', 'dhaks', 'dhals', 'dhikr', 'dhobi', 'dhole', 'dholl', 'dhols', 'dhoni', 'dhoti', 'dhows', 'dhuti', 'diact', 'dials', 'diana', 'diane', 'diary', 'diazo', 'dibbs', 'diced', 'dicer', 'dices', 'dicey', 'dicht', 'dicks', 'dicky', 'dicot', 'dicta', 'dicto', 'dicts', 'dictu', 'dicty', 'diddy', 'didie', 'didis', 'didos', 'didst', 'diebs', 'diels', 'diene', 'diets', 'diffs', 'dight', 'digit', 'dikas', 'diked', 'diker', 'dikes', 'dikey', 'dildo', 'dilli', 'dills', 'dilly', 'dimbo', 'dimer', 'dimes', 'dimly', 'dimps', 'dinar', 'dined', 'diner', 'dines', 'dinge', 'dingo', 'dings', 'dingy', 'dinic', 'dinks', 'dinky', 'dinlo', 'dinna', 'dinos', 'dints', 'dioch', 'diode', 'diols', 'diota', 'dippy', 'dipso', 'diram', 'direr', 'dirge', 'dirke', 'dirks', 'dirls', 'dirts', 'dirty', 'disas', 'disci', 'disco', 'discs', 'dishy', 'disks', 'disme', 'dital', 'ditas', 'ditch', 'dited', 'dites', 'ditsy', 'ditto', 'ditts', 'ditty', 'ditzy', 'divan', 'divas', 'dived', 'diver', 'dives', 'divey', 'divis', 'divna', 'divos', 'divot', 'divvy', 'diwan', 'dixie', 'dixit', 'diyas', 'dizen', 'dizzy', 'djinn', 'djins', 'doabs', 'doats', 'dobby', 'dobes', 'dobie', 'dobla', 'doble', 'dobra', 'dobro', 'docht', 'docks', 'docos', 'docus', 'doddy', 'dodge', 'dodgy', 'dodos', 'doeks', 'doers', 'doest', 'doeth', 'doffs', 'dogal', 'dogan', 'doges', 'dogey', 'doggo', 'doggy', 'dogie', 'dogly', 'dogma', 'dohyo', 'doilt', 'doily', 'doing', 'doits', 'dojos', 'dolce', 'dolci', 'doled', 'dolee', 'doles', 'doley', 'dolia', 'dolie', 'dolls', 'dolly', 'dolma', 'dolor', 'dolos', 'dolts', 'domal', 'domed', 'domes', 'domic', 'donah', 'donas', 'donee', 'doner', 'donga', 'dongs', 'donko', 'donna', 'donne', 'donny', 'donor', 'donsy', 'donut', 'doobs', 'dooce', 'doody', 'doofs', 'dooks', 'dooky', 'doole', 'dools', 'dooly', 'dooms', 'doomy', 'doona', 'doorn', 'doors', 'doozy', 'dopas', 'doped', 'doper', 'dopes', 'dopey', 'doppe', 'dorad', 'dorba', 'dorbs', 'doree', 'dores', 'doric', 'doris', 'dorje', 'dorks', 'dorky', 'dorms', 'dormy', 'dorps', 'dorrs', 'dorsa', 'dorse', 'dorts', 'dorty', 'dosai', 'dosas', 'dosed', 'doseh', 'doser', 'doses', 'dosha', 'dotal', 'doted', 'doter', 'dotes', 'dotty', 'douar', 'doubt', 'douce', 'doucs', 'dough', 'douks', 'doula', 'douma', 'doums', 'doups', 'doura', 'douse', 'douts', 'doved', 'doven', 'dover', 'doves', 'dovie', 'dowak', 'dowar', 'dowds', 'dowdy', 'dowed', 'dowel', 'dower', 'dowfs', 'dowie', 'dowle', 'dowls', 'dowly', 'downa', 'downs', 'downy', 'dowps', 'dowry', 'dowse', 'dowts', 'doxed', 'doxes', 'doxie', 'doyen', 'doyly', 'dozed', 'dozen', 'dozer', 'dozes', 'drabs', 'drack', 'draco', 'draff', 'draft', 'drags', 'drail', 'drain', 'drake', 'drama', 'drams', 'drank', 'drant', 'drape', 'draps', 'drapy', 'drats', 'drave', 'drawl', 'drawn', 'draws', 'drays', 'dread', 'dream', 'drear', 'dreck', 'dreed', 'dreer', 'drees', 'dregs', 'dreks', 'drent', 'drere', 'dress', 'drest', 'dreys', 'dribs', 'drice', 'dried', 'drier', 'dries', 'drift', 'drill', 'drily', 'drink', 'drips', 'dript', 'drive', 'drock', 'droid', 'droil', 'droit', 'droke', 'drole', 'droll', 'drome', 'drone', 'drony', 'droob', 'droog', 'drook', 'drool', 'droop', 'drops', 'dropt', 'dross', 'drouk', 'drove', 'drown', 'drows', 'drubs', 'drugs', 'druid', 'drums', 'drunk', 'drupe', 'druse', 'drusy', 'druxy', 'dryad', 'dryas', 'dryer', 'dryly', 'dsobo', 'dsomo', 'duads', 'duals', 'duans', 'duars', 'dubbo', 'dubby', 'ducal', 'ducat', 'duces', 'duchy', 'ducks', 'ducky', 'ducti', 'ducts', 'duddy', 'duded', 'dudes', 'duels', 'duets', 'duett', 'duffs', 'dufus', 'duing', 'duits', 'dukas', 'duked', 'dukes', 'dukka', 'dukun', 'dulce', 'dules', 'dulia', 'dulls', 'dully', 'dulse', 'dumas', 'dumbo', 'dumbs', 'dumka', 'dumky', 'dummy', 'dumps', 'dumpy', 'dunam', 'dunce', 'dunch', 'dunes', 'dungs', 'dungy', 'dunks', 'dunno', 'dunny', 'dunsh', 'dunts', 'duomi', 'duomo', 'duped', 'duper', 'dupes', 'duple', 'duply', 'duppy', 'dural', 'duras', 'dured', 'dures', 'durgy', 'durns', 'duroc', 'duros', 'duroy', 'durra', 'durrs', 'durry', 'durst', 'durum', 'durzi', 'dusks', 'dusky', 'dusts', 'dusty', 'dutch', 'duvet', 'duxes', 'dwaal', 'dwale', 'dwalm', 'dwams', 'dwamy', 'dwang', 'dwarf', 'dwaum', 'dweeb', 'dwell', 'dwelt', 'dwile', 'dwine', 'dyads', 'dyers', 'dying', 'dyked', 'dykes', 'dykey', 'dykon', 'dynel', 'dynes', 'dynos', 'dzhos', 'eager', 'eagle', 'eagly', 'eagre', 'ealed', 'eales', 'eaned', 'eards', 'eared', 'earls', 'early', 'earns', 'earnt', 'earst', 'earth', 'eased', 'easel', 'easer', 'eases', 'easle', 'easts', 'eaten', 'eater', 'eathe', 'eatin', 'eaved', 'eaver', 'eaves', 'ebank', 'ebbed', 'ebbet', 'ebena', 'ebene', 'ebike', 'ebons', 'ebony', 'ebook', 'ecads', 'ecard', 'ecash', 'eched', 'eches', 'echos', 'ecigs', 'eclat', 'ecole', 'ecrus', 'edema', 'edged', 'edger', 'edges', 'edict', 'edify', 'edile', 'edits', 'educe', 'educt', 'eejit', 'eensy', 'eerie', 'eeven', 'eever', 'eevns', 'effed', 'effer', 'efits', 'egads', 'egers', 'egest', 'eggar', 'egged', 'egger', 'egmas', 'egret', 'ehing', 'eider', 'eidos', 'eight', 'eigne', 'eiked', 'eikon', 'eilds', 'eiron', 'eisel', 'eject', 'ejido', 'ekdam', 'eking', 'ekkas', 'elain', 'eland', 'elans', 'elate', 'elbow', 'elchi', 'elder', 'eldin', 'elect', 'eleet', 'elegy', 'elemi', 'elfed', 'elfin', 'eliad', 'elide', 'elint', 'elite', 'elmen', 'eloge', 'elogy', 'eloin', 'elope', 'elops', 'elpee', 'elsin', 'elude', 'elute', 'elvan', 'elven', 'elver', 'elves', 'emacs', 'email', 'embar', 'embay', 'embed', 'ember', 'embog', 'embow', 'embox', 'embus', 'emcee', 'emeer', 'emend', 'emerg', 'emery', 'emeus', 'emics', 'emirs', 'emits', 'emmas', 'emmer', 'emmet', 'emmew', 'emmys', 'emoji', 'emong', 'emote', 'emove', 'empts', 'empty', 'emule', 'emure', 'emyde', 'emyds', 'enact', 'enarm', 'enate', 'ended', 'ender', 'endew', 'endow', 'endue', 'enema', 'enemy', 'enews', 'enfix', 'eniac', 'enjoy', 'enlit', 'enmew', 'ennog', 'ennui', 'enoki', 'enols', 'enorm', 'enows', 'enrol', 'ensew', 'ensky', 'ensue', 'enter', 'entia', 'entre', 'entry', 'enure', 'enurn', 'envoi', 'envoy', 'enzym', 'eolid', 'eorls', 'eosin', 'epact', 'epees', 'epena', 'epene', 'ephah', 'ephas', 'ephod', 'ephor', 'epics', 'epoch', 'epode', 'epopt', 'epoxy', 'eppie', 'epris', 'equal', 'eques', 'equid', 'equip', 'erase', 'erbia', 'erect', 'erevs', 'ergon', 'ergos', 'ergot', 'erhus', 'erica', 'erick', 'erics', 'ering', 'erned', 'ernes', 'erode', 'erose', 'erred', 'error', 'erses', 'eruct', 'erugo', 'erupt', 'eruvs', 'erven', 'ervil', 'escar', 'escot', 'esile', 'eskar', 'esker', 'esnes', 'esrog', 'essay', 'esses', 'ester', 'estoc', 'estop', 'estro', 'etage', 'etape', 'etats', 'etens', 'ethal', 'ether', 'ethic', 'ethne', 'ethos', 'ethyl', 'etics', 'etnas', 'etrog', 'ettin', 'ettle', 'etude', 'etuis', 'etwee', 'etyma', 'eughs', 'euked', 'eupad', 'euros', 'eusol', 'evade', 'evegs', 'evens', 'event', 'evert', 'every', 'evets', 'evhoe', 'evict', 'evils', 'evite', 'evohe', 'evoke', 'ewers', 'ewest', 'ewhow', 'ewked', 'exact', 'exalt', 'exams', 'excel', 'exeat', 'execs', 'exeem', 'exeme', 'exert', 'exfil', 'exier', 'exies', 'exile', 'exine', 'exing', 'exist', 'exite', 'exits', 'exode', 'exome', 'exons', 'expat', 'expel', 'expos', 'extol', 'extra', 'exude', 'exuls', 'exult', 'exurb', 'eyass', 'eyers', 'eying', 'eyots', 'eyras', 'eyres', 'eyrie', 'eyrir', 'ezine', 'fabbo', 'fabby', 'fable', 'faced', 'facer', 'faces', 'facet', 'facey', 'facia', 'facie', 'facta', 'facto', 'facts', 'facty', 'faddy', 'faded', 'fader', 'fades', 'fadge', 'fados', 'faena', 'faery', 'faffs', 'faffy', 'faggy', 'fagin', 'fagot', 'faiks', 'fails', 'faine', 'fains', 'faint', 'faire', 'fairs', 'fairy', 'faith', 'faked', 'faker', 'fakes', 'fakey', 'fakie', 'fakir', 'falaj', 'fales', 'falls', 'false', 'falsy', 'famed', 'fames', 'fanal', 'fancy', 'fands', 'fanes', 'fanga', 'fango', 'fangs', 'fanks', 'fanny', 'fanon', 'fanos', 'fanum', 'faqir', 'farad', 'farce', 'farci', 'farcy', 'fards', 'fared', 'farer', 'fares', 'farle', 'farls', 'farms', 'faros', 'farro', 'farse', 'farts', 'fasci', 'fasti', 'fasts', 'fatal', 'fated', 'fates', 'fatly', 'fatso', 'fatty', 'fatwa', 'fauch', 'faugh', 'fauld', 'fault', 'fauna', 'fauns', 'faurd', 'faute', 'fauts', 'fauve', 'favas', 'favel', 'faver', 'faves', 'favor', 'favus', 'fawns', 'fawny', 'faxed', 'faxes', 'fayed', 'fayer', 'fayne', 'fayre', 'fazed', 'fazes', 'feals', 'feard', 'feare', 'fears', 'feart', 'fease', 'feast', 'feats', 'feaze', 'fecal', 'feces', 'fecht', 'fecit', 'fecks', 'fedai', 'fedex', 'feebs', 'feeds', 'feels', 'feely', 'feens', 'feers', 'feese', 'feeze', 'fehme', 'feign', 'feint', 'feist', 'felch', 'felid', 'felix', 'fella', 'fells', 'felly', 'felon', 'felts', 'felty', 'femal', 'femes', 'femic', 'femme', 'femmy', 'femur', 'fence', 'fends', 'fendy', 'fenis', 'fenks', 'fenny', 'fents', 'feods', 'feoff', 'feral', 'ferer', 'feres', 'feria', 'ferly', 'fermi', 'ferms', 'ferns', 'ferny', 'ferox', 'ferry', 'fesse', 'festa', 'fests', 'festy', 'fetal', 'fetas', 'fetch', 'feted', 'fetes', 'fetid', 'fetor', 'fetta', 'fetts', 'fetus', 'fetwa', 'feuar', 'feuds', 'feued', 'fever', 'fewer', 'feyed', 'feyer', 'feyly', 'fezes', 'fezzy', 'fiars', 'fiats', 'fiber', 'fibre', 'fibro', 'fices', 'fiche', 'fichu', 'ficin', 'ficos', 'ficta', 'ficus', 'fides', 'fidge', 'fidos', 'fidus', 'fiefs', 'field', 'fiend', 'fient', 'fiere', 'fieri', 'fiers', 'fiery', 'fiest', 'fifed', 'fifer', 'fifes', 'fifis', 'fifth', 'fifty', 'figgy', 'fight', 'figos', 'fiked', 'fikes', 'filar', 'filch', 'filed', 'filer', 'files', 'filet', 'filii', 'filks', 'fille', 'fillo', 'fills', 'filly', 'filmi', 'films', 'filmy', 'filon', 'filos', 'filth', 'filum', 'final', 'finca', 'finch', 'finds', 'fined', 'finer', 'fines', 'finis', 'finks', 'finny', 'finos', 'fiord', 'fiqhs', 'fique', 'fired', 'firer', 'fires', 'firie', 'firks', 'firma', 'firms', 'firni', 'firns', 'firry', 'first', 'firth', 'fiscs', 'fisho', 'fishy', 'fisks', 'fists', 'fisty', 'fitch', 'fitly', 'fitna', 'fitte', 'fitts', 'fiver', 'fives', 'fixed', 'fixer', 'fixes', 'fixie', 'fixit', 'fizzy', 'fjeld', 'fjord', 'flabs', 'flack', 'flaff', 'flags', 'flail', 'flair', 'flake', 'flaks', 'flaky', 'flame', 'flamm', 'flams', 'flamy', 'flane', 'flank', 'flans', 'flaps', 'flare', 'flary', 'flash', 'flask', 'flats', 'flava', 'flawn', 'flaws', 'flawy', 'flaxy', 'flays', 'fleam', 'fleas', 'fleck', 'fleek', 'fleer', 'flees', 'fleet', 'flegs', 'fleme', 'flesh', 'fleur', 'flews', 'flexi', 'flexo', 'fleys', 'flick', 'flics', 'flied', 'flier', 'flies', 'flimp', 'flims', 'fling', 'flint', 'flips', 'flirs', 'flirt', 'flisk', 'flite', 'flits', 'flitt', 'float', 'flobs', 'flock', 'flocs', 'floes', 'flogs', 'flong', 'flood', 'floor', 'flops', 'flora', 'flore', 'flors', 'flory', 'flosh', 'floss', 'flota', 'flote', 'flour', 'flout', 'flown', 'flows', 'flowy', 'flubs', 'flued', 'flues', 'fluey', 'fluff', 'fluid', 'fluke', 'fluky', 'flume', 'flump', 'flung', 'flunk', 'fluor', 'flurr', 'flush', 'flute', 'fluty', 'fluyt', 'flyby', 'flyer', 'flyin', 'flype', 'flyte', 'fnarr', 'foals', 'foams', 'foamy', 'focal', 'focus', 'foehn', 'fogey', 'foggy', 'fogie', 'fogle', 'fogos', 'fogou', 'fohns', 'foids', 'foils', 'foins', 'foist', 'folds', 'foley', 'folia', 'folic', 'folie', 'folio', 'folks', 'folky', 'folly', 'fomes', 'fonda', 'fonds', 'fondu', 'fones', 'fonio', 'fonly', 'fonts', 'foods', 'foody', 'fools', 'foots', 'footy', 'foram', 'foray', 'forbs', 'forby', 'force', 'fordo', 'fords', 'forel', 'fores', 'forex', 'forge', 'forgo', 'forks', 'forky', 'forma', 'forme', 'forms', 'forte', 'forth', 'forts', 'forty', 'forum', 'forza', 'forze', 'fossa', 'fosse', 'fouat', 'fouds', 'fouer', 'fouet', 'foule', 'fouls', 'found', 'fount', 'fours', 'fouth', 'fovea', 'fowls', 'fowth', 'foxed', 'foxes', 'foxie', 'foyer', 'foyle', 'foyne', 'frabs', 'frack', 'fract', 'frags', 'frail', 'fraim', 'frais', 'frame', 'franc', 'frank', 'frape', 'fraps', 'frass', 'frate', 'frati', 'frats', 'fraud', 'fraus', 'frays', 'freak', 'freed', 'freer', 'frees', 'freet', 'freit', 'fremd', 'frena', 'freon', 'frere', 'fresh', 'frets', 'friar', 'fribs', 'fried', 'frier', 'fries', 'frigs', 'frill', 'frise', 'frisk', 'frist', 'frita', 'frite', 'frith', 'frits', 'fritt', 'fritz', 'frize', 'frizz', 'frock', 'froes', 'frogs', 'fromm', 'frond', 'frons', 'front', 'froom', 'frore', 'frorn', 'frory', 'frosh', 'frost', 'froth', 'frown', 'frows', 'frowy', 'froyo', 'froze', 'frugs', 'fruit', 'frump', 'frush', 'frust', 'fryer', 'fubar', 'fubby', 'fubsy', 'fucks', 'fucus', 'fuddy', 'fudge', 'fudgy', 'fuels', 'fuero', 'fuffs', 'fuffy', 'fugal', 'fuggy', 'fugie', 'fugio', 'fugis', 'fugle', 'fugly', 'fugue', 'fugus', 'fujis', 'fulla', 'fulls', 'fully', 'fulth', 'fulwa', 'fumed', 'fumer', 'fumes', 'fumet', 'funda', 'fundi', 'fundo', 'funds', 'fundy', 'fungi', 'fungo', 'fungs', 'funic', 'funis', 'funks', 'funky', 'funny', 'funsy', 'funts', 'fural', 'furan', 'furca', 'furls', 'furol', 'furor', 'furos', 'furrs', 'furry', 'furth', 'furze', 'furzy', 'fused', 'fusee', 'fusel', 'fuses', 'fusil', 'fusks', 'fussy', 'fusts', 'fusty', 'futon', 'fuzed', 'fuzee', 'fuzes', 'fuzil', 'fuzzy', 'fyces', 'fyked', 'fykes', 'fyles', 'fyrds', 'fytte', 'gabba', 'gabby', 'gable', 'gaddi', 'gades', 'gadge', 'gadgy', 'gadid', 'gadis', 'gadje', 'gadjo', 'gadso', 'gaffe', 'gaffs', 'gaged', 'gager', 'gages', 'gaids', 'gaily', 'gains', 'gairs', 'gaita', 'gaits', 'gaitt', 'gajos', 'galah', 'galas', 'galax', 'galea', 'galed', 'gales', 'galia', 'galis', 'galls', 'gally', 'galop', 'galut', 'galvo', 'gamas', 'gamay', 'gamba', 'gambe', 'gambo', 'gambs', 'gamed', 'gamer', 'games', 'gamey', 'gamic', 'gamin', 'gamma', 'gamme', 'gammy', 'gamps', 'gamut', 'ganch', 'gandy', 'ganef', 'ganev', 'gangs', 'ganja', 'ganks', 'ganof', 'gants', 'gaols', 'gaped', 'gaper', 'gapes', 'gapos', 'gappy', 'garam', 'garba', 'garbe', 'garbo', 'garbs', 'garda', 'garde', 'gares', 'garis', 'garms', 'garni', 'garre', 'garri', 'garth', 'garum', 'gases', 'gashy', 'gasps', 'gaspy', 'gassy', 'gasts', 'gatch', 'gated', 'gater', 'gates', 'gaths', 'gator', 'gauch', 'gaucy', 'gauds', 'gaudy', 'gauge', 'gauje', 'gault', 'gaums', 'gaumy', 'gaunt', 'gaups', 'gaurs', 'gauss', 'gauze', 'gauzy', 'gavel', 'gavot', 'gawcy', 'gawds', 'gawks', 'gawky', 'gawps', 'gawsy', 'gayal', 'gayer', 'gayly', 'gazal', 'gazar', 'gazed', 'gazer', 'gazes', 'gazon', 'gazoo', 'geals', 'geans', 'geare', 'gears', 'geasa', 'geats', 'gebur', 'gecko', 'gecks', 'geeks', 'geeky', 'geeps', 'geese', 'geest', 'geist', 'geits', 'gelds', 'gelee', 'gelid', 'gelly', 'gelts', 'gemel', 'gemma', 'gemmy', 'gemot', 'genae', 'genal', 'genas', 'genes', 'genet', 'genic', 'genie', 'genii', 'genin', 'genio', 'genip', 'genny', 'genoa', 'genom', 'genre', 'genro', 'gents', 'genty', 'genua', 'genus', 'geode', 'geoid', 'gerah', 'gerbe', 'geres', 'gerle', 'germs', 'germy', 'gerne', 'gesse', 'gesso', 'geste', 'gests', 'getas', 'getup', 'geums', 'geyan', 'geyer', 'ghast', 'ghats', 'ghaut', 'ghazi', 'ghees', 'ghest', 'ghost', 'ghoul', 'ghusl', 'ghyll', 'giant', 'gibed', 'gibel', 'giber', 'gibes', 'gibli', 'gibus', 'giddy', 'gifts', 'gigas', 'gighe', 'gigot', 'gigue', 'gilas', 'gilds', 'gilet', 'gilia', 'gills', 'gilly', 'gilpy', 'gilts', 'gimel', 'gimme', 'gimps', 'gimpy', 'ginch', 'ginga', 'ginge', 'gings', 'ginks', 'ginny', 'ginzo', 'gipon', 'gippo', 'gippy', 'gipsy', 'girds', 'girlf', 'girls', 'girly', 'girns', 'giron', 'giros', 'girrs', 'girsh', 'girth', 'girts', 'gismo', 'gisms', 'gists', 'gitch', 'gites', 'giust', 'gived', 'given', 'giver', 'gives', 'gizmo', 'glace', 'glade', 'glads', 'glady', 'glaik', 'glair', 'glamp', 'glams', 'gland', 'glans', 'glare', 'glary', 'glass', 'glatt', 'glaum', 'glaur', 'glaze', 'glazy', 'gleam', 'glean', 'gleba', 'glebe', 'gleby', 'glede', 'gleds', 'gleed', 'gleek', 'glees', 'gleet', 'gleis', 'glens', 'glent', 'gleys', 'glial', 'glias', 'glibs', 'glide', 'gliff', 'glift', 'glike', 'glime', 'glims', 'glint', 'glisk', 'glits', 'glitz', 'gloam', 'gloat', 'globe', 'globi', 'globs', 'globy', 'glode', 'glogg', 'gloms', 'gloom', 'gloop', 'glops', 'glory', 'gloss', 'glost', 'glout', 'glove', 'glows', 'glowy', 'gloze', 'glued', 'gluer', 'glues', 'gluey', 'glugg', 'glugs', 'glume', 'glums', 'gluon', 'glute', 'gluts', 'glyph', 'gnapi', 'gnarl', 'gnarr', 'gnars', 'gnash', 'gnats', 'gnawn', 'gnaws', 'gnome', 'gnows', 'goads', 'goafs', 'goaft', 'goals', 'goary', 'goats', 'goaty', 'goave', 'goban', 'gobar', 'gobbe', 'gobbi', 'gobbo', 'gobby', 'gobis', 'gobos', 'godet', 'godly', 'godso', 'goels', 'goers', 'goest', 'goeth', 'goety', 'gofer', 'goffs', 'gogga', 'gogos', 'goier', 'going', 'gojis', 'gokes', 'golds', 'goldy', 'golem', 'goles', 'golfs', 'golly', 'golpe', 'golps', 'gombo', 'gomer', 'gompa', 'gonad', 'gonch', 'gonef', 'goner', 'gongs', 'gonia', 'gonif', 'gonks', 'gonna', 'gonof', 'gonys', 'gonzo', 'gooby', 'goodo', 'goods', 'goody', 'gooey', 'goofs', 'goofy', 'googs', 'gooks', 'gooky', 'goold', 'gools', 'gooly', 'goomy', 'goons', 'goony', 'goops', 'goopy', 'goors', 'goory', 'goose', 'goosy', 'gopak', 'gopik', 'goral', 'goras', 'goray', 'gorbs', 'gordo', 'gored', 'gores', 'gorge', 'goris', 'gorms', 'gormy', 'gorps', 'gorse', 'gorsy', 'gosht', 'gosse', 'gotch', 'goths', 'gothy', 'gotta', 'gouch', 'gouge', 'gouks', 'goura', 'gourd', 'gouts', 'gouty', 'goved', 'goves', 'gowan', 'gowds', 'gowfs', 'gowks', 'gowls', 'gowns', 'goxes', 'goyim', 'goyle', 'graal', 'grabs', 'grace', 'grade', 'grads', 'graff', 'graft', 'grail', 'grain', 'graip', 'grama', 'grame', 'gramp', 'grams', 'grana', 'grand', 'grano', 'grans', 'grant', 'grape', 'graph', 'grapy', 'grasp', 'grass', 'grata', 'grate', 'grats', 'grave', 'gravs', 'gravy', 'grays', 'graze', 'great', 'grebe', 'grebo', 'grece', 'greed', 'greek', 'green', 'grees', 'greet', 'grege', 'grego', 'grein', 'grens', 'greps', 'grese', 'greve', 'grews', 'greys', 'grice', 'gride', 'grids', 'grief', 'griff', 'grift', 'grigs', 'grike', 'grill', 'grime', 'grimy', 'grind', 'grins', 'griot', 'gripe', 'grips', 'gript', 'gripy', 'grise', 'grist', 'grisy', 'grith', 'grits', 'grize', 'groan', 'groat', 'grody', 'grogs', 'groin', 'groks', 'groma', 'groms', 'grone', 'groof', 'groom', 'grope', 'gross', 'grosz', 'grots', 'grouf', 'group', 'grout', 'grove', 'grovy', 'growl', 'grown', 'grows', 'grrls', 'grrrl', 'grubs', 'grued', 'gruel', 'grues', 'grufe', 'gruff', 'grume', 'grump', 'grund', 'grunt', 'gryce', 'gryde', 'gryke', 'grype', 'grypt', 'guaco', 'guana', 'guano', 'guans', 'guard', 'guars', 'guava', 'gubba', 'gucks', 'gucky', 'gudes', 'guess', 'guest', 'guffs', 'gugas', 'guggl', 'guide', 'guido', 'guids', 'guild', 'guile', 'guilt', 'guimp', 'guiro', 'guise', 'gulab', 'gulag', 'gular', 'gulas', 'gulch', 'gules', 'gulet', 'gulfs', 'gulfy', 'gulls', 'gully', 'gulph', 'gulps', 'gulpy', 'gumbo', 'gumma', 'gummi', 'gummy', 'gumps', 'gunas', 'gundi', 'gundy', 'gunge', 'gungy', 'gunks', 'gunky', 'gunny', 'guppy', 'guqin', 'gurdy', 'gurge', 'gurks', 'gurls', 'gurly', 'gurns', 'gurry', 'gursh', 'gurus', 'gushy', 'gusla', 'gusle', 'gusli', 'gussy', 'gusto', 'gusts', 'gusty', 'gutsy', 'gutta', 'gutty', 'guyed', 'guyle', 'guyot', 'guyse', 'gwine', 'gyals', 'gyans', 'gybed', 'gybes', 'gyeld', 'gymps', 'gynae', 'gynie', 'gynny', 'gynos', 'gyoza', 'gypes', 'gypos', 'gyppo', 'gyppy', 'gypsy', 'gyral', 'gyred', 'gyres', 'gyron', 'gyros', 'gyrus', 'gytes', 'gyved', 'gyver', 'gyves', 'haafs', 'haars', 'haats', 'habit', 'hable', 'habus', 'hacek', 'hacks', 'hacky', 'hadal', 'haded', 'hades', 'hadji', 'hadst', 'haems', 'haere', 'haets', 'haffs', 'hafiz', 'hafta', 'hafts', 'haggs', 'haham', 'hahas', 'haick', 'haika', 'haiks', 'haiku', 'hails', 'haily', 'hains', 'haint', 'hairs', 'hairy', 'haith', 'hajes', 'hajis', 'hajji', 'hakam', 'hakas', 'hakea', 'hakes', 'hakim', 'hakus', 'halal', 'haldi', 'haled', 'haler', 'hales', 'halfa', 'halfs', 'halid', 'hallo', 'halls', 'halma', 'halms', 'halon', 'halos', 'halse', 'halsh', 'halts', 'halva', 'halve', 'halwa', 'hamal', 'hamba', 'hamed', 'hamel', 'hames', 'hammy', 'hamza', 'hanap', 'hance', 'hanch', 'handi', 'hands', 'handy', 'hangi', 'hangs', 'hanks', 'hanky', 'hansa', 'hanse', 'hants', 'haole', 'haoma', 'hapas', 'hapax', 'haply', 'happi', 'happy', 'hapus', 'haram', 'hards', 'hardy', 'hared', 'harem', 'hares', 'harim', 'harks', 'harls', 'harms', 'harns', 'haros', 'harps', 'harpy', 'harry', 'harsh', 'harts', 'hashy', 'hasks', 'hasps', 'hasta', 'haste', 'hasty', 'hatch', 'hated', 'hater', 'hates', 'hatha', 'hathi', 'hatty', 'hauds', 'haufs', 'haugh', 'haugo', 'hauld', 'haulm', 'hauls', 'hault', 'hauns', 'haunt', 'hause', 'haute', 'havan', 'havel', 'haven', 'haver', 'haves', 'havoc', 'hawed', 'hawks', 'hawms', 'hawse', 'hayed', 'hayer', 'hayey', 'hayle', 'hazan', 'hazed', 'hazel', 'hazer', 'hazes', 'hazle', 'heads', 'heady', 'heald', 'heals', 'heame', 'heaps', 'heapy', 'heard', 'heare', 'hears', 'heart', 'heast', 'heath', 'heats', 'heaty', 'heave', 'heavy', 'heben', 'hebes', 'hecht', 'hecks', 'heder', 'hedge', 'hedgy', 'heeds', 'heedy', 'heels', 'heeze', 'hefte', 'hefts', 'hefty', 'heiau', 'heids', 'heigh', 'heils', 'heirs', 'heist', 'hejab', 'hejra', 'heled', 'heles', 'helio', 'helix', 'hella', 'hello', 'hells', 'helly', 'helms', 'helos', 'helot', 'helps', 'helve', 'hemal', 'hemes', 'hemic', 'hemin', 'hemps', 'hempy', 'hence', 'hench', 'hends', 'henge', 'henna', 'henny', 'henry', 'hents', 'hepar', 'herbs', 'herby', 'herds', 'heres', 'herls', 'herma', 'herms', 'herns', 'heron', 'heros', 'herps', 'herry', 'herse', 'hertz', 'herye', 'hesps', 'hests', 'hetes', 'heths', 'heuch', 'heugh', 'hevea', 'hevel', 'hewed', 'hewer', 'hewgh', 'hexad', 'hexed', 'hexer', 'hexes', 'hexyl', 'heyed', 'hiant', 'hibas', 'hicks', 'hided', 'hider', 'hides', 'hiems', 'hifis', 'highs', 'hight', 'hijab', 'hijra', 'hiked', 'hiker', 'hikes', 'hikoi', 'hilar', 'hilch', 'hillo', 'hills', 'hilly', 'hilsa', 'hilts', 'hilum', 'hilus', 'himbo', 'hinau', 'hinds', 'hinge', 'hings', 'hinky', 'hinny', 'hints', 'hiois', 'hiped', 'hiper', 'hipes', 'hiply', 'hippo', 'hippy', 'hired', 'hiree', 'hirer', 'hires', 'hissy', 'hists', 'hitch', 'hithe', 'hived', 'hiver', 'hives', 'hizen', 'hoach', 'hoaed', 'hoagy', 'hoard', 'hoars', 'hoary', 'hoast', 'hobby', 'hobos', 'hocks', 'hocus', 'hodad', 'hodja', 'hoers', 'hogan', 'hogen', 'hoggs', 'hoghs', 'hogoh', 'hogos', 'hohed', 'hoick', 'hoied', 'hoiks', 'hoing', 'hoise', 'hoist', 'hokas', 'hoked', 'hokes', 'hokey', 'hokis', 'hokku', 'hokum', 'holds', 'holed', 'holes', 'holey', 'holks', 'holla', 'hollo', 'holly', 'holme', 'holms', 'holon', 'holos', 'holts', 'homas', 'homed', 'homer', 'homes', 'homey', 'homie', 'homme', 'homos', 'honan', 'honda', 'honds', 'honed', 'honer', 'hones', 'honey', 'hongi', 'hongs', 'honks', 'honky', 'honor', 'hooch', 'hoods', 'hoody', 'hooey', 'hoofs', 'hoogo', 'hooha', 'hooka', 'hooks', 'hooky', 'hooly', 'hoons', 'hoops', 'hoord', 'hoors', 'hoosh', 'hoots', 'hooty', 'hoove', 'hopak', 'hoped', 'hoper', 'hopes', 'hoppy', 'horah', 'horal', 'horas', 'horde', 'horis', 'horks', 'horme', 'horns', 'horny', 'horse', 'horst', 'horsy', 'hosed', 'hosel', 'hosen', 'hoser', 'hoses', 'hosey', 'hosta', 'hosts', 'hotch', 'hotel', 'hoten', 'hotis', 'hotly', 'hotte', 'hotty', 'houff', 'houfs', 'hough', 'hound', 'houri', 'hours', 'house', 'houts', 'hovea', 'hoved', 'hovel', 'hoven', 'hover', 'hoves', 'howay', 'howbe', 'howdy', 'howes', 'howff', 'howfs', 'howks', 'howls', 'howre', 'howso', 'howto', 'hoxed', 'hoxes', 'hoyas', 'hoyed', 'hoyle', 'hubba', 'hubby', 'hucks', 'hudna', 'hudud', 'huers', 'huffs', 'huffy', 'huger', 'huggy', 'huhus', 'huias', 'huies', 'hukou', 'hulas', 'hules', 'hulks', 'hulky', 'hullo', 'hulls', 'hully', 'human', 'humas', 'humfs', 'humic', 'humid', 'humor', 'humph', 'humps', 'humpy', 'humus', 'hunch', 'hundo', 'hunks', 'hunky', 'hunts', 'hurds', 'hurls', 'hurly', 'hurra', 'hurry', 'hurst', 'hurts', 'hurty', 'hushy', 'husks', 'husky', 'husos', 'hussy', 'hutch', 'hutia', 'huzza', 'huzzy', 'hwyls', 'hydel', 'hydra', 'hydro', 'hyena', 'hyens', 'hygge', 'hying', 'hykes', 'hylas', 'hyleg', 'hyles', 'hylic', 'hymen', 'hymns', 'hynde', 'hyoid', 'hyped', 'hyper', 'hypes', 'hypha', 'hyphy', 'hypos', 'hyrax', 'hyson', 'hythe', 'iambi', 'iambs', 'ibrik', 'icers', 'iched', 'iches', 'ichor', 'icier', 'icily', 'icing', 'icker', 'ickle', 'icons', 'ictal', 'ictic', 'ictus', 'idant', 'iddah', 'iddat', 'iddut', 'ideal', 'ideas', 'idees', 'ident', 'idiom', 'idiot', 'idled', 'idler', 'idles', 'idlis', 'idola', 'idols', 'idyll', 'idyls', 'iftar', 'igapo', 'igged', 'igloo', 'iglus', 'ignis', 'ihram', 'iiwis', 'ikans', 'ikats', 'ikons', 'ileac', 'ileal', 'ileum', 'ileus', 'iliac', 'iliad', 'ilial', 'ilium', 'iller', 'illth', 'image', 'imago', 'imagy', 'imams', 'imari', 'imaum', 'imbar', 'imbed', 'imbos', 'imbue', 'imide', 'imido', 'imids', 'imine', 'imino', 'imlis', 'immew', 'immit', 'immix', 'imped', 'impel', 'impis', 'imply', 'impot', 'impro', 'imshi', 'imshy', 'inane', 'inapt', 'inarm', 'inbox', 'inbye', 'incas', 'incel', 'incle', 'incog', 'incur', 'incus', 'incut', 'indew', 'index', 'india', 'indie', 'indol', 'indow', 'indri', 'indue', 'inept', 'inerm', 'inert', 'infer', 'infix', 'infos', 'infra', 'ingan', 'ingle', 'ingot', 'inion', 'inked', 'inker', 'inkle', 'inlay', 'inlet', 'inned', 'inner', 'innie', 'innit', 'inorb', 'input', 'inros', 'inrun', 'insee', 'inset', 'inspo', 'intel', 'inter', 'intil', 'intis', 'intra', 'intro', 'inula', 'inure', 'inurn', 'inust', 'invar', 'inver', 'inwit', 'iodic', 'iodid', 'iodin', 'ionic', 'ioras', 'iotas', 'ippon', 'irade', 'irate', 'irids', 'iring', 'irked', 'iroko', 'irone', 'irons', 'irony', 'isbas', 'ishes', 'isled', 'isles', 'islet', 'isnae', 'issei', 'issue', 'istle', 'itchy', 'items', 'ither', 'ivied', 'ivies', 'ivory', 'ixias', 'ixnay', 'ixora', 'ixtle', 'izard', 'izars', 'izzat', 'jaaps', 'jabot', 'jacal', 'jacet', 'jacks', 'jacky', 'jaded', 'jades', 'jafas', 'jaffa', 'jagas', 'jager', 'jaggs', 'jaggy', 'jagir', 'jagra', 'jails', 'jaker', 'jakes', 'jakey', 'jakie', 'jalap', 'jaleo', 'jalop', 'jambe', 'jambo', 'jambs', 'jambu', 'james', 'jammy', 'jamon', 'jamun', 'janes', 'janky', 'janns', 'janny', 'janty', 'japan', 'japed', 'japer', 'japes', 'jarks', 'jarls', 'jarps', 'jarta', 'jarul', 'jasey', 'jaspe', 'jasps', 'jatha', 'jatis', 'jatos', 'jauks', 'jaune', 'jaunt', 'jaups', 'javas', 'javel', 'jawan', 'jawed', 'jawns', 'jaxie', 'jazzy', 'jeans', 'jeats', 'jebel', 'jedis', 'jeels', 'jeely', 'jeeps', 'jeera', 'jeers', 'jeeze', 'jefes', 'jeffs', 'jehad', 'jehus', 'jelab', 'jello', 'jells', 'jelly', 'jembe', 'jemmy', 'jenny', 'jeons', 'jerid', 'jerks', 'jerky', 'jerry', 'jesse', 'jessy', 'jests', 'jesus', 'jetee', 'jetes', 'jeton', 'jetty', 'jeune', 'jewed', 'jewel', 'jewie', 'jhala', 'jheel', 'jhils', 'jiaos', 'jibba', 'jibbs', 'jibed', 'jiber', 'jibes', 'jiffs', 'jiffy', 'jiggy', 'jigot', 'jihad', 'jills', 'jilts', 'jimmy', 'jimpy', 'jingo', 'jings', 'jinks', 'jinne', 'jinni', 'jinns', 'jirds', 'jirga', 'jirre', 'jisms', 'jitis', 'jitty', 'jived', 'jiver', 'jives', 'jivey', 'jnana', 'jobed', 'jobes', 'jocko', 'jocks', 'jocky', 'jocos', 'jodel', 'joeys', 'johns', 'joins', 'joint', 'joist', 'joked', 'joker', 'jokes', 'jokey', 'jokol', 'joled', 'joles', 'jolie', 'jollo', 'jolls', 'jolly', 'jolts', 'jolty', 'jomon', 'jomos', 'jones', 'jongs', 'jonty', 'jooks', 'joram', 'jorts', 'jorum', 'jotas', 'jotty', 'jotun', 'joual', 'jougs', 'jouks', 'joule', 'jours', 'joust', 'jowar', 'jowed', 'jowls', 'jowly', 'joyed', 'jubas', 'jubes', 'jucos', 'judas', 'judge', 'judgy', 'judos', 'jugal', 'jugum', 'juice', 'juicy', 'jujus', 'juked', 'jukes', 'jukus', 'julep', 'julia', 'jumar', 'jumbo', 'jumby', 'jumps', 'jumpy', 'junco', 'junks', 'junky', 'junta', 'junto', 'jupes', 'jupon', 'jural', 'jurat', 'jurel', 'jures', 'juris', 'juror', 'juste', 'justs', 'jutes', 'jutty', 'juves', 'juvie', 'kaama', 'kabab', 'kabar', 'kabob', 'kacha', 'kacks', 'kadai', 'kades', 'kadis', 'kafir', 'kagos', 'kagus', 'kahal', 'kaiak', 'kaids', 'kaies', 'kaifs', 'kaika', 'kaiks', 'kails', 'kaims', 'kaing', 'kains', 'kajal', 'kakas', 'kakis', 'kalam', 'kalas', 'kales', 'kalif', 'kalis', 'kalpa', 'kalua', 'kamas', 'kames', 'kamik', 'kamis', 'kamme', 'kanae', 'kanal', 'kanas', 'kanat', 'kandy', 'kaneh', 'kanes', 'kanga', 'kangs', 'kanji', 'kants', 'kanzu', 'kaons', 'kapai', 'kapas', 'kapha', 'kaphs', 'kapok', 'kapow', 'kappa', 'kapur', 'kapus', 'kaput', 'karai', 'karas', 'karat', 'karee', 'karez', 'karks', 'karma', 'karns', 'karoo', 'karos', 'karri', 'karst', 'karsy', 'karts', 'karzy', 'kasha', 'kasme', 'katal', 'katas', 'katis', 'katti', 'kaugh', 'kauri', 'kauru', 'kaury', 'kaval', 'kavas', 'kawas', 'kawau', 'kawed', 'kayak', 'kayle', 'kayos', 'kazis', 'kazoo', 'kbars', 'kcals', 'keaki', 'kebab', 'kebar', 'kebob', 'kecks', 'kedge', 'kedgy', 'keech', 'keefs', 'keeks', 'keels', 'keema', 'keeno', 'keens', 'keeps', 'keets', 'keeve', 'kefir', 'kehua', 'keirs', 'kelep', 'kelim', 'kells', 'kelly', 'kelps', 'kelpy', 'kelts', 'kelty', 'kembo', 'kembs', 'kemps', 'kempt', 'kempy', 'kenaf', 'kench', 'kendo', 'kenos', 'kente', 'kents', 'kepis', 'kerbs', 'kerel', 'kerfs', 'kerky', 'kerma', 'kerne', 'kerns', 'keros', 'kerry', 'kerve', 'kesar', 'kests', 'ketas', 'ketch', 'ketes', 'ketol', 'kevel', 'kevil', 'kexes', 'keyed', 'keyer', 'khadi', 'khads', 'khafs', 'khaki', 'khana', 'khans', 'khaph', 'khats', 'khaya', 'khazi', 'kheda', 'kheer', 'kheth', 'khets', 'khirs', 'khoja', 'khors', 'khoum', 'khuds', 'khula', 'khyal', 'kiaat', 'kiack', 'kiaki', 'kiang', 'kiasu', 'kibbe', 'kibbi', 'kibei', 'kibes', 'kibla', 'kicks', 'kicky', 'kiddo', 'kiddy', 'kidel', 'kideo', 'kidge', 'kiefs', 'kiers', 'kieve', 'kievs', 'kight', 'kikay', 'kikes', 'kikoi', 'kiley', 'kilig', 'kilim', 'kills', 'kilns', 'kilos', 'kilps', 'kilts', 'kilty', 'kimbo', 'kimet', 'kinas', 'kinda', 'kinds', 'kindy', 'kines', 'kings', 'kingy', 'kinin', 'kinks', 'kinky', 'kinos', 'kiore', 'kiosk', 'kipah', 'kipas', 'kipes', 'kippa', 'kipps', 'kipsy', 'kirby', 'kirks', 'kirns', 'kirri', 'kisan', 'kissy', 'kists', 'kitab', 'kited', 'kiter', 'kites', 'kithe', 'kiths', 'kitke', 'kitty', 'kitul', 'kivas', 'kiwis', 'klang', 'klaps', 'klett', 'klick', 'klieg', 'kliks', 'klong', 'kloof', 'kluge', 'klutz', 'knack', 'knags', 'knaps', 'knarl', 'knars', 'knaur', 'knave', 'knawe', 'knead', 'kneed', 'kneel', 'knees', 'knell', 'knelt', 'knick', 'knife', 'knish', 'knits', 'knive', 'knobs', 'knock', 'knoll', 'knoop', 'knops', 'knosp', 'knots', 'knoud', 'knout', 'knowd', 'knowe', 'known', 'knows', 'knubs', 'knule', 'knurl', 'knurr', 'knurs', 'knuts', 'koala', 'koans', 'koaps', 'koban', 'kobos', 'koels', 'koffs', 'kofta', 'kogal', 'kohas', 'kohen', 'kohls', 'koine', 'koiwi', 'kojis', 'kokam', 'kokas', 'koker', 'kokra', 'kokum', 'kolas', 'kolos', 'kombi', 'kombu', 'konbu', 'kondo', 'konks', 'kooks', 'kooky', 'koori', 'kopek', 'kophs', 'kopje', 'koppa', 'korai', 'koran', 'koras', 'korat', 'kores', 'koris', 'korma', 'koros', 'korun', 'korus', 'koses', 'kotch', 'kotos', 'kotow', 'koura', 'kraal', 'krabs', 'kraft', 'krais', 'krait', 'krang', 'krans', 'kranz', 'kraut', 'krays', 'kreef', 'kreen', 'kreep', 'kreng', 'krewe', 'krill', 'kriol', 'krona', 'krone', 'kroon', 'krubi', 'krump', 'krunk', 'ksars', 'kubie', 'kudos', 'kudus', 'kudzu', 'kufis', 'kugel', 'kuias', 'kukri', 'kukus', 'kulak', 'kulan', 'kulas', 'kulfi', 'kumis', 'kumys', 'kunas', 'kunds', 'kuris', 'kurre', 'kurta', 'kurus', 'kusso', 'kusti', 'kutai', 'kutas', 'kutch', 'kutis', 'kutus', 'kuyas', 'kuzus', 'kvass', 'kvell', 'kwaai', 'kwela', 'kwink', 'kwirl', 'kyack', 'kyaks', 'kyang', 'kyars', 'kyats', 'kybos', 'kydst', 'kyles', 'kylie', 'kylin', 'kylix', 'kyloe', 'kynde', 'kynds', 'kypes', 'kyrie', 'kytes', 'kythe', 'kyudo', 'laarf', 'laari', 'labda', 'label', 'labia', 'labis', 'labne', 'labor', 'labra', 'laccy', 'laced', 'lacer', 'laces', 'lacet', 'lacey', 'lacis', 'lacka', 'lacks', 'lacky', 'laddu', 'laddy', 'laded', 'ladee', 'laden', 'lader', 'lades', 'ladle', 'ladoo', 'laers', 'laevo', 'lagan', 'lagar', 'lager', 'laggy', 'lahal', 'lahar', 'laich', 'laics', 'laide', 'laids', 'laigh', 'laika', 'laiks', 'laird', 'lairs', 'lairy', 'laith', 'laity', 'laked', 'laker', 'lakes', 'lakhs', 'lakin', 'laksa', 'laldy', 'lalls', 'lamas', 'lambs', 'lamby', 'lamed', 'lamer', 'lames', 'lamia', 'lammy', 'lamps', 'lanai', 'lanas', 'lance', 'lanch', 'lande', 'lands', 'laned', 'lanes', 'lanks', 'lanky', 'lants', 'lapas', 'lapel', 'lapin', 'lapis', 'lapje', 'lappa', 'lappy', 'lapse', 'larch', 'lards', 'lardy', 'laree', 'lares', 'larfs', 'larga', 'large', 'largo', 'laris', 'larks', 'larky', 'larns', 'larnt', 'larum', 'larva', 'lased', 'laser', 'lases', 'lassi', 'lasso', 'lassu', 'lassy', 'lasts', 'latah', 'latch', 'lated', 'laten', 'later', 'latex', 'lathe', 'lathi', 'laths', 'lathy', 'latke', 'latte', 'latus', 'lauan', 'lauch', 'laude', 'lauds', 'laufs', 'laugh', 'laund', 'laura', 'laval', 'lavas', 'laved', 'laver', 'laves', 'lavra', 'lavvy', 'lawed', 'lawer', 'lawin', 'lawks', 'lawns', 'lawny', 'lawsy', 'laxed', 'laxer', 'laxes', 'laxly', 'layby', 'layed', 'layer', 'layin', 'layup', 'lazar', 'lazed', 'lazes', 'lazos', 'lazzi', 'lazzo', 'leach', 'leads', 'leady', 'leafs', 'leafy', 'leaks', 'leaky', 'leams', 'leans', 'leant', 'leany', 'leaps', 'leapt', 'leare', 'learn', 'lears', 'leary', 'lease', 'leash', 'least', 'leats', 'leave', 'leavy', 'leaze', 'leben', 'leccy', 'leche', 'ledes', 'ledge', 'ledgy', 'ledum', 'leear', 'leech', 'leeks', 'leeps', 'leers', 'leery', 'leese', 'leets', 'leeze', 'lefte', 'lefts', 'lefty', 'legal', 'leger', 'leges', 'legge', 'leggo', 'leggy', 'legit', 'legno', 'lehrs', 'lehua', 'leirs', 'leish', 'leman', 'lemed', 'lemel', 'lemes', 'lemma', 'lemme', 'lemon', 'lemur', 'lends', 'lenes', 'lengs', 'lenis', 'lenos', 'lense', 'lenti', 'lento', 'leone', 'lepak', 'leper', 'lepid', 'lepra', 'lepta', 'lered', 'leres', 'lerps', 'lesbo', 'leses', 'lesos', 'lests', 'letch', 'lethe', 'letty', 'letup', 'leuch', 'leuco', 'leuds', 'leugh', 'levas', 'levee', 'level', 'lever', 'leves', 'levin', 'levis', 'lewis', 'lexes', 'lexis', 'lezes', 'lezza', 'lezzo', 'lezzy', 'liana', 'liane', 'liang', 'liard', 'liars', 'liart', 'libel', 'liber', 'libor', 'libra', 'libre', 'libri', 'licet', 'lichi', 'licht', 'licit', 'licks', 'lidar', 'lidos', 'liefs', 'liege', 'liens', 'liers', 'lieus', 'lieve', 'lifer', 'lifes', 'lifey', 'lifts', 'ligan', 'liger', 'ligge', 'light', 'ligne', 'liked', 'liken', 'liker', 'likes', 'likin', 'lilac', 'lills', 'lilos', 'lilts', 'lilty', 'liman', 'limas', 'limax', 'limba', 'limbi', 'limbo', 'limbs', 'limby', 'limed', 'limen', 'limes', 'limey', 'limit', 'limma', 'limns', 'limos', 'limpa', 'limps', 'linac', 'linch', 'linds', 'lindy', 'lined', 'linen', 'liner', 'lines', 'liney', 'linga', 'lingo', 'lings', 'lingy', 'linin', 'links', 'linky', 'linns', 'linny', 'linos', 'lints', 'linty', 'linum', 'linux', 'lions', 'lipas', 'lipes', 'lipid', 'lipin', 'lipos', 'lippy', 'liras', 'lirks', 'lirot', 'lises', 'lisks', 'lisle', 'lisps', 'lists', 'litai', 'litas', 'lited', 'litem', 'liter', 'lites', 'lithe', 'litho', 'liths', 'litie', 'litre', 'lived', 'liven', 'liver', 'lives', 'livid', 'livor', 'livre', 'liwaa', 'liwas', 'llama', 'llano', 'loach', 'loads', 'loafs', 'loams', 'loamy', 'loans', 'loast', 'loath', 'loave', 'lobar', 'lobby', 'lobed', 'lobes', 'lobos', 'lobus', 'local', 'loche', 'lochs', 'lochy', 'locie', 'locis', 'locks', 'locky', 'locos', 'locum', 'locus', 'loden', 'lodes', 'lodge', 'loess', 'lofts', 'lofty', 'logan', 'loges', 'loggy', 'logia', 'logic', 'logie', 'login', 'logoi', 'logon', 'logos', 'lohan', 'loids', 'loins', 'loipe', 'loirs', 'lokes', 'lokey', 'lokum', 'lolas', 'loled', 'lollo', 'lolls', 'lolly', 'lolog', 'lolos', 'lomas', 'lomed', 'lomes', 'loner', 'longa', 'longe', 'longs', 'looby', 'looed', 'looey', 'loofa', 'loofs', 'looie', 'looks', 'looky', 'looms', 'loons', 'loony', 'loops', 'loopy', 'loord', 'loose', 'loots', 'loped', 'loper', 'lopes', 'loppy', 'loral', 'loran', 'lords', 'lordy', 'lorel', 'lores', 'loric', 'loris', 'lorry', 'losed', 'losel', 'losen', 'loser', 'loses', 'lossy', 'lotah', 'lotas', 'lotes', 'lotic', 'lotos', 'lotsa', 'lotta', 'lotte', 'lotto', 'lotus', 'loued', 'lough', 'louie', 'louis', 'louma', 'lound', 'louns', 'loupe', 'loups', 'loure', 'lours', 'loury', 'louse', 'lousy', 'louts', 'lovat', 'loved', 'lovee', 'lover', 'loves', 'lovey', 'lovie', 'lowan', 'lowed', 'lowen', 'lower', 'lowes', 'lowly', 'lownd', 'lowne', 'lowns', 'lowps', 'lowry', 'lowse', 'lowth', 'lowts', 'loxed', 'loxes', 'loyal', 'lozen', 'luach', 'luaus', 'lubed', 'lubes', 'lubra', 'luces', 'lucid', 'lucks', 'lucky', 'lucre', 'ludes', 'ludic', 'ludos', 'luffa', 'luffs', 'luged', 'luger', 'luges', 'lulls', 'lulus', 'lumas', 'lumbi', 'lumen', 'lumme', 'lummy', 'lumps', 'lumpy', 'lunar', 'lunas', 'lunch', 'lunes', 'lunet', 'lunge', 'lungi', 'lungs', 'lunks', 'lunts', 'lupin', 'lupus', 'lurch', 'lured', 'lurer', 'lures', 'lurex', 'lurgi', 'lurgy', 'lurid', 'lurks', 'lurry', 'lurve', 'luser', 'lushy', 'lusks', 'lusts', 'lusty', 'lusus', 'lutea', 'luted', 'luter', 'lutes', 'luvvy', 'luxed', 'luxer', 'luxes', 'lweis', 'lyams', 'lyard', 'lyart', 'lyase', 'lycea', 'lycee', 'lycra', 'lying', 'lymes', 'lymph', 'lynch', 'lynes', 'lyres', 'lyric', 'lysed', 'lyses', 'lysin', 'lysis', 'lysol', 'lyssa', 'lyted', 'lytes', 'lythe', 'lytic', 'lytta', 'maaed', 'maare', 'maars', 'maban', 'mabes', 'macas', 'macaw', 'macca', 'maced', 'macer', 'maces', 'mache', 'machi', 'macho', 'machs', 'macka', 'macks', 'macle', 'macon', 'macro', 'macte', 'madal', 'madam', 'madar', 'maddy', 'madge', 'madid', 'madly', 'mados', 'madre', 'maedi', 'maerl', 'mafia', 'mafic', 'mafts', 'magas', 'mages', 'maggs', 'magic', 'magma', 'magna', 'magot', 'magus', 'mahal', 'mahem', 'mahis', 'mahoe', 'mahrs', 'mahua', 'mahwa', 'maids', 'maiko', 'maiks', 'maile', 'maill', 'mailo', 'mails', 'maims', 'mains', 'maire', 'mairs', 'maise', 'maist', 'maize', 'majas', 'majat', 'majoe', 'major', 'majos', 'makaf', 'makai', 'makan', 'makar', 'makee', 'maker', 'makes', 'makie', 'makis', 'makos', 'malae', 'malai', 'malam', 'malar', 'malas', 'malax', 'maleo', 'males', 'malic', 'malik', 'malis', 'malky', 'malls', 'malms', 'malmy', 'malts', 'malty', 'malus', 'malva', 'malwa', 'mamak', 'mamas', 'mamba', 'mambo', 'mambu', 'mamee', 'mamey', 'mamie', 'mamil', 'mamma', 'mammy', 'manas', 'manat', 'mandi', 'mands', 'mandy', 'maneb', 'maned', 'maneh', 'manes', 'manet', 'manga', 'mange', 'mangi', 'mango', 'mangs', 'mangy', 'mania', 'manic', 'manie', 'manis', 'manks', 'manky', 'manly', 'manna', 'manny', 'manoa', 'manor', 'manos', 'manse', 'manso', 'manta', 'mante', 'manto', 'mants', 'manty', 'manul', 'manus', 'manzo', 'mapau', 'mapes', 'maple', 'mapou', 'mappy', 'maqam', 'maqui', 'marae', 'marah', 'maral', 'maran', 'maras', 'maray', 'march', 'marcs', 'mards', 'mardy', 'mares', 'marga', 'marge', 'margo', 'margs', 'maria', 'marid', 'maril', 'marka', 'marks', 'marle', 'marls', 'marly', 'marma', 'marms', 'maron', 'maror', 'marra', 'marri', 'marry', 'marse', 'marsh', 'marts', 'marua', 'marvy', 'masas', 'mased', 'maser', 'mases', 'masha', 'mashy', 'masks', 'mason', 'massa', 'masse', 'massy', 'masts', 'masty', 'masur', 'masus', 'masut', 'matai', 'match', 'mated', 'mater', 'mates', 'matey', 'mathe', 'maths', 'matin', 'matlo', 'matra', 'matsu', 'matte', 'matts', 'matty', 'matza', 'matzo', 'mauby', 'mauds', 'mauka', 'maula', 'mauls', 'maums', 'maumy', 'maund', 'maunt', 'mauri', 'mausy', 'mauts', 'mauve', 'mauvy', 'mauzy', 'maven', 'mavie', 'mavin', 'mavis', 'mawed', 'mawks', 'mawky', 'mawla', 'mawns', 'mawps', 'mawrs', 'maxed', 'maxes', 'maxim', 'maxis', 'mayan', 'mayas', 'maybe', 'mayed', 'mayor', 'mayos', 'mayst', 'mazac', 'mazak', 'mazar', 'mazas', 'mazed', 'mazel', 'mazer', 'mazes', 'mazet', 'mazey', 'mazut', 'mbari', 'mbars', 'mbila', 'mbira', 'mbret', 'mbube', 'mbuga', 'meads', 'meake', 'meaks', 'meals', 'mealy', 'meane', 'means', 'meant', 'meany', 'meare', 'mease', 'meath', 'meats', 'meaty', 'mebbe', 'mebos', 'mecca', 'mecha', 'mechs', 'mecks', 'mecum', 'medal', 'media', 'medic', 'medii', 'medin', 'medle', 'meech', 'meeds', 'meeja', 'meeps', 'meers', 'meets', 'meffs', 'meids', 'meiko', 'meils', 'meins', 'meint', 'meiny', 'meism', 'meith', 'mekka', 'melam', 'melas', 'melba', 'melch', 'melds', 'melee', 'meles', 'melic', 'melik', 'mells', 'meloe', 'melon', 'melos', 'melts', 'melty', 'memes', 'memic', 'memos', 'menad', 'mence', 'mends', 'mened', 'menes', 'menge', 'mengs', 'menil', 'mensa', 'mense', 'mensh', 'menta', 'mento', 'ments', 'menus', 'meous', 'meows', 'merch', 'mercs', 'mercy', 'merde', 'merds', 'mered', 'merel', 'merer', 'meres', 'merge', 'meril', 'meris', 'merit', 'merks', 'merle', 'merls', 'merry', 'merse', 'mersk', 'mesad', 'mesal', 'mesas', 'mesca', 'mesel', 'mesem', 'meses', 'meshy', 'mesia', 'mesic', 'mesne', 'meson', 'messy', 'mesto', 'mesyl', 'metal', 'metas', 'meted', 'meteg', 'metel', 'meter', 'metes', 'methi', 'metho', 'meths', 'methy', 'metic', 'metif', 'metis', 'metol', 'metre', 'metro', 'metta', 'meums', 'meuse', 'meved', 'meves', 'mewed', 'mewls', 'meynt', 'mezes', 'mezza', 'mezze', 'mezzo', 'mgals', 'mhorr', 'miais', 'miaou', 'miaow', 'miasm', 'miaul', 'micas', 'miche', 'michi', 'micht', 'micks', 'micky', 'micos', 'micra', 'micro', 'middy', 'midge', 'midgy', 'midis', 'midst', 'miens', 'mieux', 'mieve', 'miffs', 'miffy', 'mifty', 'miggs', 'might', 'migma', 'migod', 'mihas', 'mihis', 'mikan', 'miked', 'mikes', 'mikos', 'mikra', 'mikva', 'milch', 'milds', 'miler', 'miles', 'milfs', 'milia', 'milko', 'milks', 'milky', 'mille', 'mills', 'milly', 'milor', 'milos', 'milpa', 'milts', 'milty', 'miltz', 'mimed', 'mimeo', 'mimer', 'mimes', 'mimic', 'mimis', 'mimsy', 'minae', 'minar', 'minas', 'mince', 'mincy', 'mindi', 'minds', 'mined', 'miner', 'mines', 'minge', 'mingi', 'mings', 'mingy', 'minim', 'minis', 'minke', 'minks', 'minny', 'minor', 'minos', 'minse', 'mints', 'minty', 'minus', 'minxy', 'miraa', 'mirah', 'mirch', 'mired', 'mires', 'mirex', 'mirid', 'mirin', 'mirkn', 'mirks', 'mirky', 'mirls', 'mirly', 'miros', 'mirrl', 'mirrs', 'mirth', 'mirvs', 'mirza', 'misal', 'misch', 'misdo', 'miser', 'mises', 'misgo', 'misky', 'misls', 'misos', 'missa', 'missy', 'misto', 'mists', 'misty', 'mitas', 'mitch', 'miter', 'mites', 'mitey', 'mitie', 'mitis', 'mitre', 'mitry', 'mitta', 'mitts', 'mivey', 'mivvy', 'mixed', 'mixen', 'mixer', 'mixes', 'mixie', 'mixis', 'mixte', 'mixup', 'miyas', 'mizen', 'mizes', 'mizzy', 'mmkay', 'mneme', 'moais', 'moaky', 'moals', 'moana', 'moans', 'moany', 'moars', 'moats', 'mobby', 'mobed', 'mobee', 'mobes', 'mobey', 'mobie', 'moble', 'mobos', 'mocap', 'mocha', 'mochi', 'mochs', 'mochy', 'mocks', 'mocky', 'mocos', 'mocus', 'modal', 'model', 'modem', 'moder', 'modes', 'modge', 'modii', 'modin', 'modoc', 'modom', 'modus', 'moeni', 'moers', 'mofos', 'mogar', 'mogas', 'moggy', 'mogos', 'mogra', 'mogue', 'mogul', 'mohar', 'mohel', 'mohos', 'mohrs', 'mohua', 'mohur', 'moile', 'moils', 'moira', 'moire', 'moist', 'moits', 'moity', 'mojos', 'moker', 'mokes', 'mokey', 'mokis', 'mokky', 'mokos', 'mokus', 'molal', 'molar', 'molas', 'molds', 'moldy', 'moled', 'moler', 'moles', 'moley', 'molie', 'molla', 'molle', 'mollo', 'molls', 'molly', 'moloi', 'molos', 'molto', 'molts', 'molue', 'molvi', 'molys', 'momes', 'momie', 'momma', 'momme', 'mommy', 'momos', 'mompe', 'momus', 'monad', 'monal', 'monas', 'monde', 'mondo', 'moner', 'money', 'mongo', 'mongs', 'monic', 'monie', 'monks', 'monos', 'monpe', 'monte', 'month', 'monty', 'moobs', 'mooch', 'moods', 'moody', 'mooed', 'mooey', 'mooks', 'moola', 'mooli', 'mools', 'mooly', 'moong', 'mooni', 'moons', 'moony', 'moops', 'moors', 'moory', 'moose', 'mooth', 'moots', 'moove', 'moped', 'moper', 'mopes', 'mopey', 'moppy', 'mopsy', 'mopus', 'morae', 'morah', 'moral', 'moran', 'moras', 'morat', 'moray', 'moree', 'morel', 'mores', 'morgy', 'moria', 'morin', 'mormo', 'morna', 'morne', 'morns', 'moron', 'moror', 'morph', 'morra', 'morro', 'morse', 'morts', 'moruk', 'mosed', 'moses', 'mosey', 'mosks', 'mosso', 'mossy', 'moste', 'mosto', 'mosts', 'moted', 'motel', 'moten', 'motes', 'motet', 'motey', 'moths', 'mothy', 'motif', 'motis', 'moton', 'motor', 'motte', 'motto', 'motts', 'motty', 'motus', 'motza', 'mouch', 'moues', 'moufs', 'mould', 'moule', 'mouls', 'moult', 'mouly', 'mound', 'mount', 'moups', 'mourn', 'mouse', 'moust', 'mousy', 'mouth', 'moved', 'mover', 'moves', 'movie', 'mowas', 'mowed', 'mower', 'mowie', 'mowra', 'moxas', 'moxie', 'moyas', 'moyle', 'moyls', 'mozed', 'mozes', 'mozos', 'mpret', 'mrads', 'msasa', 'mtepe', 'mucho', 'mucic', 'mucid', 'mucin', 'mucko', 'mucks', 'mucky', 'mucor', 'mucro', 'mucus', 'mudar', 'muddy', 'mudge', 'mudif', 'mudim', 'mudir', 'mudra', 'muffs', 'muffy', 'mufti', 'mugga', 'muggs', 'muggy', 'mugho', 'mugil', 'mugos', 'muhly', 'muids', 'muils', 'muirs', 'muiry', 'muist', 'mujik', 'mukim', 'mukti', 'mulai', 'mulch', 'mulct', 'muled', 'mules', 'muley', 'mulga', 'mulie', 'mulla', 'mulls', 'mulse', 'mulsh', 'mumbo', 'mumms', 'mummy', 'mumph', 'mumps', 'mumsy', 'mumus', 'munch', 'munds', 'mundu', 'munga', 'munge', 'mungi', 'mungo', 'mungs', 'mungy', 'munia', 'munis', 'munja', 'munjs', 'munts', 'muntu', 'muons', 'mural', 'muras', 'mured', 'mures', 'murex', 'murgh', 'murgi', 'murid', 'murks', 'murky', 'murls', 'murly', 'murra', 'murre', 'murri', 'murrs', 'murry', 'murth', 'murti', 'muruk', 'murva', 'musar', 'musca', 'mused', 'musee', 'muser', 'muses', 'muset', 'musha', 'mushy', 'music', 'musit', 'musks', 'musky', 'musos', 'musse', 'mussy', 'musta', 'musth', 'musts', 'musty', 'mutas', 'mutch', 'muted', 'muter', 'mutes', 'mutha', 'mutic', 'mutis', 'muton', 'mutti', 'mutts', 'mutum', 'muvva', 'muxed', 'muxes', 'muzak', 'muzzy', 'mvula', 'mvule', 'mvuli', 'myall', 'myals', 'mylar', 'mynah', 'mynas', 'myoid', 'myoma', 'myons', 'myope', 'myops', 'myopy', 'myrrh', 'mysid', 'mysie', 'mythi', 'myths', 'mythy', 'myxos', 'mzees', 'naams', 'naans', 'naats', 'nabam', 'nabby', 'nabes', 'nabis', 'nabks', 'nabla', 'nabob', 'nache', 'nacho', 'nacre', 'nadas', 'nadir', 'naeve', 'naevi', 'naffs', 'nagar', 'nagas', 'nages', 'naggy', 'nagor', 'nahal', 'naiad', 'naibs', 'naice', 'naids', 'naieo', 'naifs', 'naiks', 'nails', 'naily', 'nains', 'naios', 'naira', 'nairu', 'naive', 'najib', 'nakas', 'naked', 'naker', 'nakfa', 'nalas', 'naled', 'nalla', 'namad', 'namak', 'namaz', 'named', 'namer', 'names', 'namma', 'namus', 'nanas', 'nance', 'nancy', 'nandu', 'nanna', 'nanny', 'nanos', 'nante', 'nanti', 'nanto', 'nants', 'nanty', 'nanua', 'napas', 'naped', 'napes', 'napoh', 'napoo', 'nappa', 'nappe', 'nappy', 'naras', 'narco', 'narcs', 'nards', 'nares', 'naric', 'naris', 'narks', 'narky', 'narod', 'narra', 'narre', 'nasal', 'nashi', 'nasho', 'nasis', 'nason', 'nasty', 'nasus', 'natak', 'natal', 'natch', 'nates', 'natis', 'natto', 'natty', 'natya', 'nauch', 'naunt', 'naval', 'navar', 'naved', 'navel', 'naves', 'navew', 'navvy', 'nawab', 'nawal', 'nazar', 'nazes', 'nazir', 'nazis', 'nazzy', 'nduja', 'neafe', 'neals', 'neant', 'neaps', 'nears', 'neath', 'neato', 'neats', 'nebby', 'nebek', 'nebel', 'neche', 'necks', 'neddy', 'neebs', 'needs', 'needy', 'neefs', 'neeld', 'neele', 'neemb', 'neems', 'neeps', 'neese', 'neeze', 'nefie', 'negri', 'negro', 'negus', 'neifs', 'neigh', 'neist', 'neive', 'nelia', 'nelis', 'nelly', 'nemas', 'nemic', 'nemns', 'nempt', 'nenes', 'nenta', 'neons', 'neosa', 'neoza', 'neper', 'nepit', 'neral', 'neram', 'nerds', 'nerdy', 'nerfs', 'nerka', 'nerks', 'nerol', 'nerts', 'nertz', 'nerve', 'nervy', 'neski', 'nests', 'nesty', 'netas', 'netes', 'netop', 'netta', 'netts', 'netty', 'neuks', 'neume', 'neums', 'nevel', 'never', 'neves', 'nevis', 'nevus', 'nevvy', 'newbs', 'newed', 'newel', 'newer', 'newie', 'newly', 'newsy', 'newts', 'nexal', 'nexin', 'nexts', 'nexum', 'nexus', 'ngaio', 'ngaka', 'ngana', 'ngapi', 'ngati', 'ngege', 'ngoma', 'ngoni', 'ngram', 'ngwee', 'nibby', 'nicad', 'niced', 'nicer', 'nicey', 'niche', 'nicht', 'nicks', 'nicky', 'nicol', 'nidal', 'nided', 'nides', 'nidor', 'nidus', 'niece', 'niefs', 'niess', 'nieve', 'nifes', 'niffs', 'niffy', 'nifle', 'nifty', 'niger', 'nigga', 'nighs', 'night', 'nigre', 'nigua', 'nihil', 'nikab', 'nikah', 'nikau', 'nilas', 'nills', 'nimbi', 'nimbs', 'nimby', 'nimps', 'niner', 'nines', 'ninja', 'ninny', 'ninon', 'ninta', 'ninth', 'niopo', 'nioza', 'nipas', 'nipet', 'nippy', 'niqab', 'nirls', 'nirly', 'nisei', 'nisin', 'nisse', 'nisus', 'nital', 'niter', 'nites', 'nitid', 'niton', 'nitre', 'nitro', 'nitry', 'nitta', 'nitto', 'nitty', 'nival', 'nivas', 'nivel', 'nixed', 'nixer', 'nixes', 'nixie', 'nizam', 'njirl', 'nkosi', 'nmoli', 'nmols', 'noahs', 'nobby', 'noble', 'nobly', 'nocks', 'nodal', 'noddy', 'noded', 'nodes', 'nodum', 'nodus', 'noels', 'noema', 'noeme', 'nogal', 'noggs', 'noggy', 'nohow', 'noias', 'noils', 'noily', 'noint', 'noire', 'noirs', 'noise', 'noisy', 'nokes', 'noles', 'nolle', 'nolls', 'nolos', 'nomad', 'nomas', 'nomen', 'nomes', 'nomic', 'nomoi', 'nomos', 'nonan', 'nonas', 'nonce', 'noncy', 'nonda', 'nondo', 'nones', 'nonet', 'nongs', 'nonic', 'nonis', 'nonna', 'nonno', 'nonny', 'nonyl', 'noobs', 'noois', 'nooit', 'nooks', 'nooky', 'noone', 'noons', 'noops', 'noose', 'noove', 'nopal', 'noria', 'norie', 'noris', 'norks', 'norma', 'norms', 'north', 'nosed', 'noser', 'noses', 'nosey', 'noshi', 'nosir', 'notal', 'notam', 'notch', 'noted', 'noter', 'notes', 'notum', 'nougs', 'nouja', 'nould', 'noule', 'nouls', 'nouns', 'nouny', 'noups', 'noust', 'novae', 'novas', 'novel', 'novia', 'novio', 'novum', 'noway', 'nowds', 'nowed', 'nowls', 'nowts', 'nowty', 'noxal', 'noxas', 'noxes', 'noyau', 'noyed', 'noyes', 'nrtta', 'nrtya', 'nsima', 'nubby', 'nubia', 'nucha', 'nucin', 'nuddy', 'nuder', 'nudes', 'nudge', 'nudgy', 'nudie', 'nudzh', 'nuevo', 'nuffs', 'nugae', 'nujol', 'nuked', 'nukes', 'nulla', 'nullo', 'nulls', 'nully', 'numbs', 'numen', 'nummy', 'numps', 'nunks', 'nunky', 'nunny', 'nunus', 'nuque', 'nurds', 'nurdy', 'nurls', 'nurrs', 'nurse', 'nurts', 'nurtz', 'nused', 'nuses', 'nutso', 'nutsy', 'nutty', 'nyaff', 'nyala', 'nyams', 'nying', 'nylon', 'nymph', 'nyong', 'nyssa', 'nyung', 'nyuse', 'nyuze', 'oafos', 'oaked', 'oaken', 'oaker', 'oakum', 'oared', 'oarer', 'oasal', 'oases', 'oasis', 'oasts', 'oaten', 'oater', 'oaths', 'oaves', 'obang', 'obbos', 'obeah', 'obeli', 'obese', 'obeys', 'obias', 'obied', 'obiit', 'obits', 'objet', 'oboes', 'obole', 'oboli', 'obols', 'occam', 'occur', 'ocean', 'ocher', 'oches', 'ochre', 'ochry', 'ocker', 'ocote', 'ocrea', 'octad', 'octal', 'octan', 'octas', 'octet', 'octic', 'octli', 'octyl', 'oculi', 'odahs', 'odals', 'odder', 'oddly', 'odeon', 'odeum', 'odism', 'odist', 'odium', 'odoom', 'odors', 'odour', 'odums', 'odyle', 'odyls', 'ofays', 'offal', 'offed', 'offer', 'offie', 'oflag', 'often', 'ofter', 'ofuro', 'ogams', 'ogeed', 'ogees', 'oggin', 'ogham', 'ogive', 'ogled', 'ogler', 'ogles', 'ogmic', 'ogres', 'ohelo', 'ohias', 'ohing', 'ohmic', 'ohone', 'oicks', 'oidia', 'oiled', 'oiler', 'oilet', 'oinks', 'oints', 'oiran', 'ojime', 'okapi', 'okays', 'okehs', 'okies', 'oking', 'okole', 'okras', 'okrug', 'oktas', 'olate', 'olden', 'older', 'oldie', 'oldly', 'olehs', 'oleic', 'olein', 'olent', 'oleos', 'oleum', 'oleyl', 'oligo', 'olios', 'oliva', 'olive', 'ollas', 'ollav', 'oller', 'ollie', 'ology', 'olona', 'olpae', 'olpes', 'omasa', 'omber', 'ombre', 'ombus', 'omdah', 'omdas', 'omdda', 'omdeh', 'omees', 'omega', 'omens', 'omers', 'omiai', 'omits', 'omlah', 'ommel', 'ommin', 'omnes', 'omovs', 'omrah', 'omuls', 'oncer', 'onces', 'oncet', 'oncus', 'ondes', 'ondol', 'onely', 'oners', 'onery', 'ongon', 'onion', 'onium', 'onkus', 'onlap', 'onlay', 'onmun', 'onned', 'onsen', 'onset', 'ontal', 'ontic', 'ooaas', 'oobit', 'oohed', 'ooids', 'oojah', 'oomph', 'oonts', 'oopak', 'ooped', 'oopsy', 'oorie', 'ooses', 'ootid', 'ooyah', 'oozed', 'oozes', 'oozie', 'oozle', 'opahs', 'opals', 'opens', 'opepe', 'opera', 'opery', 'opgaf', 'opihi', 'opine', 'oping', 'opium', 'oppos', 'opsat', 'opsin', 'opsit', 'opted', 'opter', 'optic', 'opzit', 'orach', 'oracy', 'orals', 'orang', 'orans', 'orant', 'orate', 'orbat', 'orbed', 'orbic', 'orbit', 'orcas', 'orcin', 'order', 'ordie', 'ordos', 'oread', 'orfes', 'orful', 'organ', 'orgia', 'orgic', 'orgue', 'oribi', 'oriel', 'origo', 'orixa', 'orles', 'orlon', 'orlop', 'ormer', 'ornee', 'ornis', 'orped', 'orpin', 'orris', 'ortet', 'ortho', 'orval', 'orzos', 'osars', 'oscar', 'osetr', 'oseys', 'oshac', 'osier', 'oskin', 'oslin', 'osmic', 'osmol', 'osone', 'ossia', 'ostia', 'otaku', 'otary', 'other', 'othyl', 'otium', 'ottar', 'otter', 'ottos', 'oubit', 'ouche', 'oucht', 'oueds', 'ouens', 'ought', 'ouija', 'oulks', 'oumas', 'ounce', 'oundy', 'oupas', 'ouped', 'ouphe', 'ouphs', 'ourey', 'ourie', 'ousel', 'ousia', 'ousts', 'outby', 'outdo', 'outed', 'outen', 'outer', 'outgo', 'outie', 'outre', 'outro', 'outta', 'ouzel', 'ouzos', 'ovals', 'ovary', 'ovate', 'ovels', 'ovens', 'overs', 'overt', 'ovine', 'ovism', 'ovist', 'ovoid', 'ovoli', 'ovolo', 'ovule', 'oware', 'owari', 'owche', 'owers', 'owies', 'owing', 'owled', 'owler', 'owlet', 'owned', 'owner', 'ownio', 'owres', 'owrie', 'owsen', 'oxbow', 'oxeas', 'oxers', 'oxeye', 'oxide', 'oxids', 'oxies', 'oxime', 'oxims', 'oxine', 'oxlip', 'oxman', 'oxmen', 'oxter', 'oyama', 'oyers', 'ozeki', 'ozena', 'ozone', 'ozzie', 'paaho', 'paals', 'paans', 'pacai', 'pacas', 'pacay', 'paced', 'pacer', 'paces', 'pacey', 'pacha', 'packs', 'packy', 'pacos', 'pacta', 'pacts', 'padam', 'padas', 'paddo', 'paddy', 'padis', 'padle', 'padma', 'padou', 'padre', 'padri', 'paean', 'paedo', 'paeon', 'pagan', 'paged', 'pager', 'pages', 'pagle', 'pagne', 'pagod', 'pagri', 'pahit', 'pahos', 'pahus', 'paiks', 'pails', 'pains', 'paint', 'paipe', 'paips', 'paire', 'pairs', 'paisa', 'paise', 'pakay', 'pakka', 'pakki', 'pakua', 'pakul', 'palak', 'palar', 'palas', 'palay', 'palea', 'paled', 'paler', 'pales', 'palet', 'palis', 'palki', 'palla', 'palls', 'pallu', 'pally', 'palms', 'palmy', 'palpi', 'palps', 'palsa', 'palsy', 'palus', 'pamby', 'pampa', 'panax', 'pance', 'panch', 'panda', 'pands', 'pandy', 'paned', 'panel', 'panes', 'panga', 'pangs', 'panic', 'panim', 'panir', 'panko', 'panks', 'panna', 'panne', 'panni', 'panny', 'pansy', 'panto', 'pants', 'panty', 'paoli', 'paolo', 'papad', 'papal', 'papas', 'papaw', 'paper', 'papes', 'papey', 'pappi', 'pappy', 'papri', 'parae', 'paras', 'parch', 'parcs', 'pardi', 'pards', 'pardy', 'pared', 'paren', 'pareo', 'parer', 'pares', 'pareu', 'parev', 'parge', 'pargo', 'parid', 'paris', 'parka', 'parki', 'parks', 'parky', 'parle', 'parly', 'parma', 'parmo', 'parms', 'parol', 'parps', 'parra', 'parrs', 'parry', 'parse', 'parte', 'parti', 'parts', 'party', 'parve', 'parvo', 'pasag', 'pasar', 'pasch', 'paseo', 'pases', 'pasha', 'pashm', 'paska', 'pasmo', 'paspy', 'passe', 'passu', 'pasta', 'paste', 'pasts', 'pasty', 'patas', 'patch', 'pated', 'patee', 'patel', 'paten', 'pater', 'pates', 'paths', 'patia', 'patin', 'patio', 'patka', 'patly', 'patsy', 'patta', 'patte', 'pattu', 'patty', 'patus', 'pauas', 'pauls', 'pause', 'pauxi', 'pavan', 'pavas', 'paved', 'paven', 'paver', 'paves', 'pavid', 'pavie', 'pavin', 'pavis', 'pavon', 'pavvy', 'pawas', 'pawaw', 'pawed', 'pawer', 'pawks', 'pawky', 'pawls', 'pawns', 'paxes', 'payed', 'payee', 'payer', 'payor', 'paysd', 'peace', 'peach', 'peage', 'peags', 'peake', 'peaks', 'peaky', 'peals', 'peans', 'peare', 'pearl', 'pears', 'peart', 'pease', 'peasy', 'peats', 'peaty', 'peavy', 'peaze', 'pebas', 'pecan', 'pechs', 'pecia', 'pecke', 'pecks', 'pecky', 'pects', 'pedal', 'pedes', 'pedis', 'pedon', 'pedos', 'pedro', 'peece', 'peeks', 'peeky', 'peels', 'peely', 'peens', 'peent', 'peeoy', 'peepe', 'peeps', 'peepy', 'peers', 'peery', 'peeve', 'peevo', 'peggy', 'peghs', 'pegma', 'pegos', 'peine', 'peins', 'peise', 'peisy', 'peize', 'pekan', 'pekau', 'pekea', 'pekes', 'pekid', 'pekin', 'pekoe', 'pelas', 'pelau', 'pelch', 'peles', 'pelfs', 'pells', 'pelma', 'pelog', 'pelon', 'pelsh', 'pelta', 'pelts', 'pelus', 'penal', 'pence', 'pends', 'pendu', 'pened', 'penes', 'pengo', 'penie', 'penis', 'penks', 'penna', 'penne', 'penni', 'penny', 'pense', 'pensy', 'pents', 'peola', 'peons', 'peony', 'pepla', 'peple', 'pepon', 'pepos', 'peppy', 'pepsi', 'pequi', 'perae', 'perai', 'perce', 'perch', 'percs', 'perdu', 'perdy', 'perea', 'peres', 'perfs', 'peril', 'peris', 'perks', 'perky', 'perle', 'perls', 'perms', 'permy', 'perne', 'perns', 'perog', 'perps', 'perry', 'perse', 'persp', 'perst', 'perts', 'perve', 'pervo', 'pervs', 'pervy', 'pesch', 'pesky', 'pesos', 'pesta', 'pesto', 'pests', 'pesty', 'petal', 'petar', 'peter', 'petit', 'petos', 'petre', 'petri', 'petti', 'petto', 'petty', 'pewed', 'pewee', 'pewit', 'peyse', 'pfftt', 'phage', 'phang', 'phare', 'pharm', 'phase', 'phasm', 'pheer', 'pheme', 'phene', 'pheon', 'phese', 'phial', 'phies', 'phish', 'phizz', 'phlox', 'phobe', 'phoca', 'phone', 'phono', 'phons', 'phony', 'phooh', 'phooo', 'phota', 'photo', 'phots', 'photy', 'phpht', 'phubs', 'phuts', 'phutu', 'phwat', 'phyla', 'phyle', 'phyma', 'phynx', 'physa', 'piais', 'piani', 'piano', 'pians', 'pibal', 'pical', 'picas', 'piccy', 'picey', 'pichi', 'picks', 'picky', 'picon', 'picot', 'picra', 'picul', 'piece', 'pieds', 'piend', 'piers', 'piert', 'pieta', 'piets', 'piety', 'piezo', 'piggy', 'pight', 'pigly', 'pigmy', 'piing', 'pikas', 'pikau', 'piked', 'pikel', 'piker', 'pikes', 'pikey', 'pikis', 'pikul', 'pilae', 'pilaf', 'pilao', 'pilar', 'pilau', 'pilaw', 'pilch', 'pilea', 'piled', 'pilei', 'piler', 'piles', 'piley', 'pilin', 'pilis', 'pills', 'pilon', 'pilot', 'pilow', 'pilum', 'pilus', 'pimas', 'pimps', 'pinas', 'pinax', 'pince', 'pinch', 'pinda', 'pinds', 'pined', 'piner', 'pines', 'piney', 'pinga', 'pinge', 'pingo', 'pings', 'pinko', 'pinks', 'pinky', 'pinna', 'pinny', 'pinol', 'pinon', 'pinot', 'pinta', 'pinto', 'pints', 'pinup', 'pions', 'piony', 'pious', 'pioye', 'pioys', 'pipal', 'pipas', 'piped', 'piper', 'pipes', 'pipet', 'pipid', 'pipis', 'pipit', 'pippy', 'pipul', 'pique', 'piqui', 'pirai', 'pirks', 'pirls', 'pirns', 'pirog', 'pirre', 'pirri', 'pirrs', 'pisco', 'pises', 'pisky', 'pisos', 'pissy', 'piste', 'pitas', 'pitch', 'piths', 'pithy', 'piton', 'pitot', 'pitso', 'pitsu', 'pitta', 'pittu', 'piuma', 'piums', 'pivos', 'pivot', 'pixel', 'pixes', 'pixie', 'piyut', 'pized', 'pizer', 'pizes', 'pizza', 'plaas', 'place', 'plack', 'plaga', 'plage', 'plaid', 'plaig', 'plain', 'plait', 'planc', 'plane', 'planh', 'plank', 'plans', 'plant', 'plaps', 'plash', 'plasm', 'plast', 'plate', 'plats', 'platt', 'platy', 'plaud', 'plaur', 'plavs', 'playa', 'plays', 'plaza', 'plead', 'pleas', 'pleat', 'plebe', 'plebs', 'pleck', 'pleep', 'plein', 'plena', 'plene', 'pleno', 'pleon', 'plesh', 'plets', 'plews', 'plexi', 'plica', 'plied', 'plier', 'plies', 'pligs', 'plims', 'pling', 'plink', 'plips', 'plish', 'ploat', 'ploce', 'plock', 'plods', 'ploit', 'plomb', 'plong', 'plonk', 'plook', 'ploot', 'plops', 'plore', 'plots', 'plotz', 'plouk', 'plout', 'plows', 'plowt', 'ploye', 'ploys', 'pluck', 'pluds', 'plues', 'pluff', 'plugs', 'pluke', 'plumb', 'plume', 'plump', 'plums', 'plumy', 'plung', 'plunk', 'pluot', 'plups', 'plush', 'plute', 'pluto', 'pluty', 'plyer', 'pneus', 'poach', 'poaka', 'poake', 'poalo', 'pobby', 'poboy', 'pocan', 'poche', 'pocho', 'pocks', 'pocky', 'podal', 'poddy', 'podex', 'podge', 'podgy', 'podia', 'podos', 'podus', 'poems', 'poena', 'poeps', 'poesy', 'poete', 'poets', 'pogey', 'pogge', 'poggy', 'pogos', 'pogue', 'pohed', 'poilu', 'poind', 'point', 'poire', 'poise', 'pokal', 'poked', 'poker', 'pokes', 'pokey', 'pokie', 'pokit', 'polar', 'poled', 'poler', 'poles', 'poley', 'polio', 'polis', 'polje', 'polka', 'polks', 'pollo', 'polls', 'polly', 'polos', 'polts', 'polyp', 'polys', 'pomas', 'pombe', 'pomes', 'pomme', 'pommy', 'pomos', 'pompa', 'pomps', 'ponce', 'poncy', 'ponds', 'pondy', 'pones', 'poney', 'ponga', 'pongo', 'pongs', 'pongy', 'ponks', 'ponor', 'ponto', 'ponts', 'ponty', 'ponzu', 'pooay', 'pooch', 'poods', 'pooed', 'pooey', 'poofs', 'poofy', 'poohs', 'poohy', 'pooja', 'pooka', 'pooks', 'pools', 'pooly', 'poons', 'poopa', 'poops', 'poopy', 'poori', 'poort', 'poots', 'pooty', 'poove', 'poovy', 'popes', 'popia', 'popos', 'poppa', 'poppy', 'popsy', 'popup', 'porae', 'poral', 'porch', 'pored', 'porer', 'pores', 'porey', 'porge', 'porgy', 'porin', 'porks', 'porky', 'porno', 'porns', 'porny', 'porta', 'porte', 'porth', 'ports', 'porty', 'porus', 'posca', 'posed', 'poser', 'poses', 'poset', 'posey', 'posho', 'posit', 'posol', 'posse', 'poste', 'posts', 'potae', 'potai', 'potch', 'poted', 'potes', 'potin', 'potoo', 'potro', 'potsy', 'potto', 'potts', 'potty', 'pouce', 'pouch', 'pouff', 'poufs', 'poufy', 'pouis', 'pouke', 'pouks', 'poule', 'poulp', 'poult', 'pound', 'poupe', 'poupt', 'pours', 'pousy', 'pouts', 'pouty', 'povos', 'powan', 'power', 'powie', 'powin', 'powis', 'powlt', 'pownd', 'powns', 'powny', 'powre', 'powsy', 'poxed', 'poxes', 'poyas', 'poynt', 'poyou', 'poyse', 'pozzy', 'praam', 'prads', 'prags', 'prahu', 'prams', 'prana', 'prang', 'prank', 'praos', 'praps', 'prase', 'prate', 'prats', 'pratt', 'praty', 'praus', 'prawn', 'prays', 'preak', 'predy', 'preed', 'preem', 'preen', 'prees', 'preif', 'preke', 'prems', 'premy', 'prent', 'preon', 'preop', 'preps', 'presa', 'prese', 'press', 'prest', 'preta', 'preux', 'preve', 'prexy', 'preys', 'prial', 'prian', 'price', 'prick', 'pricy', 'pride', 'pridy', 'pried', 'prief', 'prier', 'pries', 'prigs', 'prill', 'prima', 'prime', 'primi', 'primo', 'primp', 'prims', 'primy', 'pring', 'prink', 'print', 'prion', 'prior', 'prise', 'prism', 'priss', 'prius', 'privy', 'prize', 'proal', 'proas', 'probe', 'probs', 'proby', 'prodd', 'prods', 'proem', 'profs', 'progs', 'proin', 'proke', 'prole', 'proll', 'promo', 'proms', 'prone', 'prong', 'pronk', 'proof', 'prook', 'proot', 'props', 'prora', 'prore', 'prose', 'proso', 'pross', 'prost', 'prosy', 'proto', 'proud', 'proul', 'prove', 'prowk', 'prowl', 'prows', 'proxy', 'proyn', 'prude', 'prune', 'pruno', 'prunt', 'pruny', 'pruta', 'pryan', 'pryer', 'pryse', 'psalm', 'pseud', 'pshaw', 'pshut', 'psias', 'psion', 'psoae', 'psoai', 'psoas', 'psora', 'psych', 'psyop', 'ptish', 'ptype', 'pubby', 'pubco', 'pubes', 'pubic', 'pubis', 'pubsy', 'pucan', 'pucer', 'puces', 'pucka', 'pucks', 'puddy', 'pudge', 'pudgy', 'pudic', 'pudor', 'pudsy', 'pudus', 'puers', 'puffa', 'puffs', 'puffy', 'puggy', 'pugil', 'puhas', 'pujah', 'pujas', 'pukas', 'puked', 'puker', 'pukes', 'pukey', 'pukka', 'pukus', 'pulao', 'pulas', 'puled', 'puler', 'pules', 'pulik', 'pulis', 'pulka', 'pulks', 'pulli', 'pulls', 'pully', 'pulmo', 'pulps', 'pulpy', 'pulse', 'pulus', 'pulut', 'pumas', 'pumie', 'pumps', 'pumpy', 'punas', 'punce', 'punch', 'punga', 'pungi', 'pungo', 'pungs', 'pungy', 'punim', 'punji', 'punka', 'punks', 'punky', 'punny', 'punto', 'punts', 'punty', 'pupae', 'pupal', 'pupas', 'pupil', 'puppa', 'puppy', 'pupus', 'purao', 'purau', 'purda', 'purdy', 'pured', 'puree', 'purer', 'pures', 'purga', 'purge', 'purin', 'puris', 'purls', 'puros', 'purps', 'purpy', 'purre', 'purrs', 'purry', 'purse', 'pursy', 'purty', 'puses', 'pushy', 'pusle', 'pussy', 'putas', 'puter', 'putid', 'putin', 'puton', 'putos', 'putti', 'putto', 'putts', 'puttu', 'putty', 'putza', 'puuko', 'puyas', 'puzel', 'puzta', 'pwned', 'pyats', 'pyets', 'pygal', 'pygmy', 'pyins', 'pylon', 'pyned', 'pynes', 'pyoid', 'pyots', 'pyral', 'pyran', 'pyres', 'pyrex', 'pyric', 'pyros', 'pyrus', 'pyuff', 'pyxed', 'pyxes', 'pyxie', 'pyxis', 'pzazz', 'qadis', 'qaids', 'qajaq', 'qanat', 'qapik', 'qibla', 'qilas', 'qipao', 'qophs', 'qorma', 'quabs', 'quack', 'quads', 'quaff', 'quags', 'quail', 'quair', 'quais', 'quake', 'quaky', 'quale', 'qualm', 'qualy', 'quank', 'quant', 'quare', 'quark', 'quarl', 'quart', 'quash', 'quasi', 'quass', 'quate', 'quats', 'quawk', 'quaws', 'quayd', 'quays', 'qubit', 'quean', 'queck', 'queek', 'queem', 'queen', 'queer', 'quell', 'queme', 'quena', 'quern', 'query', 'queso', 'quest', 'quete', 'queue', 'queyn', 'queys', 'queyu', 'quibs', 'quich', 'quick', 'quids', 'quies', 'quiet', 'quiff', 'quila', 'quill', 'quilt', 'quims', 'quina', 'quine', 'quink', 'quino', 'quins', 'quint', 'quipo', 'quips', 'quipu', 'quire', 'quirk', 'quirl', 'quirt', 'quist', 'quite', 'quits', 'quoad', 'quods', 'quoif', 'quoin', 'quois', 'quoit', 'quoll', 'quonk', 'quops', 'quork', 'quorl', 'quota', 'quote', 'quoth', 'quouk', 'quoys', 'quran', 'qursh', 'quyte', 'raads', 'raake', 'rabat', 'rabbi', 'rabic', 'rabid', 'rabis', 'raced', 'racer', 'races', 'rache', 'racks', 'racon', 'radar', 'raddi', 'raddy', 'radge', 'radgy', 'radif', 'radii', 'radio', 'radix', 'radon', 'rafee', 'raffs', 'raffy', 'rafik', 'rafiq', 'rafts', 'rafty', 'ragas', 'ragde', 'raged', 'ragee', 'rager', 'rages', 'ragga', 'raggs', 'raggy', 'ragis', 'ragus', 'rahed', 'rahui', 'raiah', 'raias', 'raids', 'raike', 'raiks', 'raile', 'rails', 'raine', 'rains', 'rainy', 'raird', 'raise', 'raita', 'raith', 'raits', 'rajah', 'rajas', 'rajes', 'raked', 'rakee', 'raker', 'rakes', 'rakhi', 'rakia', 'rakis', 'rakki', 'raksi', 'rakus', 'rales', 'ralli', 'rally', 'ralph', 'ramal', 'ramee', 'ramen', 'rames', 'ramet', 'ramie', 'ramin', 'ramis', 'rammy', 'ramon', 'ramps', 'ramse', 'ramsh', 'ramus', 'ranas', 'rance', 'ranch', 'rando', 'rands', 'randy', 'raned', 'ranee', 'ranes', 'ranga', 'range', 'rangi', 'rangs', 'rangy', 'ranid', 'ranis', 'ranke', 'ranks', 'ranns', 'ranny', 'ranse', 'rants', 'ranty', 'raped', 'rapee', 'raper', 'rapes', 'raphe', 'rapid', 'rapin', 'rappe', 'rapso', 'rared', 'raree', 'rarer', 'rares', 'rarks', 'rasam', 'rasas', 'rased', 'raser', 'rases', 'rasps', 'raspy', 'rasse', 'rasta', 'ratal', 'ratan', 'ratas', 'ratch', 'rated', 'ratel', 'rater', 'rates', 'ratha', 'rathe', 'raths', 'ratio', 'ratoo', 'ratos', 'ratti', 'ratty', 'ratus', 'rauli', 'rauns', 'raupo', 'raved', 'ravel', 'raven', 'raver', 'raves', 'ravey', 'ravin', 'rawdy', 'rawer', 'rawin', 'rawks', 'rawly', 'rawns', 'raxed', 'raxes', 'rayah', 'rayas', 'rayed', 'rayle', 'rayls', 'rayne', 'rayon', 'razai', 'razed', 'razee', 'razer', 'razes', 'razet', 'razoo', 'razor', 'reach', 'react', 'readd', 'reads', 'ready', 'reais', 'reaks', 'realm', 'realo', 'reals', 'reame', 'reams', 'reamy', 'reans', 'reaps', 'reard', 'rearm', 'rears', 'reast', 'reata', 'reate', 'reave', 'rebab', 'rebar', 'rebbe', 'rebec', 'rebel', 'rebid', 'rebit', 'rebop', 'rebud', 'rebus', 'rebut', 'rebuy', 'recal', 'recap', 'recce', 'recco', 'reccy', 'recep', 'recit', 'recks', 'recon', 'recta', 'recte', 'recti', 'recto', 'recue', 'recur', 'recut', 'redan', 'redds', 'reddy', 'reded', 'redes', 'redia', 'redid', 'redif', 'redig', 'redip', 'redly', 'redon', 'redos', 'redox', 'redry', 'redub', 'redug', 'redux', 'redye', 'reeaf', 'reech', 'reede', 'reeds', 'reedy', 'reefs', 'reefy', 'reeks', 'reeky', 'reels', 'reely', 'reems', 'reens', 'reerd', 'reest', 'reeve', 'reeze', 'refan', 'refed', 'refel', 'refer', 'reffo', 'refis', 'refit', 'refix', 'refly', 'refry', 'regal', 'regar', 'reges', 'reget', 'regex', 'reggo', 'regia', 'regie', 'regle', 'regma', 'regna', 'regos', 'regot', 'regur', 'rehab', 'rehem', 'reifs', 'reify', 'reign', 'reiki', 'reiks', 'reine', 'reing', 'reink', 'reins', 'reird', 'reist', 'reive', 'rejas', 'rejig', 'rejon', 'reked', 'rekes', 'rekey', 'relax', 'relay', 'relet', 'relic', 'relie', 'relit', 'rello', 'relos', 'reman', 'remap', 'remen', 'remet', 'remex', 'remit', 'remix', 'remou', 'renal', 'renay', 'rends', 'rendu', 'renew', 'reney', 'renga', 'rengs', 'renig', 'renin', 'renks', 'renne', 'renos', 'rente', 'rents', 'reoil', 'reorg', 'repas', 'repat', 'repay', 'repeg', 'repel', 'repen', 'repin', 'repla', 'reply', 'repos', 'repot', 'repps', 'repro', 'repun', 'reput', 'reran', 'rerig', 'rerun', 'resam', 'resat', 'resaw', 'resay', 'resee', 'reses', 'reset', 'resew', 'resid', 'resin', 'resit', 'resod', 'resol', 'resow', 'resto', 'rests', 'resty', 'resue', 'resus', 'retag', 'retam', 'retax', 'retch', 'retem', 'retia', 'retie', 'retin', 'retip', 'retox', 'retro', 'retry', 'reune', 'reups', 'reuse', 'revel', 'revet', 'revie', 'revow', 'revue', 'rewan', 'rewax', 'rewed', 'rewet', 'rewin', 'rewon', 'rewth', 'rexes', 'rezes', 'rhabd', 'rheas', 'rheid', 'rheme', 'rheum', 'rhies', 'rhime', 'rhine', 'rhino', 'rhody', 'rhomb', 'rhone', 'rhumb', 'rhyme', 'rhymy', 'rhyne', 'rhyta', 'riads', 'rials', 'riant', 'riata', 'riato', 'ribas', 'ribby', 'ribes', 'riced', 'ricer', 'rices', 'ricey', 'riche', 'richt', 'ricin', 'ricks', 'rider', 'rides', 'ridge', 'ridgy', 'ridic', 'riels', 'riems', 'rieve', 'rifer', 'riffs', 'riffy', 'rifle', 'rifte', 'rifts', 'rifty', 'riggs', 'right', 'rigid', 'rigmo', 'rigol', 'rigor', 'rikka', 'rikwa', 'riled', 'riles', 'riley', 'rille', 'rills', 'rilly', 'rimae', 'rimed', 'rimer', 'rimes', 'rimon', 'rimus', 'rince', 'rinds', 'rindy', 'rines', 'ringe', 'rings', 'ringy', 'rinks', 'rinse', 'rioja', 'rione', 'riots', 'rioty', 'riped', 'ripen', 'riper', 'ripes', 'ripps', 'riqqs', 'risen', 'riser', 'rises', 'rishi', 'risks', 'risky', 'risps', 'rists', 'risus', 'rites', 'rithe', 'ritts', 'ritzy', 'rival', 'rivas', 'rived', 'rivel', 'riven', 'river', 'rives', 'rivet', 'riyal', 'rizas', 'roach', 'roads', 'roady', 'roake', 'roaky', 'roams', 'roans', 'roany', 'roars', 'roary', 'roast', 'roate', 'robbo', 'robed', 'rober', 'robes', 'robin', 'roble', 'robot', 'robug', 'robur', 'roche', 'rocks', 'rocky', 'roded', 'rodeo', 'rodes', 'rodny', 'roers', 'rogan', 'roger', 'rogue', 'roguy', 'rohan', 'rohes', 'rohun', 'rohus', 'roids', 'roils', 'roily', 'roins', 'roist', 'rojak', 'rojis', 'roked', 'roker', 'rokes', 'rokey', 'rokos', 'rolag', 'roleo', 'roles', 'rolfs', 'rolls', 'rolly', 'romal', 'roman', 'romeo', 'romer', 'romps', 'rompu', 'rompy', 'ronde', 'rondo', 'roneo', 'rones', 'ronin', 'ronne', 'ronte', 'ronts', 'ronuk', 'roods', 'roofs', 'roofy', 'rooks', 'rooky', 'rooms', 'roomy', 'roons', 'roops', 'roopy', 'roosa', 'roose', 'roost', 'roots', 'rooty', 'roped', 'roper', 'ropes', 'ropey', 'roque', 'roral', 'rores', 'roric', 'rorid', 'rorie', 'rorts', 'rorty', 'rosal', 'rosco', 'rosed', 'roses', 'roset', 'rosha', 'roshi', 'rosin', 'rosit', 'rosps', 'rossa', 'rosso', 'rosti', 'rosts', 'rotal', 'rotan', 'rotas', 'rotch', 'roted', 'rotes', 'rotis', 'rotls', 'roton', 'rotor', 'rotos', 'rotta', 'rotte', 'rotto', 'rotty', 'rouen', 'roues', 'rouet', 'roufs', 'rouge', 'rough', 'rougy', 'rouks', 'rouky', 'roule', 'rouls', 'roums', 'round', 'roups', 'roupy', 'rouse', 'roust', 'route', 'routh', 'routs', 'roved', 'roven', 'rover', 'roves', 'rowan', 'rowdy', 'rowed', 'rowel', 'rowen', 'rower', 'rowet', 'rowie', 'rowme', 'rownd', 'rowns', 'rowth', 'rowts', 'royal', 'royet', 'royne', 'royst', 'rozes', 'rozet', 'rozit', 'ruach', 'ruana', 'rubai', 'ruban', 'rubby', 'rubel', 'rubes', 'rubin', 'rubio', 'ruble', 'rubli', 'rubor', 'rubus', 'ruche', 'ruchy', 'rucks', 'rudas', 'rudds', 'ruddy', 'ruder', 'rudes', 'rudie', 'rudis', 'rueda', 'ruers', 'ruffe', 'ruffs', 'ruffy', 'rufus', 'rugae', 'rugal', 'rugas', 'rugby', 'ruggy', 'ruice', 'ruing', 'ruins', 'rukhs', 'ruled', 'ruler', 'rules', 'rully', 'rumal', 'rumba', 'rumbo', 'rumen', 'rumes', 'rumly', 'rummy', 'rumor', 'rumpo', 'rumps', 'rumpy', 'runce', 'runch', 'runds', 'runed', 'runer', 'runes', 'rungs', 'runic', 'runny', 'runos', 'runts', 'runty', 'runup', 'ruote', 'rupee', 'rupia', 'rural', 'rurps', 'rurus', 'rusas', 'ruses', 'rushy', 'rusks', 'rusky', 'rusma', 'russe', 'rusts', 'rusty', 'ruths', 'rutin', 'rutty', 'ruvid', 'ryals', 'rybat', 'ryiji', 'ryijy', 'ryked', 'rykes', 'rymer', 'rymme', 'rynds', 'ryoti', 'ryots', 'ryper', 'rypin', 'rythe', 'ryugi', 'saags', 'sabal', 'sabed', 'saber', 'sabes', 'sabha', 'sabin', 'sabir', 'sabji', 'sable', 'sabos', 'sabot', 'sabra', 'sabre', 'sabzi', 'sacks', 'sacra', 'sacre', 'saddo', 'saddy', 'sades', 'sadhe', 'sadhu', 'sadic', 'sadis', 'sadly', 'sados', 'sadza', 'saeta', 'safed', 'safer', 'safes', 'sagar', 'sagas', 'sager', 'sages', 'saggy', 'sagos', 'sagum', 'sahab', 'saheb', 'sahib', 'saice', 'saick', 'saics', 'saids', 'saiga', 'sails', 'saims', 'saine', 'sains', 'saint', 'sairs', 'saist', 'saith', 'sajou', 'sakai', 'saker', 'sakes', 'sakia', 'sakis', 'sakti', 'salad', 'salal', 'salas', 'salat', 'salep', 'sales', 'salet', 'salic', 'salis', 'salix', 'salle', 'sally', 'salmi', 'salol', 'salon', 'salop', 'salpa', 'salps', 'salsa', 'salse', 'salto', 'salts', 'salty', 'salud', 'salue', 'salut', 'salve', 'salvo', 'saman', 'samas', 'samba', 'sambo', 'samek', 'samel', 'samen', 'sames', 'samey', 'samfi', 'samfu', 'sammy', 'sampi', 'samps', 'sanad', 'sands', 'sandy', 'saned', 'saner', 'sanes', 'sanga', 'sangh', 'sango', 'sangs', 'sanko', 'sansa', 'santo', 'sants', 'saola', 'sapan', 'sapid', 'sapor', 'sappy', 'saran', 'sards', 'sared', 'saree', 'sarge', 'sargo', 'sarin', 'sarir', 'saris', 'sarks', 'sarky', 'sarod', 'saros', 'sarus', 'sarvo', 'saser', 'sasin', 'sasse', 'sassy', 'satai', 'satay', 'sated', 'satem', 'sater', 'sates', 'satin', 'satis', 'satyr', 'sauba', 'sauce', 'sauch', 'saucy', 'saugh', 'sauls', 'sault', 'sauna', 'saunf', 'saunt', 'saury', 'saute', 'sauts', 'sauve', 'saved', 'saver', 'saves', 'savey', 'savin', 'savor', 'savoy', 'savvy', 'sawah', 'sawed', 'sawer', 'saxes', 'sayas', 'sayed', 'sayee', 'sayer', 'sayid', 'sayne', 'sayon', 'sayst', 'sazes', 'scabs', 'scads', 'scaff', 'scags', 'scail', 'scala', 'scald', 'scale', 'scall', 'scalp', 'scaly', 'scamp', 'scams', 'scand', 'scans', 'scant', 'scapa', 'scape', 'scapi', 'scare', 'scarf', 'scarp', 'scars', 'scart', 'scary', 'scath', 'scats', 'scatt', 'scaud', 'scaup', 'scaur', 'scaws', 'sceat', 'scena', 'scend', 'scene', 'scent', 'schav', 'schif', 'schmo', 'schul', 'schwa', 'scifi', 'scind', 'scion', 'scire', 'sclim', 'scobe', 'scody', 'scoff', 'scogs', 'scold', 'scone', 'scoog', 'scoop', 'scoot', 'scopa', 'scope', 'scops', 'score', 'scorn', 'scorp', 'scote', 'scots', 'scoug', 'scoup', 'scour', 'scout', 'scowl', 'scowp', 'scows', 'scrab', 'scrae', 'scrag', 'scram', 'scran', 'scrap', 'scrat', 'scraw', 'scray', 'scree', 'screw', 'scrim', 'scrip', 'scrob', 'scrod', 'scrog', 'scroo', 'scrow', 'scrub', 'scrum', 'scuba', 'scudi', 'scudo', 'scuds', 'scuff', 'scuft', 'scugs', 'sculk', 'scull', 'sculp', 'sculs', 'scums', 'scups', 'scurf', 'scurs', 'scuse', 'scuta', 'scute', 'scuts', 'scuzz', 'scyes', 'sdayn', 'sdein', 'seals', 'seame', 'seams', 'seamy', 'seans', 'seare', 'sears', 'sease', 'seats', 'seaze', 'sebum', 'secco', 'sechs', 'sects', 'sedan', 'seder', 'sedes', 'sedge', 'sedgy', 'sedum', 'seeds', 'seedy', 'seeks', 'seeld', 'seels', 'seely', 'seems', 'seeps', 'seepy', 'seers', 'sefer', 'segar', 'segas', 'segni', 'segno', 'segol', 'segos', 'segue', 'sehri', 'seifs', 'seils', 'seine', 'seirs', 'seise', 'seism', 'seity', 'seiza', 'seize', 'sekos', 'sekts', 'selah', 'seles', 'selfs', 'selfy', 'selky', 'sella', 'selle', 'sells', 'selva', 'semas', 'semee', 'semen', 'semes', 'semie', 'semis', 'senas', 'sends', 'senes', 'senex', 'sengi', 'senna', 'senor', 'sensa', 'sense', 'sensi', 'sensu', 'sente', 'senti', 'sents', 'senvy', 'senza', 'sepad', 'sepal', 'sepia', 'sepic', 'sepoy', 'seppo', 'septa', 'septs', 'serac', 'serai', 'seral', 'sered', 'serer', 'seres', 'serfs', 'serge', 'seria', 'seric', 'serif', 'serin', 'serir', 'serks', 'seron', 'serow', 'serra', 'serre', 'serrs', 'serry', 'serum', 'serve', 'servo', 'sesey', 'sessa', 'setae', 'setal', 'seter', 'seths', 'seton', 'setts', 'setup', 'sevak', 'seven', 'sever', 'sevir', 'sewan', 'sewar', 'sewed', 'sewel', 'sewen', 'sewer', 'sewin', 'sexed', 'sexer', 'sexes', 'sexor', 'sexto', 'sexts', 'seyen', 'sezes', 'shack', 'shade', 'shads', 'shady', 'shaft', 'shags', 'shahs', 'shaka', 'shake', 'shako', 'shakt', 'shaky', 'shale', 'shall', 'shalm', 'shalt', 'shaly', 'shama', 'shame', 'shams', 'shand', 'shank', 'shans', 'shape', 'shaps', 'shard', 'share', 'shark', 'sharn', 'sharp', 'shart', 'shash', 'shaul', 'shave', 'shawl', 'shawm', 'shawn', 'shaws', 'shaya', 'shays', 'shchi', 'sheaf', 'sheal', 'shear', 'sheas', 'sheds', 'sheel', 'sheen', 'sheep', 'sheer', 'sheet', 'sheik', 'shelf', 'shell', 'shend', 'sheng', 'shent', 'sheol', 'sherd', 'shere', 'shero', 'shets', 'sheva', 'shewn', 'shews', 'shiai', 'shied', 'shiel', 'shier', 'shies', 'shift', 'shill', 'shily', 'shims', 'shine', 'shins', 'shiny', 'shiok', 'ships', 'shire', 'shirk', 'shirr', 'shirs', 'shirt', 'shish', 'shiso', 'shist', 'shite', 'shits', 'shiur', 'shiva', 'shive', 'shivs', 'shlep', 'shlub', 'shmek', 'shmoe', 'shoal', 'shoat', 'shock', 'shoed', 'shoer', 'shoes', 'shogi', 'shogs', 'shoji', 'shojo', 'shola', 'shone', 'shonk', 'shook', 'shool', 'shoon', 'shoos', 'shoot', 'shope', 'shops', 'shore', 'shorl', 'shorn', 'short', 'shote', 'shots', 'shott', 'shoud', 'shout', 'shove', 'showd', 'shown', 'shows', 'showy', 'shoyu', 'shred', 'shrew', 'shris', 'shrow', 'shrub', 'shrug', 'shtar', 'shtik', 'shtum', 'shtup', 'shuba', 'shuck', 'shule', 'shuln', 'shuls', 'shuns', 'shunt', 'shura', 'shush', 'shute', 'shuts', 'shwas', 'shyer', 'shyly', 'sials', 'sibbs', 'sibia', 'sibyl', 'sices', 'sicht', 'sicko', 'sicks', 'sicky', 'sidas', 'sided', 'sider', 'sides', 'sidey', 'sidha', 'sidhe', 'sidle', 'siege', 'sield', 'siens', 'sient', 'sieth', 'sieur', 'sieve', 'sifts', 'sighs', 'sight', 'sigil', 'sigla', 'sigma', 'signa', 'signs', 'sigri', 'sijos', 'sikas', 'siker', 'sikes', 'silds', 'siled', 'silen', 'siler', 'siles', 'silex', 'silks', 'silky', 'sills', 'silly', 'silos', 'silts', 'silty', 'silva', 'simar', 'simas', 'simba', 'simis', 'simps', 'simul', 'since', 'sinds', 'sined', 'sines', 'sinew', 'singe', 'sings', 'sinhs', 'sinks', 'sinky', 'sinsi', 'sinus', 'siped', 'sipes', 'sippy', 'sired', 'siree', 'siren', 'sires', 'sirih', 'siris', 'siroc', 'sirra', 'sirup', 'sisal', 'sises', 'sissy', 'sista', 'sists', 'sitar', 'sitch', 'sited', 'sites', 'sithe', 'sitka', 'situp', 'situs', 'siver', 'sixer', 'sixes', 'sixmo', 'sixte', 'sixth', 'sixty', 'sizar', 'sized', 'sizel', 'sizer', 'sizes', 'skags', 'skail', 'skald', 'skank', 'skarn', 'skart', 'skate', 'skats', 'skatt', 'skaws', 'skean', 'skear', 'skeds', 'skeed', 'skeef', 'skeen', 'skeer', 'skees', 'skeet', 'skeev', 'skeez', 'skegg', 'skegs', 'skein', 'skelf', 'skell', 'skelm', 'skelp', 'skene', 'skens', 'skeos', 'skeps', 'skerm', 'skers', 'skets', 'skews', 'skids', 'skied', 'skier', 'skies', 'skiey', 'skiff', 'skill', 'skimo', 'skimp', 'skims', 'skink', 'skins', 'skint', 'skios', 'skips', 'skirl', 'skirr', 'skirt', 'skite', 'skits', 'skive', 'skivy', 'sklim', 'skoal', 'skobe', 'skody', 'skoff', 'skofs', 'skogs', 'skols', 'skool', 'skort', 'skosh', 'skran', 'skrik', 'skroo', 'skuas', 'skugs', 'skulk', 'skull', 'skunk', 'skyed', 'skyer', 'skyey', 'skyfs', 'skyre', 'skyrs', 'skyte', 'slabs', 'slack', 'slade', 'slaes', 'slags', 'slaid', 'slain', 'slake', 'slams', 'slane', 'slang', 'slank', 'slant', 'slaps', 'slart', 'slash', 'slate', 'slats', 'slaty', 'slave', 'slaws', 'slays', 'slebs', 'sleds', 'sleek', 'sleep', 'sleer', 'sleet', 'slept', 'slews', 'sleys', 'slice', 'slick', 'slide', 'slier', 'slily', 'slime', 'slims', 'slimy', 'sling', 'slink', 'slipe', 'slips', 'slipt', 'slish', 'slits', 'slive', 'sloan', 'slobs', 'sloes', 'slogs', 'sloid', 'slojd', 'sloka', 'slomo', 'sloom', 'sloop', 'sloot', 'slope', 'slops', 'slopy', 'slorm', 'slosh', 'sloth', 'slots', 'slove', 'slows', 'sloyd', 'slubb', 'slubs', 'slued', 'slues', 'sluff', 'slugs', 'sluit', 'slump', 'slums', 'slung', 'slunk', 'slurb', 'slurp', 'slurs', 'sluse', 'slush', 'sluts', 'slyer', 'slyly', 'slype', 'smaak', 'smack', 'smaik', 'small', 'smalm', 'smalt', 'smarm', 'smart', 'smash', 'smaze', 'smear', 'smeek', 'smees', 'smeik', 'smeke', 'smell', 'smelt', 'smerk', 'smews', 'smick', 'smile', 'smily', 'smirk', 'smirr', 'smirs', 'smite', 'smith', 'smits', 'smize', 'smock', 'smogs', 'smoke', 'smoko', 'smoky', 'smolt', 'smoor', 'smoot', 'smore', 'smorg', 'smote', 'smout', 'smowt', 'smugs', 'smurs', 'smush', 'smuts', 'snabs', 'snack', 'snafu', 'snags', 'snail', 'snake', 'snaky', 'snaps', 'snare', 'snarf', 'snark', 'snarl', 'snars', 'snary', 'snash', 'snath', 'snaws', 'snead', 'sneak', 'sneap', 'snebs', 'sneck', 'sneds', 'sneed', 'sneer', 'snees', 'snell', 'snibs', 'snick', 'snide', 'snied', 'snies', 'sniff', 'snift', 'snigs', 'snipe', 'snips', 'snipy', 'snirt', 'snits', 'snive', 'snobs', 'snods', 'snoek', 'snoep', 'snogs', 'snoke', 'snood', 'snook', 'snool', 'snoop', 'snoot', 'snore', 'snort', 'snots', 'snout', 'snowk', 'snows', 'snowy', 'snubs', 'snuck', 'snuff', 'snugs', 'snush', 'snyes', 'soaks', 'soaps', 'soapy', 'soare', 'soars', 'soave', 'sobas', 'sober', 'socas', 'soces', 'socia', 'socko', 'socks', 'socle', 'sodas', 'soddy', 'sodic', 'sodom', 'sofar', 'sofas', 'softa', 'softs', 'softy', 'soger', 'soggy', 'sohur', 'soils', 'soily', 'sojas', 'sojus', 'sokah', 'soken', 'sokes', 'sokol', 'solah', 'solan', 'solar', 'solas', 'solde', 'soldi', 'soldo', 'solds', 'soled', 'solei', 'soler', 'soles', 'solid', 'solon', 'solos', 'solum', 'solus', 'solve', 'soman', 'somas', 'sonar', 'sonce', 'sonde', 'sones', 'songo', 'songs', 'songy', 'sonic', 'sonly', 'sonne', 'sonny', 'sonse', 'sonsy', 'sooey', 'sooks', 'sooky', 'soole', 'sools', 'sooms', 'soops', 'soote', 'sooth', 'soots', 'sooty', 'sophs', 'sophy', 'sopor', 'soppy', 'sopra', 'soral', 'soras', 'sorbi', 'sorbo', 'sorbs', 'sorda', 'sordo', 'sords', 'sored', 'soree', 'sorel', 'sorer', 'sores', 'sorex', 'sorgo', 'sorns', 'sorra', 'sorry', 'sorta', 'sorts', 'sorus', 'soths', 'sotol', 'sotto', 'souce', 'souct', 'sough', 'souks', 'souls', 'souly', 'soums', 'sound', 'soups', 'soupy', 'sours', 'souse', 'south', 'souts', 'sowar', 'sowce', 'sowed', 'sower', 'sowff', 'sowfs', 'sowle', 'sowls', 'sowms', 'sownd', 'sowne', 'sowps', 'sowse', 'sowth', 'soxes', 'soyas', 'soyle', 'soyuz', 'sozin', 'space', 'spack', 'spacy', 'spade', 'spado', 'spads', 'spaed', 'spaer', 'spaes', 'spags', 'spahi', 'spail', 'spain', 'spait', 'spake', 'spald', 'spale', 'spall', 'spalt', 'spams', 'spane', 'spang', 'spank', 'spans', 'spard', 'spare', 'spark', 'spars', 'spart', 'spasm', 'spate', 'spats', 'spaul', 'spawl', 'spawn', 'spaws', 'spayd', 'spays', 'spaza', 'spazz', 'speak', 'speal', 'spean', 'spear', 'speat', 'speck', 'specs', 'spect', 'speed', 'speel', 'speer', 'speil', 'speir', 'speks', 'speld', 'spelk', 'spell', 'spelt', 'spend', 'spent', 'speos', 'sperm', 'spesh', 'spets', 'speug', 'spews', 'spewy', 'spial', 'spica', 'spice', 'spick', 'spics', 'spicy', 'spide', 'spied', 'spiel', 'spier', 'spies', 'spiff', 'spifs', 'spike', 'spiks', 'spiky', 'spile', 'spill', 'spilt', 'spims', 'spina', 'spine', 'spink', 'spins', 'spiny', 'spire', 'spirt', 'spiry', 'spite', 'spits', 'spitz', 'spivs', 'splat', 'splay', 'split', 'splog', 'spode', 'spods', 'spoil', 'spoke', 'spoof', 'spook', 'spool', 'spoom', 'spoon', 'spoor', 'spoot', 'spore', 'spork', 'sport', 'sposa', 'sposh', 'sposo', 'spots', 'spout', 'sprad', 'sprag', 'sprat', 'spray', 'spred', 'spree', 'sprew', 'sprig', 'sprit', 'sprod', 'sprog', 'sprue', 'sprug', 'spuds', 'spued', 'spuer', 'spues', 'spugs', 'spule', 'spume', 'spumy', 'spunk', 'spurn', 'spurs', 'spurt', 'sputa', 'spyal', 'spyre', 'squab', 'squad', 'squat', 'squaw', 'squee', 'squeg', 'squib', 'squid', 'squit', 'squiz', 'srsly', 'stabs', 'stack', 'stade', 'staff', 'stage', 'stags', 'stagy', 'staid', 'staig', 'stain', 'stair', 'stake', 'stale', 'stalk', 'stall', 'stamp', 'stand', 'stane', 'stang', 'stank', 'stans', 'staph', 'staps', 'stare', 'stark', 'starn', 'starr', 'stars', 'start', 'stary', 'stash', 'state', 'stats', 'statu', 'staun', 'stave', 'staws', 'stays', 'stead', 'steak', 'steal', 'steam', 'stean', 'stear', 'stedd', 'stede', 'steds', 'steed', 'steek', 'steel', 'steem', 'steen', 'steep', 'steer', 'steez', 'steik', 'steil', 'stein', 'stela', 'stele', 'stell', 'steme', 'stems', 'stend', 'steno', 'stens', 'stent', 'steps', 'stept', 'stere', 'stern', 'stets', 'stews', 'stewy', 'steys', 'stich', 'stick', 'stied', 'sties', 'stiff', 'stilb', 'stile', 'still', 'stilt', 'stime', 'stims', 'stimy', 'sting', 'stink', 'stint', 'stipa', 'stipe', 'stire', 'stirk', 'stirp', 'stirs', 'stive', 'stivy', 'stoae', 'stoai', 'stoas', 'stoat', 'stobs', 'stock', 'stoep', 'stogs', 'stogy', 'stoic', 'stoit', 'stoke', 'stole', 'stoln', 'stoma', 'stomp', 'stond', 'stone', 'stong', 'stonk', 'stonn', 'stony', 'stood', 'stook', 'stool', 'stoop', 'stoor', 'stope', 'stops', 'stopt', 'store', 'stork', 'storm', 'story', 'stoss', 'stots', 'stott', 'stoun', 'stoup', 'stour', 'stout', 'stove', 'stown', 'stowp', 'stows', 'strad', 'strae', 'strag', 'strak', 'strap', 'straw', 'stray', 'strep', 'strew', 'stria', 'strig', 'strim', 'strip', 'strop', 'strow', 'stroy', 'strum', 'strut', 'stubs', 'stuck', 'stucs', 'stude', 'studs', 'study', 'stuff', 'stull', 'stulm', 'stumm', 'stump', 'stums', 'stung', 'stunk', 'stuns', 'stunt', 'stupa', 'stupe', 'sture', 'sturt', 'stush', 'styed', 'styes', 'style', 'styli', 'stylo', 'styme', 'stymy', 'styre', 'styte', 'suave', 'subah', 'subak', 'subas', 'subby', 'suber', 'subha', 'succi', 'sucks', 'sucky', 'sucre', 'sudan', 'sudds', 'sudor', 'sudsy', 'suede', 'suent', 'suers', 'suete', 'suets', 'suety', 'sugan', 'sugar', 'sughs', 'sugos', 'suhur', 'suids', 'suing', 'suint', 'suite', 'suits', 'sujee', 'sukhs', 'sukis', 'sukuk', 'sulci', 'sulfa', 'sulfo', 'sulks', 'sulky', 'sulls', 'sully', 'sulph', 'sulus', 'sumac', 'sumis', 'summa', 'sumos', 'sumph', 'sumps', 'sunis', 'sunks', 'sunna', 'sunns', 'sunny', 'sunts', 'sunup', 'suona', 'suped', 'super', 'supes', 'supra', 'surah', 'sural', 'suras', 'surat', 'surds', 'sured', 'surer', 'sures', 'surfs', 'surfy', 'surge', 'surgy', 'surly', 'surra', 'sused', 'suses', 'sushi', 'susus', 'sutor', 'sutra', 'sutta', 'swabs', 'swack', 'swads', 'swage', 'swags', 'swail', 'swain', 'swale', 'swaly', 'swami', 'swamp', 'swamy', 'swang', 'swank', 'swans', 'swaps', 'swapt', 'sward', 'sware', 'swarf', 'swarm', 'swart', 'swash', 'swath', 'swats', 'swayl', 'sways', 'sweal', 'swear', 'sweat', 'swede', 'sweed', 'sweel', 'sweep', 'sweer', 'swees', 'sweet', 'sweir', 'swell', 'swelt', 'swept', 'swerf', 'sweys', 'swies', 'swift', 'swigs', 'swile', 'swill', 'swims', 'swine', 'swing', 'swink', 'swipe', 'swire', 'swirl', 'swish', 'swiss', 'swith', 'swits', 'swive', 'swizz', 'swobs', 'swole', 'swoll', 'swoln', 'swoon', 'swoop', 'swops', 'swopt', 'sword', 'swore', 'sworn', 'swots', 'swoun', 'swung', 'sybbe', 'sybil', 'syboe', 'sybow', 'sycee', 'syces', 'sycon', 'syeds', 'syens', 'syker', 'sykes', 'sylis', 'sylph', 'sylva', 'symar', 'synch', 'syncs', 'synds', 'syned', 'synes', 'synod', 'synth', 'syped', 'sypes', 'syphs', 'syrah', 'syren', 'syrup', 'sysop', 'sythe', 'syver', 'taals', 'taata', 'tabac', 'tabby', 'taber', 'tabes', 'tabid', 'tabis', 'tabla', 'table', 'tabls', 'taboo', 'tabor', 'tabos', 'tabun', 'tabus', 'tacan', 'taces', 'tacet', 'tache', 'tachi', 'tacho', 'tachs', 'tacit', 'tacks', 'tacky', 'tacos', 'tacts', 'tadah', 'taels', 'taffy', 'tafia', 'taggy', 'tagma', 'tagua', 'tahas', 'tahrs', 'taiga', 'taigs', 'taiko', 'tails', 'tains', 'taint', 'taira', 'taish', 'taits', 'tajes', 'takas', 'taken', 'taker', 'takes', 'takhi', 'takht', 'takin', 'takis', 'takky', 'talak', 'talaq', 'talar', 'talas', 'talcs', 'talcy', 'talea', 'taler', 'tales', 'talik', 'talks', 'talky', 'talls', 'tally', 'talma', 'talon', 'talpa', 'taluk', 'talus', 'tamal', 'tamas', 'tamed', 'tamer', 'tames', 'tamin', 'tamis', 'tammy', 'tamps', 'tanas', 'tanga', 'tangi', 'tango', 'tangs', 'tangy', 'tanhs', 'tania', 'tanka', 'tanks', 'tanky', 'tanna', 'tansu', 'tansy', 'tante', 'tanti', 'tanto', 'tanty', 'tapas', 'taped', 'tapen', 'taper', 'tapes', 'tapet', 'tapir', 'tapis', 'tappa', 'tapus', 'taras', 'tardo', 'tards', 'tardy', 'tared', 'tares', 'targa', 'targe', 'tarka', 'tarns', 'taroc', 'tarok', 'taros', 'tarot', 'tarps', 'tarre', 'tarry', 'tarse', 'tarsi', 'tarte', 'tarts', 'tarty', 'tarzy', 'tasar', 'tasca', 'tased', 'taser', 'tases', 'tasks', 'tassa', 'tasse', 'tasso', 'taste', 'tasto', 'tasty', 'tatar', 'tater', 'tates', 'taths', 'tatie', 'tatou', 'tatts', 'tatty', 'tatus', 'taube', 'tauld', 'taunt', 'tauon', 'taupe', 'tauts', 'tauty', 'tavah', 'tavas', 'taver', 'tawaf', 'tawai', 'tawas', 'tawed', 'tawer', 'tawie', 'tawny', 'tawse', 'tawts', 'taxed', 'taxer', 'taxes', 'taxis', 'taxol', 'taxon', 'taxor', 'taxus', 'tayra', 'tazza', 'tazze', 'teach', 'teade', 'teads', 'teaed', 'teaks', 'teals', 'teams', 'tears', 'teary', 'tease', 'teats', 'teaze', 'techs', 'techy', 'tecta', 'tecum', 'teddy', 'teels', 'teems', 'teend', 'teene', 'teens', 'teeny', 'teers', 'teeth', 'teets', 'teffs', 'teggs', 'tegua', 'tegus', 'tehee', 'tehrs', 'teiid', 'teils', 'teind', 'teins', 'tekke', 'telae', 'telco', 'teles', 'telex', 'telia', 'telic', 'tells', 'telly', 'teloi', 'telos', 'temed', 'temes', 'tempi', 'tempo', 'temps', 'tempt', 'temse', 'tench', 'tends', 'tendu', 'tenes', 'tenet', 'tenge', 'tenia', 'tenne', 'tenno', 'tenny', 'tenon', 'tenor', 'tense', 'tenth', 'tents', 'tenty', 'tenue', 'tepal', 'tepas', 'tepee', 'tepid', 'tepoy', 'terai', 'teras', 'terce', 'terek', 'teres', 'terfe', 'terfs', 'terga', 'terms', 'terne', 'terns', 'terra', 'terre', 'terry', 'terse', 'terts', 'terza', 'tesla', 'testa', 'teste', 'tests', 'testy', 'tetes', 'teths', 'tetra', 'tetri', 'teuch', 'teugh', 'tewed', 'tewel', 'tewit', 'texas', 'texes', 'texta', 'texts', 'thack', 'thagi', 'thaim', 'thale', 'thali', 'thana', 'thane', 'thang', 'thank', 'thans', 'thanx', 'tharm', 'thars', 'thaws', 'thawt', 'thawy', 'thebe', 'theca', 'theed', 'theek', 'thees', 'theft', 'thegn', 'theic', 'thein', 'their', 'thelf', 'thema', 'theme', 'thens', 'theor', 'theow', 'there', 'therm', 'these', 'thesp', 'theta', 'thete', 'thews', 'thewy', 'thick', 'thief', 'thigh', 'thigs', 'thilk', 'thill', 'thine', 'thing', 'think', 'thins', 'thiol', 'third', 'thirl', 'thoft', 'thole', 'tholi', 'thong', 'thorn', 'thoro', 'thorp', 'those', 'thots', 'thous', 'thowl', 'thrae', 'thraw', 'three', 'threw', 'thrid', 'thrip', 'throb', 'throe', 'throw', 'thrum', 'thuds', 'thugs', 'thuja', 'thumb', 'thump', 'thunk', 'thurl', 'thuya', 'thyme', 'thymi', 'thymy', 'tians', 'tiara', 'tiare', 'tiars', 'tibia', 'tical', 'ticca', 'ticed', 'tices', 'tichy', 'ticks', 'ticky', 'tidal', 'tiddy', 'tided', 'tides', 'tiefs', 'tiers', 'tiffs', 'tifos', 'tifts', 'tiger', 'tiges', 'tight', 'tigon', 'tikas', 'tikes', 'tikia', 'tikis', 'tikka', 'tilak', 'tilde', 'tiled', 'tiler', 'tiles', 'tills', 'tilly', 'tilth', 'tilts', 'timbo', 'timed', 'timer', 'times', 'timid', 'timon', 'timps', 'tinas', 'tinct', 'tinds', 'tinea', 'tined', 'tines', 'tinge', 'tings', 'tinks', 'tinny', 'tinto', 'tints', 'tinty', 'tipis', 'tippy', 'tipsy', 'tipup', 'tired', 'tires', 'tirls', 'tiros', 'tirrs', 'tirth', 'titan', 'titar', 'titas', 'titch', 'titer', 'tithe', 'tithi', 'titin', 'titir', 'titis', 'title', 'titre', 'titty', 'titup', 'tiyin', 'tiyns', 'tizes', 'tizzy', 'toads', 'toady', 'toast', 'toaze', 'tocks', 'tocky', 'tocos', 'today', 'todde', 'toddy', 'todea', 'todos', 'toeas', 'toffs', 'toffy', 'tofts', 'tofus', 'togae', 'togas', 'toged', 'toges', 'togue', 'tohos', 'toidy', 'toile', 'toils', 'toing', 'toise', 'toits', 'toity', 'tokay', 'toked', 'token', 'toker', 'tokes', 'tokos', 'tolan', 'tolar', 'tolas', 'toled', 'toles', 'tolls', 'tolly', 'tolts', 'tolus', 'tolyl', 'toman', 'tombo', 'tombs', 'tomen', 'tomes', 'tomia', 'tomin', 'tomme', 'tommy', 'tomos', 'tomoz', 'tonal', 'tondi', 'tondo', 'toned', 'toner', 'tones', 'toney', 'tonga', 'tongs', 'tonic', 'tonka', 'tonks', 'tonne', 'tonus', 'tools', 'tooms', 'toons', 'tooth', 'toots', 'topaz', 'toped', 'topee', 'topek', 'toper', 'topes', 'tophe', 'tophi', 'tophs', 'topic', 'topis', 'topoi', 'topos', 'toppy', 'toque', 'torah', 'toran', 'toras', 'torch', 'torcs', 'tores', 'toric', 'torii', 'toros', 'torot', 'torrs', 'torse', 'torsi', 'torsk', 'torso', 'torta', 'torte', 'torts', 'torus', 'tosas', 'tosed', 'toses', 'toshy', 'tossy', 'tosyl', 'total', 'toted', 'totem', 'toter', 'totes', 'totty', 'touch', 'tough', 'touks', 'touns', 'tours', 'touse', 'tousy', 'touts', 'touze', 'touzy', 'towai', 'towed', 'towel', 'tower', 'towie', 'towno', 'towns', 'towny', 'towse', 'towsy', 'towts', 'towze', 'towzy', 'toxic', 'toxin', 'toyed', 'toyer', 'toyon', 'toyos', 'tozed', 'tozes', 'tozie', 'trabs', 'trace', 'track', 'tract', 'trade', 'trads', 'trady', 'traga', 'tragi', 'trags', 'tragu', 'traik', 'trail', 'train', 'trait', 'tramp', 'trams', 'trank', 'tranq', 'trans', 'trant', 'trape', 'trapo', 'traps', 'trapt', 'trash', 'trass', 'trats', 'tratt', 'trave', 'trawl', 'trayf', 'trays', 'tread', 'treat', 'treck', 'treed', 'treen', 'trees', 'trefa', 'treif', 'treks', 'trema', 'trems', 'trend', 'tress', 'trest', 'trets', 'trews', 'treyf', 'treys', 'triac', 'triad', 'trial', 'tribe', 'trice', 'trick', 'tride', 'tried', 'trier', 'tries', 'trifa', 'triff', 'trigo', 'trigs', 'trike', 'trild', 'trill', 'trims', 'trine', 'trins', 'triol', 'trior', 'trios', 'tripe', 'trips', 'tripy', 'trist', 'trite', 'troad', 'troak', 'troat', 'trock', 'trode', 'trods', 'trogs', 'trois', 'troke', 'troll', 'tromp', 'trona', 'tronc', 'trone', 'tronk', 'trons', 'troop', 'trooz', 'trope', 'tropo', 'troth', 'trots', 'trout', 'trove', 'trows', 'troys', 'truce', 'truck', 'trued', 'truer', 'trues', 'trugo', 'trugs', 'trull', 'truly', 'trump', 'trunk', 'truss', 'trust', 'truth', 'tryer', 'tryke', 'tryma', 'tryps', 'tryst', 'tsade', 'tsadi', 'tsars', 'tsked', 'tsuba', 'tsubo', 'tuans', 'tuart', 'tuath', 'tubae', 'tubal', 'tubar', 'tubas', 'tubby', 'tubed', 'tuber', 'tubes', 'tucks', 'tufas', 'tuffe', 'tuffs', 'tufts', 'tufty', 'tugra', 'tuile', 'tuina', 'tuism', 'tuktu', 'tules', 'tulip', 'tulle', 'tulpa', 'tulps', 'tulsi', 'tumid', 'tummy', 'tumor', 'tumps', 'tumpy', 'tunas', 'tunds', 'tuned', 'tuner', 'tunes', 'tungs', 'tunic', 'tunny', 'tupek', 'tupik', 'tuple', 'tuque', 'turbo', 'turds', 'turfs', 'turfy', 'turks', 'turme', 'turms', 'turns', 'turnt', 'turon', 'turps', 'turrs', 'tushy', 'tusks', 'tusky', 'tutee', 'tutes', 'tutor', 'tutti', 'tutty', 'tutus', 'tuxes', 'tuyer', 'twaes', 'twain', 'twals', 'twang', 'twank', 'twats', 'tways', 'tweak', 'tweed', 'tweel', 'tween', 'tweep', 'tweer', 'tweet', 'twerk', 'twerp', 'twice', 'twier', 'twigs', 'twill', 'twilt', 'twine', 'twink', 'twins', 'twiny', 'twire', 'twirk', 'twirl', 'twirp', 'twist', 'twite', 'twits', 'twixt', 'twocs', 'twoer', 'twonk', 'twyer', 'tyees', 'tyers', 'tying', 'tyiyn', 'tykes', 'tyler', 'tymps', 'tynde', 'tyned', 'tynes', 'typal', 'typed', 'types', 'typey', 'typic', 'typos', 'typps', 'typto', 'tyran', 'tyred', 'tyres', 'tyros', 'tythe', 'tzars', 'ubacs', 'ubity', 'udals', 'udder', 'udons', 'udyog', 'ugali', 'ugged', 'uhlan', 'uhuru', 'ukase', 'ulama', 'ulans', 'ulcer', 'ulema', 'ulmin', 'ulmos', 'ulnad', 'ulnae', 'ulnar', 'ulnas', 'ulpan', 'ultra', 'ulvas', 'ulyie', 'ulzie', 'umami', 'umbel', 'umber', 'umble', 'umbos', 'umbra', 'umbre', 'umiac', 'umiak', 'umiaq', 'ummah', 'ummas', 'ummed', 'umped', 'umphs', 'umpie', 'umpty', 'umrah', 'umras', 'unagi', 'unais', 'unapt', 'unarm', 'unary', 'unaus', 'unbag', 'unban', 'unbar', 'unbed', 'unbid', 'unbox', 'uncap', 'unces', 'uncia', 'uncle', 'uncos', 'uncoy', 'uncus', 'uncut', 'undam', 'undee', 'under', 'undid', 'undos', 'undue', 'undug', 'uneth', 'unfed', 'unfit', 'unfix', 'ungag', 'unget', 'ungod', 'ungot', 'ungum', 'unhat', 'unhip', 'unica', 'unify', 'union', 'unios', 'unite', 'units', 'unity', 'unjam', 'unked', 'unket', 'unkey', 'unkid', 'unkut', 'unlap', 'unlaw', 'unlay', 'unled', 'unleg', 'unlet', 'unlid', 'unlit', 'unmad', 'unman', 'unmet', 'unmew', 'unmix', 'unode', 'unold', 'unown', 'unpay', 'unpeg', 'unpen', 'unpin', 'unply', 'unpot', 'unput', 'unred', 'unrid', 'unrig', 'unrip', 'unsaw', 'unsay', 'unsee', 'unset', 'unsew', 'unsex', 'unsod', 'unsub', 'untag', 'untax', 'untie', 'until', 'untin', 'unwed', 'unwet', 'unwit', 'unwon', 'unzip', 'upbow', 'upbye', 'updos', 'updry', 'upend', 'upful', 'upjet', 'uplay', 'upled', 'uplit', 'upped', 'upper', 'upran', 'uprun', 'upsee', 'upset', 'upsey', 'uptak', 'upter', 'uptie', 'uraei', 'urali', 'uraos', 'urare', 'urari', 'urase', 'urate', 'urban', 'urbex', 'urbia', 'urdee', 'ureal', 'ureas', 'uredo', 'ureic', 'ureid', 'urena', 'urent', 'urged', 'urger', 'urges', 'urial', 'urine', 'urite', 'urman', 'urnal', 'urned', 'urped', 'ursae', 'ursid', 'urson', 'urubu', 'urupa', 'urvas', 'usage', 'usens', 'users', 'useta', 'usher', 'using', 'usnea', 'usnic', 'usque', 'ustad', 'uster', 'usual', 'usure', 'usurp', 'usury', 'uteri', 'utero', 'utile', 'utter', 'uveal', 'uveas', 'uvula', 'vacas', 'vacay', 'vacua', 'vacui', 'vacuo', 'vadas', 'vaded', 'vades', 'vadge', 'vagal', 'vague', 'vagus', 'vaids', 'vails', 'vaire', 'vairs', 'vairy', 'vajra', 'vakas', 'vakil', 'vales', 'valet', 'valid', 'valis', 'valli', 'valor', 'valse', 'value', 'valve', 'vamps', 'vampy', 'vanda', 'vaned', 'vanes', 'vanga', 'vangs', 'vants', 'vaped', 'vaper', 'vapes', 'vapid', 'vapor', 'varan', 'varas', 'varda', 'vardo', 'vardy', 'varec', 'vares', 'varia', 'varix', 'varna', 'varus', 'varve', 'vasal', 'vases', 'vasts', 'vasty', 'vatas', 'vatha', 'vatic', 'vatje', 'vatos', 'vatus', 'vauch', 'vault', 'vaunt', 'vaute', 'vauts', 'vawte', 'vaxes', 'veale', 'veals', 'vealy', 'veena', 'veeps', 'veers', 'veery', 'vegan', 'vegas', 'veges', 'veggo', 'vegie', 'vegos', 'vehme', 'veils', 'veily', 'veins', 'veiny', 'velar', 'velds', 'veldt', 'veles', 'vells', 'velum', 'venae', 'venal', 'venas', 'vends', 'vendu', 'veney', 'venge', 'venin', 'venom', 'venti', 'vents', 'venue', 'venus', 'verba', 'verbs', 'verde', 'verge', 'verra', 'verre', 'verry', 'versa', 'verse', 'verso', 'verst', 'verte', 'verts', 'vertu', 'verve', 'vespa', 'vesta', 'vests', 'vetch', 'veuve', 'veves', 'vexed', 'vexer', 'vexes', 'vexil', 'vezir', 'vials', 'viand', 'vibed', 'vibes', 'vibex', 'vibey', 'vicar', 'viced', 'vices', 'vichy', 'vicus', 'video', 'viers', 'vieux', 'views', 'viewy', 'vifda', 'viffs', 'vigas', 'vigia', 'vigil', 'vigor', 'vilde', 'viler', 'villa', 'ville', 'villi', 'vills', 'vimen', 'vinal', 'vinas', 'vinca', 'vined', 'viner', 'vines', 'vinew', 'vinho', 'vinic', 'vinny', 'vinos', 'vints', 'vinyl', 'viola', 'viold', 'viols', 'viper', 'viral', 'vired', 'vireo', 'vires', 'virga', 'virge', 'virgo', 'virid', 'virls', 'virtu', 'virus', 'visas', 'vised', 'vises', 'visie', 'visit', 'visna', 'visne', 'vison', 'visor', 'vista', 'visto', 'vitae', 'vital', 'vitas', 'vitex', 'vitro', 'vitta', 'vivas', 'vivat', 'vivda', 'viver', 'vives', 'vivid', 'vivos', 'vivre', 'vixen', 'vizir', 'vizor', 'vlast', 'vleis', 'vlies', 'vlogs', 'voars', 'vobla', 'vocab', 'vocal', 'voces', 'voddy', 'vodka', 'vodou', 'vodun', 'voema', 'vogie', 'vogue', 'voice', 'voici', 'voids', 'voila', 'voile', 'voips', 'volae', 'volar', 'voled', 'voles', 'volet', 'volke', 'volks', 'volta', 'volte', 'volti', 'volts', 'volva', 'volve', 'vomer', 'vomit', 'voted', 'voter', 'votes', 'vouch', 'vouge', 'voulu', 'vowed', 'vowel', 'vower', 'voxel', 'voxes', 'vozhd', 'vraic', 'vrils', 'vroom', 'vrous', 'vrouw', 'vrows', 'vuggs', 'vuggy', 'vughs', 'vughy', 'vulgo', 'vulns', 'vulva', 'vutty', 'vygie', 'vying', 'waacs', 'wacke', 'wacko', 'wacks', 'wacky', 'wadas', 'wadds', 'waddy', 'waded', 'wader', 'wades', 'wadge', 'wadis', 'wadts', 'wafer', 'waffs', 'wafts', 'waged', 'wager', 'wages', 'wagga', 'wagon', 'wagyu', 'wahay', 'wahey', 'wahoo', 'waide', 'waifs', 'waift', 'wails', 'wains', 'wairs', 'waist', 'waite', 'waits', 'waive', 'wakas', 'waked', 'waken', 'waker', 'wakes', 'wakfs', 'waldo', 'walds', 'waled', 'waler', 'wales', 'walie', 'walis', 'walks', 'walla', 'walls', 'wally', 'walty', 'waltz', 'wamed', 'wames', 'wamus', 'wands', 'waned', 'wanes', 'waney', 'wangs', 'wanks', 'wanky', 'wanle', 'wanly', 'wanna', 'wanta', 'wants', 'wanty', 'wanze', 'waqfs', 'warbs', 'warby', 'wards', 'wared', 'wares', 'warez', 'warks', 'warms', 'warns', 'warps', 'warre', 'warst', 'warts', 'warty', 'wases', 'washi', 'washy', 'wasms', 'wasps', 'waspy', 'waste', 'wasts', 'watap', 'watch', 'water', 'watts', 'wauff', 'waugh', 'wauks', 'waulk', 'wauls', 'waurs', 'waved', 'waver', 'waves', 'wavey', 'wawas', 'wawes', 'wawls', 'waxed', 'waxen', 'waxer', 'waxes', 'wayed', 'wazir', 'wazoo', 'weald', 'weals', 'weamb', 'weans', 'wears', 'weary', 'weave', 'webby', 'weber', 'wecht', 'wedel', 'wedge', 'wedgy', 'weeds', 'weedy', 'weeis', 'weeke', 'weeks', 'weels', 'weems', 'weens', 'weeny', 'weeps', 'weepy', 'weest', 'weete', 'weets', 'wefte', 'wefts', 'weids', 'weigh', 'weils', 'weird', 'weirs', 'weise', 'weize', 'wekas', 'welch', 'welds', 'welke', 'welks', 'welkt', 'wells', 'welly', 'welsh', 'welts', 'wembs', 'wench', 'wends', 'wenge', 'wenny', 'wents', 'werfs', 'weros', 'wersh', 'wests', 'wetas', 'wetly', 'wexed', 'wexes', 'whack', 'whale', 'whamo', 'whams', 'whang', 'whaps', 'whare', 'wharf', 'whata', 'whats', 'whaup', 'whaur', 'wheal', 'whear', 'wheat', 'wheek', 'wheel', 'wheen', 'wheep', 'wheft', 'whelk', 'whelm', 'whelp', 'whens', 'where', 'whets', 'whews', 'wheys', 'which', 'whids', 'whies', 'whiff', 'whift', 'whigs', 'while', 'whilk', 'whims', 'whine', 'whins', 'whiny', 'whios', 'whips', 'whipt', 'whirl', 'whirr', 'whirs', 'whish', 'whisk', 'whiss', 'whist', 'white', 'whits', 'whity', 'whizz', 'whole', 'whomp', 'whoof', 'whoop', 'whoot', 'whops', 'whore', 'whorl', 'whort', 'whose', 'whoso', 'whows', 'whump', 'whups', 'whyda', 'wicca', 'wicks', 'wicky', 'widdy', 'widen', 'wider', 'wides', 'widow', 'width', 'wield', 'wiels', 'wifed', 'wifes', 'wifey', 'wifie', 'wifts', 'wifty', 'wigan', 'wigga', 'wiggy', 'wight', 'wikis', 'wilco', 'wilds', 'wiled', 'wiles', 'wilga', 'wilis', 'wilja', 'wills', 'willy', 'wilts', 'wimps', 'wimpy', 'wince', 'winch', 'winds', 'windy', 'wined', 'wines', 'winey', 'winge', 'wings', 'wingy', 'winks', 'winky', 'winna', 'winns', 'winos', 'winze', 'wiped', 'wiper', 'wipes', 'wired', 'wirer', 'wires', 'wirra', 'wirri', 'wised', 'wiser', 'wises', 'wisha', 'wisht', 'wisps', 'wispy', 'wists', 'witan', 'witch', 'wited', 'wites', 'withe', 'withs', 'withy', 'witty', 'wived', 'wiver', 'wives', 'wizen', 'wizes', 'wizzo', 'woads', 'woady', 'woald', 'wocks', 'wodge', 'wodgy', 'woful', 'wojus', 'woken', 'woker', 'wokka', 'wolds', 'wolfs', 'wolly', 'wolve', 'woman', 'womas', 'wombs', 'womby', 'women', 'womyn', 'wonga', 'wongi', 'wonks', 'wonky', 'wonts', 'woods', 'woody', 'wooed', 'wooer', 'woofs', 'woofy', 'woold', 'wools', 'wooly', 'woons', 'woops', 'woopy', 'woose', 'woosh', 'wootz', 'woozy', 'words', 'wordy', 'works', 'worky', 'world', 'worms', 'wormy', 'worry', 'worse', 'worst', 'worth', 'worts', 'would', 'wound', 'woven', 'wowed', 'wowee', 'wowse', 'woxen', 'wrack', 'wrang', 'wraps', 'wrapt', 'wrast', 'wrate', 'wrath', 'wrawl', 'wreak', 'wreck', 'wrens', 'wrest', 'wrick', 'wried', 'wrier', 'wries', 'wring', 'wrist', 'write', 'writs', 'wroke', 'wrong', 'wroot', 'wrote', 'wroth', 'wrung', 'wryer', 'wryly', 'wuddy', 'wudus', 'wuffs', 'wulls', 'wunga', 'wurst', 'wuses', 'wushu', 'wussy', 'wuxia', 'wyled', 'wyles', 'wynds', 'wynns', 'wyted', 'wytes', 'wythe', 'xebec', 'xenia', 'xenic', 'xenon', 'xeric', 'xerox', 'xerus', 'xoana', 'xolos', 'xrays', 'xviii', 'xylan', 'xylem', 'xylic', 'xylol', 'xylyl', 'xysti', 'xysts', 'yaars', 'yaass', 'yabas', 'yabba', 'yabby', 'yacca', 'yacht', 'yacka', 'yacks', 'yadda', 'yaffs', 'yager', 'yages', 'yagis', 'yagna', 'yahoo', 'yaird', 'yajna', 'yakka', 'yakow', 'yales', 'yamen', 'yampa', 'yampy', 'yamun', 'yandy', 'yangs', 'yanks', 'yapok', 'yapon', 'yapps', 'yappy', 'yarak', 'yarco', 'yards', 'yarer', 'yarfa', 'yarks', 'yarns', 'yarra', 'yarrs', 'yarta', 'yarto', 'yates', 'yatra', 'yauds', 'yauld', 'yaups', 'yawed', 'yawey', 'yawls', 'yawns', 'yawny', 'yawps', 'yayas', 'ybore', 'yclad', 'ycled', 'ycond', 'ydrad', 'ydred', 'yeads', 'yeahs', 'yealm', 'yeans', 'yeard', 'yearn', 'years', 'yeast', 'yecch', 'yechs', 'yechy', 'yedes', 'yeeds', 'yeeek', 'yeesh', 'yeggs', 'yelks', 'yells', 'yelms', 'yelps', 'yelts', 'yenta', 'yente', 'yerba', 'yerds', 'yerks', 'yeses', 'yesks', 'yests', 'yesty', 'yetis', 'yetts', 'yeuch', 'yeuks', 'yeuky', 'yeven', 'yeves', 'yewen', 'yexed', 'yexes', 'yfere', 'yield', 'yiked', 'yikes', 'yills', 'yince', 'yipes', 'yippy', 'yirds', 'yirks', 'yirrs', 'yirth', 'yites', 'yitie', 'ylems', 'ylide', 'ylids', 'ylike', 'ylkes', 'ymolt', 'ympes', 'yobbo', 'yobby', 'yocks', 'yodel', 'yodhs', 'yodle', 'yogas', 'yogee', 'yoghs', 'yogic', 'yogin', 'yogis', 'yohah', 'yohay', 'yoick', 'yojan', 'yokan', 'yoked', 'yokeg', 'yokel', 'yoker', 'yokes', 'yokul', 'yolks', 'yolky', 'yolps', 'yomim', 'yomps', 'yonic', 'yonis', 'yonks', 'yonny', 'yoofs', 'yoops', 'yopos', 'yoppo', 'yores', 'yorga', 'yorks', 'yorps', 'youks', 'young', 'yourn', 'yours', 'yourt', 'youse', 'youth', 'yowed', 'yowes', 'yowie', 'yowls', 'yowsa', 'yowza', 'yoyos', 'yrapt', 'yrent', 'yrivd', 'yrneh', 'ysame', 'ytost', 'yuans', 'yucas', 'yucca', 'yucch', 'yucko', 'yucks', 'yucky', 'yufts', 'yugas', 'yuked', 'yukes', 'yukky', 'yukos', 'yulan', 'yules', 'yummo', 'yummy', 'yumps', 'yupon', 'yuppy', 'yurta', 'yurts', 'yuzus', 'zabra', 'zacks', 'zaida', 'zaide', 'zaidy', 'zaire', 'zakat', 'zamac', 'zamak', 'zaman', 'zambo', 'zamia', 'zamis', 'zanja', 'zante', 'zanza', 'zanze', 'zappy', 'zarda', 'zarfs', 'zaris', 'zatis', 'zawns', 'zaxes', 'zayde', 'zayin', 'zazen', 'zeals', 'zebec', 'zebra', 'zebub', 'zebus', 'zedas', 'zeera', 'zeins', 'zendo', 'zerda', 'zerks', 'zeros', 'zests', 'zesty', 'zetas', 'zexes', 'zezes', 'zhomo', 'zhush', 'zhuzh', 'zibet', 'ziffs', 'zigan', 'zikrs', 'zilas', 'zilch', 'zilla', 'zills', 'zimbi', 'zimbs', 'zinco', 'zincs', 'zincy', 'zineb', 'zines', 'zings', 'zingy', 'zinke', 'zinky', 'zinos', 'zippo', 'zippy', 'ziram', 'zitis', 'zitty', 'zizel', 'zizit', 'zlote', 'zloty', 'zoaea', 'zobos', 'zobus', 'zocco', 'zoeae', 'zoeal', 'zoeas', 'zoism', 'zoist', 'zokor', 'zolle', 'zombi', 'zonae', 'zonal', 'zonda', 'zoned', 'zoner', 'zones', 'zonks', 'zooea', 'zooey', 'zooid', 'zooks', 'zooms', 'zoomy', 'zoons', 'zooty', 'zoppa', 'zoppo', 'zoril', 'zoris', 'zorro', 'zorse', 'zouks', 'zowee', 'zowie', 'zulus', 'zupan', 'zupas', 'zuppa', 'zurfs', 'zuzim', 'zygal', 'zygon', 'zymes', 'zymic'
]

print(f'{len(guess_words_long) = }')


len(guess_words_long) = 14855


In [ ]:
### Wordle word list

wordle_word_list = [
    'aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort', 'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage', 'adapt', 'adept', 'admin', 'admit', 'adobe', 'adopt', 'adore', 'adorn', 'adult', 'affix', 'afire', 'afoot', 'afoul', 'after', 'again', 'agape', 'agate', 'agent', 'agile', 'aging', 'aglow', 'agony', 'agora', 'agree', 'ahead', 'aider', 'aisle', 'alarm', 'album', 'alert', 'algae', 'alibi', 'alien', 'align', 'alike', 'alive', 'allay', 'alley', 'allot', 'allow', 'alloy', 'aloft', 'alone', 'along', 'aloof', 'aloud', 'alpha', 'altar', 'alter', 'amass', 'amaze', 'amber', 'amble', 'amend', 'amiss', 'amity', 'among', 'ample', 'amply', 'amuse', 'angel', 'anger', 'angle', 'angry', 'angst', 'anime', 'ankle', 'annex', 'annoy', 'annul', 'anode', 'antic', 'anvil', 'aorta', 'apart', 'aphid', 'aping', 'apnea', 'apple', 'apply', 'apron', 'aptly', 'arbor', 'ardor', 'arena', 'argue', 'arise', 'armor', 'aroma', 'arose', 'array', 'arrow', 'arson', 'artsy', 'ascot', 'ashen', 'aside', 'askew', 'assay', 'asset', 'atoll', 'atone', 'attic', 'audio', 'audit', 'augur', 'aunty', 'avail', 'avert', 'avian', 'avoid', 'await', 'awake', 'award', 'aware', 'awash', 'awful', 'awoke', 'axial', 'axiom', 'axion', 'azure', 'bacon', 'badge', 'badly', 'bagel', 'baggy', 'baker', 'baler', 'balmy', 'banal', 'banjo', 'barge', 'baron', 'basal', 'basic', 'basil', 'basin', 'basis', 'baste', 'batch', 'bathe', 'baton', 'batty', 'bawdy', 'bayou', 'beach', 'beady', 'beard', 'beast', 'beech', 'beefy', 'befit', 'began', 'begat', 'beget', 'begin', 'begun', 'being', 'belch', 'belie', 'belle', 'belly', 'below', 'bench', 'beret', 'berry', 'berth', 'beset', 'betel', 'bevel', 'bezel', 'bible', 'bicep', 'biddy', 'bigot', 'bilge', 'billy', 'binge', 'bingo', 'biome', 'birch', 'birth', 'bison', 'bitty', 'black', 'blade', 'blame', 'bland', 'blank', 'blare', 'blast', 'blaze', 'bleak', 'bleat', 'bleed', 'bleep', 'blend', 'bless', 'blimp', 'blind', 'blink', 'bliss', 'blitz', 'bloat', 'block', 'bloke', 'blond', 'blood', 'bloom', 'blown', 'bluer', 'bluff', 'blunt', 'blurb', 'blurt', 'blush', 'board', 'boast', 'bobby', 'boney', 'bongo', 'bonus', 'booby', 'boost', 'booth', 'booty', 'booze', 'boozy', 'borax', 'borne', 'bosom', 'bossy', 'botch', 'bough', 'boule', 'bound', 'bowel', 'boxer', 'brace', 'braid', 'brain', 'brake', 'brand', 'brash', 'brass', 'brave', 'bravo', 'brawl', 'brawn', 'bread', 'break', 'breed', 'briar', 'bribe', 'brick', 'bride', 'brief', 'brine', 'bring', 'brink', 'briny', 'brisk', 'broad', 'broil', 'broke', 'brood', 'brook', 'broom', 'broth', 'brown', 'brunt', 'brush', 'brute', 'buddy', 'budge', 'buggy', 'bugle', 'build', 'built', 'bulge', 'bulky', 'bully', 'bunch', 'bunny', 'burly', 'burnt', 'burst', 'bused', 'bushy', 'butch', 'butte', 'buxom', 'buyer', 'bylaw', 'cabal', 'cabby', 'cabin', 'cable', 'cacao', 'cache', 'cacti', 'caddy', 'cadet', 'cagey', 'cairn', 'camel', 'cameo', 'canal', 'candy', 'canny', 'canoe', 'canon', 'caper', 'caput', 'carat', 'cargo', 'carol', 'carry', 'carve', 'caste', 'catch', 'cater', 'catty', 'caulk', 'cause', 'cavil', 'cease', 'cedar', 'cello', 'chafe', 'chaff', 'chain', 'chair', 'chalk', 'champ', 'chant', 'chaos', 'chard', 'charm', 'chart', 'chase', 'chasm', 'cheap', 'cheat', 'check', 'cheek', 'cheer', 'chess', 'chest', 'chick', 'chide', 'chief', 'child', 'chili', 'chill', 'chime', 'china', 'chirp', 'chock', 'choir', 'choke', 'chord', 'chore', 'chose', 'chuck', 'chump', 'chunk', 'churn', 'chute', 'cider', 'cigar', 'cinch', 'circa', 'civic', 'civil', 'clack', 'claim', 'clamp', 'clang', 'clank', 'clash', 'clasp', 'class', 'clean', 'clear', 'cleat', 'cleft', 'clerk', 'click', 'cliff', 'climb', 'cling', 'clink', 'cloak', 'clock', 'clone', 'close', 'cloth', 'cloud', 'clout', 'clove', 'clown', 'cluck', 'clued', 'clump', 'clung', 'coach', 'coast', 'cobra', 'cocoa', 'colon', 'color', 'comet', 'comfy', 'comic', 'comma', 'conch', 'condo', 'conic', 'copse', 'coral', 'corer', 'corny', 'couch', 'cough', 'could', 'count', 'coupe', 'court', 'coven', 'cover', 'covet', 'covey', 'cower', 'coyly', 'crack', 'craft', 'cramp', 'crane', 'crank', 'crash', 'crass', 'crate', 'crave', 'crawl', 'craze', 'crazy', 'creak', 'cream', 'credo', 'creed', 'creek', 'creep', 'creme', 'crepe', 'crept', 'cress', 'crest', 'crick', 'cried', 'crier', 'crime', 'crimp', 'crisp', 'croak', 'crock', 'crone', 'crony', 'crook', 'cross', 'croup', 'crowd', 'crown', 'crude', 'cruel', 'crumb', 'crump', 'crush', 'crust', 'crypt', 'cubic', 'cumin', 'curio', 'curly', 'curry', 'curse', 'curve', 'curvy', 'cutie', 'cyber', 'cycle', 'cynic', 'daddy', 'daily', 'dairy', 'daisy', 'dally', 'dance', 'dandy', 'datum', 'daunt', 'dealt', 'death', 'debar', 'debit', 'debug', 'debut', 'decal', 'decay', 'decor', 'decoy', 'decry', 'defer', 'deign', 'deity', 'delay', 'delta', 'delve', 'demon', 'demur', 'denim', 'dense', 'depot', 'depth', 'derby', 'deter', 'detox', 'deuce', 'devil', 'diary', 'dicey', 'digit', 'dilly', 'dimly', 'diner', 'dingo', 'dingy', 'diode', 'dirge', 'dirty', 'disco', 'ditch', 'ditto', 'ditty', 'diver', 'dizzy', 'dodge', 'dodgy', 'dogma', 'doing', 'dolly', 'donor', 'donut', 'dopey', 'doubt', 'dough', 'dowdy', 'dowel', 'downy', 'dowry', 'dozen', 'draft', 'drain', 'drake', 'drama', 'drank', 'drape', 'drawl', 'drawn', 'dread', 'dream', 'dress', 'dried', 'drier', 'drift', 'drill', 'drink', 'drive', 'droit', 'droll', 'drone', 'drool', 'droop', 'dross', 'drove', 'drown', 'druid', 'drunk', 'dryer', 'dryly', 'duchy', 'dully', 'dummy', 'dumpy', 'dunce', 'dusky', 'dusty', 'dutch', 'duvet', 'dwarf', 'dwell', 'dwelt', 'dying', 'eager', 'eagle', 'early', 'earth', 'easel', 'eaten', 'eater', 'ebony', 'eclat', 'edict', 'edify', 'eerie', 'egret', 'eight', 'eject', 'eking', 'elate', 'elbow', 'elder', 'elect', 'elegy', 'elfin', 'elide', 'elite', 'elope', 'elude', 'email', 'embed', 'ember', 'emcee', 'empty', 'enact', 'endow', 'enema', 'enemy', 'enjoy', 'ennui', 'ensue', 'enter', 'entry', 'envoy', 'epoch', 'epoxy', 'equal', 'equip', 'erase', 'erect', 'erode', 'error', 'erupt', 'essay', 'ester', 'ether', 'ethic', 'ethos', 'etude', 'evade', 'event', 'every', 'evict', 'evoke', 'exact', 'exalt', 'excel', 'exert', 'exile', 'exist', 'expel', 'extol', 'extra', 'exult', 'eying', 'fable', 'facet', 'faint', 'fairy', 'faith', 'false', 'fancy', 'fanny', 'farce', 'fatal', 'fatty', 'fault', 'fauna', 'favor', 'feast', 'fecal', 'feign', 'fella', 'felon', 'femme', 'femur', 'fence', 'feral', 'ferry', 'fetal', 'fetch', 'fetid', 'fetus', 'fever', 'fewer', 'fiber', 'fibre', 'ficus', 'field', 'fiend', 'fiery', 'fifth', 'fifty', 'fight', 'filer', 'filet', 'filly', 'filmy', 'filth', 'final', 'finch', 'finer', 'first', 'fishy', 'fixer', 'fizzy', 'fjord', 'flack', 'flail', 'flair', 'flake', 'flaky', 'flame', 'flank', 'flare', 'flash', 'flask', 'fleck', 'fleet', 'flesh', 'flick', 'flier', 'fling', 'flint', 'flirt', 'float', 'flock', 'flood', 'floor', 'flora', 'floss', 'flour', 'flout', 'flown', 'fluff', 'fluid', 'fluke', 'flume', 'flung', 'flunk', 'flush', 'flute', 'flyer', 'foamy', 'focal', 'focus', 'foggy', 'foist', 'folio', 'folly', 'foray', 'force', 'forge', 'forgo', 'forte', 'forth', 'forty', 'forum', 'found', 'foyer', 'frail', 'frame', 'frank', 'fraud', 'freak', 'freed', 'freer', 'fresh', 'friar', 'fried', 'frill', 'frisk', 'fritz', 'frock', 'frond', 'front', 'frost', 'froth', 'frown', 'froze', 'fruit', 'fudge', 'fugue', 'fully', 'fungi', 'funky', 'funny', 'furor', 'furry', 'fussy', 'fuzzy', 'gaffe', 'gaily', 'gamer', 'gamma', 'gamut', 'gassy', 'gaudy', 'gauge', 'gaunt', 'gauze', 'gavel', 'gawky', 'gayer', 'gayly', 'gazer', 'gecko', 'geeky', 'geese', 'genie', 'genre', 'ghost', 'ghoul', 'giant', 'giddy', 'gipsy', 'girly', 'girth', 'given', 'giver', 'glade', 'gland', 'glare', 'glass', 'glaze', 'gleam', 'glean', 'glide', 'glint', 'gloat', 'globe', 'gloom', 'glory', 'gloss', 'glove', 'glyph', 'gnash', 'gnome', 'godly', 'going', 'golem', 'golly', 'gonad', 'goner', 'goody', 'gooey', 'goofy', 'goose', 'gorge', 'gouge', 'gourd', 'grace', 'grade', 'graft', 'grail', 'grain', 'grand', 'grant', 'grape', 'graph', 'grasp', 'grass', 'grate', 'grave', 'gravy', 'graze', 'great', 'greed', 'green', 'greet', 'grief', 'grill', 'grime', 'grimy', 'grind', 'gripe', 'groan', 'groin', 'groom', 'grope', 'gross', 'group', 'grout', 'grove', 'growl', 'grown', 'gruel', 'gruff', 'grunt', 'guard', 'guava', 'guess', 'guest', 'guide', 'guild', 'guile', 'guilt', 'guise', 'gulch', 'gully', 'gumbo', 'gummy', 'guppy', 'gusto', 'gusty', 'gypsy', 'habit', 'hairy', 'halve', 'handy', 'happy', 'hardy', 'harem', 'harpy', 'harry', 'harsh', 'haste', 'hasty', 'hatch', 'hater', 'haunt', 'haute', 'haven', 'havoc', 'hazel', 'heady', 'heard', 'heart', 'heath', 'heave', 'heavy', 'hedge', 'hefty', 'heist', 'helix', 'hello', 'hence', 'heron', 'hilly', 'hinge', 'hippo', 'hippy', 'hitch', 'hoard', 'hobby', 'hoist', 'holly', 'homer', 'honey', 'honor', 'horde', 'horny', 'horse', 'hotel', 'hotly', 'hound', 'house', 'hovel', 'hover', 'howdy', 'human', 'humid', 'humor', 'humph', 'humus', 'hunch', 'hunky', 'hurry', 'husky', 'hussy', 'hutch', 'hydro', 'hyena', 'hymen', 'hyper', 'icily', 'icing', 'ideal', 'idiom', 'idiot', 'idler', 'idyll', 'igloo', 'iliac', 'image', 'imbue', 'impel', 'imply', 'inane', 'inbox', 'incur', 'index', 'inept', 'inert', 'infer', 'ingot', 'inlay', 'inlet', 'inner', 'input', 'inter', 'intro', 'ionic', 'irate', 'irony', 'islet', 'issue', 'itchy', 'ivory', 'jaunt', 'jazzy', 'jelly', 'jerky', 'jetty', 'jewel', 'jiffy', 'joint', 'joist', 'joker', 'jolly', 'joust', 'judge', 'juice', 'juicy', 'jumbo', 'jumpy', 'junta', 'junto', 'juror', 'kappa', 'karma', 'kayak', 'kebab', 'khaki', 'kinky', 'kiosk', 'kitty', 'knack', 'knave', 'knead', 'kneed', 'kneel', 'knelt', 'knife', 'knock', 'knoll', 'known', 'koala', 'krill', 'label', 'labor', 'laden', 'ladle', 'lager', 'lance', 'lanky', 'lapel', 'lapse', 'large', 'larva', 'lasso', 'latch', 'later', 'lathe', 'latte', 'laugh', 'layer', 'leach', 'leafy', 'leaky', 'leant', 'leapt', 'learn', 'lease', 'leash', 'least', 'leave', 'ledge', 'leech', 'leery', 'lefty', 'legal', 'leggy', 'lemon', 'lemur', 'leper', 'level', 'lever', 'libel', 'liege', 'light', 'liken', 'lilac', 'limbo', 'limit', 'linen', 'liner', 'lingo', 'lipid', 'lithe', 'liver', 'livid', 'llama', 'loamy', 'loath', 'lobby', 'local', 'locus', 'lodge', 'lofty', 'logic', 'login', 'loopy', 'loose', 'lorry', 'loser', 'louse', 'lousy', 'lover', 'lower', 'lowly', 'loyal', 'lucid', 'lucky', 'lumen', 'lumpy', 'lunar', 'lunch', 'lunge', 'lupus', 'lurch', 'lurid', 'lusty', 'lying', 'lymph', 'lynch', 'lyric', 'macaw', 'macho', 'macro', 'madam', 'madly', 'mafia', 'magic', 'magma', 'maize', 'major', 'maker', 'mambo', 'mamma', 'mammy', 'manga', 'mange', 'mango', 'mangy', 'mania', 'manic', 'manly', 'manor', 'maple', 'march', 'marry', 'marsh', 'mason', 'masse', 'match', 'matey', 'mauve', 'maxim', 'maybe', 'mayor', 'mealy', 'meant', 'meaty', 'mecca', 'medal', 'media', 'medic', 'melee', 'melon', 'mercy', 'merge', 'merit', 'merry', 'metal', 'meter', 'metro', 'micro', 'midge', 'midst', 'might', 'milky', 'mimic', 'mince', 'miner', 'minim', 'minor', 'minty', 'minus', 'mirth', 'miser', 'missy', 'mocha', 'modal', 'model', 'modem', 'mogul', 'moist', 'molar', 'moldy', 'money', 'month', 'moody', 'moose', 'moral', 'moron', 'morph', 'mossy', 'motel', 'motif', 'motor', 'motto', 'moult', 'mound', 'mount', 'mourn', 'mouse', 'mouth', 'mover', 'movie', 'mower', 'mucky', 'mucus', 'muddy', 'mulch', 'mummy', 'munch', 'mural', 'murky', 'mushy', 'music', 'musky', 'musty', 'myrrh', 'nadir', 'naive', 'nanny', 'nasal', 'nasty', 'natal', 'naval', 'navel', 'needy', 'neigh', 'nerdy', 'nerve', 'never', 'newer', 'newly', 'nicer', 'niche', 'niece', 'night', 'ninja', 'ninny', 'ninth', 'noble', 'nobly', 'noise', 'noisy', 'nomad', 'noose', 'north', 'nosey', 'notch', 'novel', 'nudge', 'nurse', 'nutty', 'nylon', 'nymph', 'oaken', 'obese', 'occur', 'ocean', 'octal', 'octet', 'odder', 'oddly', 'offal', 'offer', 'often', 'olden', 'older', 'olive', 'ombre', 'omega', 'onion', 'onset', 'opera', 'opine', 'opium', 'optic', 'orbit', 'order', 'organ', 'other', 'otter', 'ought', 'ounce', 'outdo', 'outer', 'outgo', 'ovary', 'ovate', 'overt', 'ovine', 'ovoid', 'owing', 'owner', 'oxide', 'ozone', 'paddy', 'pagan', 'paint', 'paler', 'palsy', 'panel', 'panic', 'pansy', 'papal', 'paper', 'parer', 'parka', 'parry', 'parse', 'party', 'pasta', 'paste', 'pasty', 'patch', 'patio', 'patsy', 'patty', 'pause', 'payee', 'payer', 'peace', 'peach', 'pearl', 'pecan', 'pedal', 'penal', 'pence', 'penne', 'penny', 'perch', 'peril', 'perky', 'pesky', 'pesto', 'petal', 'petty', 'phase', 'phone', 'phony', 'photo', 'piano', 'picky', 'piece', 'piety', 'piggy', 'pilot', 'pinch', 'piney', 'pinky', 'pinto', 'piper', 'pique', 'pitch', 'pithy', 'pivot', 'pixel', 'pixie', 'pizza', 'place', 'plaid', 'plain', 'plait', 'plane', 'plank', 'plant', 'plate', 'plaza', 'plead', 'pleat', 'plied', 'plier', 'pluck', 'plumb', 'plume', 'plump', 'plunk', 'plush', 'poesy', 'point', 'poise', 'poker', 'polar', 'polka', 'polyp', 'pooch', 'poppy', 'porch', 'poser', 'posit', 'posse', 'pouch', 'pound', 'pouty', 'power', 'prank', 'prawn', 'preen', 'press', 'price', 'prick', 'pride', 'pried', 'prime', 'primo', 'primp', 'print', 'prior', 'prism', 'privy', 'prize', 'probe', 'prone', 'prong', 'proof', 'prose', 'proud', 'prove', 'prowl', 'proxy', 'prude', 'prune', 'psalm', 'pubic', 'pudgy', 'puffy', 'pulpy', 'pulse', 'punch', 'pupal', 'pupil', 'puppy', 'puree', 'purer', 'purge', 'purse', 'pushy', 'putty', 'pygmy', 'quack', 'quail', 'quake', 'qualm', 'quark', 'quart', 'quash', 'quasi', 'queen', 'queer', 'quell', 'query', 'quest', 'queue', 'quick', 'quiet', 'quill', 'quilt', 'quirk', 'quite', 'quota', 'quote', 'quoth', 'rabbi', 'rabid', 'racer', 'radar', 'radii', 'radio', 'rainy', 'raise', 'rajah', 'rally', 'ralph', 'ramen', 'ranch', 'randy', 'range', 'rapid', 'rarer', 'raspy', 'ratio', 'ratty', 'raven', 'rayon', 'razor', 'reach', 'react', 'ready', 'realm', 'rearm', 'rebar', 'rebel', 'rebus', 'rebut', 'recap', 'recur', 'recut', 'reedy', 'refer', 'refit', 'regal', 'rehab', 'reign', 'relax', 'relay', 'relic', 'remit', 'renal', 'renew', 'repay', 'repel', 'reply', 'rerun', 'reset', 'resin', 'retch', 'retro', 'retry', 'reuse', 'revel', 'revue', 'rhino', 'rhyme', 'rider', 'ridge', 'rifle', 'right', 'rigid', 'rigor', 'rinse', 'ripen', 'riper', 'risen', 'riser', 'risky', 'rival', 'river', 'rivet', 'roach', 'roast', 'robin', 'robot', 'rocky', 'rodeo', 'roger', 'rogue', 'roomy', 'roost', 'rotor', 'rouge', 'rough', 'round', 'rouse', 'route', 'rover', 'rowdy', 'rower', 'royal', 'ruddy', 'ruder', 'rugby', 'ruler', 'rumba', 'rumor', 'rupee', 'rural', 'rusty', 'sadly', 'safer', 'saint', 'salad', 'sally', 'salon', 'salsa', 'salty', 'salve', 'salvo', 'sandy', 'saner', 'sappy', 'sassy', 'satin', 'satyr', 'sauce', 'saucy', 'sauna', 'saute', 'savor', 'savoy', 'savvy', 'scald', 'scale', 'scalp', 'scaly', 'scamp', 'scant', 'scare', 'scarf', 'scary', 'scene', 'scent', 'scion', 'scoff', 'scold', 'scone', 'scoop', 'scope', 'score', 'scorn', 'scour', 'scout', 'scowl', 'scram', 'scrap', 'scree', 'screw', 'scrub', 'scrum', 'scuba', 'sedan', 'seedy', 'segue', 'seize', 'semen', 'sense', 'sepia', 'serif', 'serum', 'serve', 'setup', 'seven', 'sever', 'sewer', 'shack', 'shade', 'shady', 'shaft', 'shake', 'shaky', 'shale', 'shall', 'shalt', 'shame', 'shank', 'shape', 'shard', 'share', 'shark', 'sharp', 'shave', 'shawl', 'shear', 'sheen', 'sheep', 'sheer', 'sheet', 'sheik', 'shelf', 'shell', 'shied', 'shift', 'shine', 'shiny', 'shire', 'shirk', 'shirt', 'shoal', 'shock', 'shone', 'shook', 'shoot', 'shore', 'shorn', 'short', 'shout', 'shove', 'shown', 'showy', 'shrew', 'shrub', 'shrug', 'shuck', 'shunt', 'shush', 'shyly', 'siege', 'sieve', 'sight', 'sigma', 'silky', 'silly', 'since', 'sinew', 'singe', 'siren', 'sissy', 'sixth', 'sixty', 'skate', 'skier', 'skiff', 'skill', 'skimp', 'skirt', 'skulk', 'skull', 'skunk', 'slack', 'slain', 'slang', 'slant', 'slash', 'slate', 'slave', 'sleek', 'sleep', 'sleet', 'slept', 'slice', 'slick', 'slide', 'slime', 'slimy', 'sling', 'slink', 'sloop', 'slope', 'slosh', 'sloth', 'slump', 'slung', 'slunk', 'slurp', 'slush', 'slyly', 'smack', 'small', 'smart', 'smash', 'smear', 'smell', 'smelt', 'smile', 'smirk', 'smite', 'smith', 'smock', 'smoke', 'smoky', 'smote', 'snack', 'snail', 'snake', 'snaky', 'snare', 'snarl', 'sneak', 'sneer', 'snide', 'sniff', 'snipe', 'snoop', 'snore', 'snort', 'snout', 'snowy', 'snuck', 'snuff', 'soapy', 'sober', 'soggy', 'solar', 'solid', 'solve', 'sonar', 'sonic', 'sooth', 'sooty', 'sorry', 'sound', 'south', 'sower', 'space', 'spade', 'spank', 'spare', 'spark', 'spasm', 'spawn', 'speak', 'spear', 'speck', 'speed', 'spell', 'spelt', 'spend', 'spent', 'sperm', 'spice', 'spicy', 'spied', 'spiel', 'spike', 'spiky', 'spill', 'spilt', 'spine', 'spiny', 'spire', 'spite', 'splat', 'split', 'spoil', 'spoke', 'spoof', 'spook', 'spool', 'spoon', 'spore', 'sport', 'spout', 'spray', 'spree', 'sprig', 'spunk', 'spurn', 'spurt', 'squad', 'squat', 'squib', 'stack', 'staff', 'stage', 'staid', 'stain', 'stair', 'stake', 'stale', 'stalk', 'stall', 'stamp', 'stand', 'stank', 'stare', 'stark', 'start', 'stash', 'state', 'stave', 'stead', 'steak', 'steal', 'steam', 'steed', 'steel', 'steep', 'steer', 'stein', 'stern', 'stick', 'stiff', 'still', 'stilt', 'sting', 'stink', 'stint', 'stock', 'stoic', 'stoke', 'stole', 'stomp', 'stone', 'stony', 'stood', 'stool', 'stoop', 'store', 'stork', 'storm', 'story', 'stout', 'stove', 'strap', 'straw', 'stray', 'strip', 'strut', 'stuck', 'study', 'stuff', 'stump', 'stung', 'stunk', 'stunt', 'style', 'suave', 'sugar', 'suing', 'suite', 'sulky', 'sully', 'sumac', 'sunny', 'super', 'surer', 'surge', 'surly', 'sushi', 'swami', 'swamp', 'swarm', 'swash', 'swath', 'swear', 'sweat', 'sweep', 'sweet', 'swell', 'swept', 'swift', 'swill', 'swine', 'swing', 'swirl', 'swish', 'swoon', 'swoop', 'sword', 'swore', 'sworn', 'swung', 'synod', 'syrup', 'tabby', 'table', 'taboo', 'tacit', 'tacky', 'taffy', 'taint', 'taken', 'taker', 'tally', 'talon', 'tamer', 'tango', 'tangy', 'taper', 'tapir', 'tardy', 'tarot', 'taste', 'tasty', 'tatty', 'taunt', 'tawny', 'teach', 'teary', 'tease', 'teddy', 'teeth', 'tempo', 'tenet', 'tenor', 'tense', 'tenth', 'tepee', 'tepid', 'terra', 'terse', 'testy', 'thank', 'theft', 'their', 'theme', 'there', 'these', 'theta', 'thick', 'thief', 'thigh', 'thing', 'think', 'third', 'thong', 'thorn', 'those', 'three', 'threw', 'throb', 'throw', 'thrum', 'thumb', 'thump', 'thyme', 'tiara', 'tibia', 'tidal', 'tiger', 'tight', 'tilde', 'timer', 'timid', 'tipsy', 'titan', 'tithe', 'title', 'toast', 'today', 'toddy', 'token', 'tonal', 'tonga', 'tonic', 'tooth', 'topaz', 'topic', 'torch', 'torso', 'torus', 'total', 'totem', 'touch', 'tough', 'towel', 'tower', 'toxic', 'toxin', 'trace', 'track', 'tract', 'trade', 'trail', 'train', 'trait', 'tramp', 'trash', 'trawl', 'tread', 'treat', 'trend', 'triad', 'trial', 'tribe', 'trice', 'trick', 'tried', 'tripe', 'trite', 'troll', 'troop', 'trope', 'trout', 'trove', 'truce', 'truck', 'truer', 'truly', 'trump', 'trunk', 'truss', 'trust', 'truth', 'tryst', 'tubal', 'tuber', 'tulip', 'tulle', 'tumor', 'tunic', 'turbo', 'tutor', 'twang', 'tweak', 'tweed', 'tweet', 'twice', 'twine', 'twirl', 'twist', 'twixt', 'tying', 'udder', 'ulcer', 'ultra', 'umbra', 'uncle', 'uncut', 'under', 'undid', 'undue', 'unfed', 'unfit', 'unify', 'union', 'unite', 'unity', 'unlit', 'unmet', 'unset', 'untie', 'until', 'unwed', 'unzip', 'upper', 'upset', 'urban', 'urine', 'usage', 'usher', 'using', 'usual', 'usurp', 'utile', 'utter', 'uvula', 'vague', 'valet', 'valid', 'valor', 'value', 'valve', 'vapid', 'vapor', 'vault', 'vaunt', 'vegan', 'venom', 'venue', 'verge', 'verse', 'verso', 'verve', 'vicar', 'video', 'vigil', 'vigor', 'villa', 'vinyl', 'viola', 'viper', 'viral', 'virus', 'visit', 'visor', 'vista', 'vital', 'vivid', 'vixen', 'vocal', 'vodka', 'vogue', 'voice', 'voila', 'vomit', 'voter', 'vouch', 'vowel', 'vying', 'wacky', 'wafer', 'wager', 'wagon', 'waist', 'waive', 'waltz', 'warty', 'waste', 'watch', 'water', 'waver', 'waxen', 'weary', 'weave', 'wedge', 'weedy', 'weigh', 'weird', 'welch', 'welsh', 'wench', 'whack', 'whale', 'wharf', 'wheat', 'wheel', 'whelp', 'where', 'which', 'whiff', 'while', 'whine', 'whiny', 'whirl', 'whisk', 'white', 'whole', 'whoop', 'whose', 'widen', 'wider', 'widow', 'width', 'wield', 'wight', 'willy', 'wimpy', 'wince', 'winch', 'windy', 'wiser', 'wispy', 'witch', 'witty', 'woken', 'woman', 'women', 'woody', 'wooer', 'wooly', 'woozy', 'wordy', 'world', 'worry', 'worse', 'worst', 'worth', 'would', 'wound', 'woven', 'wrack', 'wrath', 'wreak', 'wreck', 'wrest', 'wring', 'wrist', 'write', 'wrong', 'wrote', 'wrung', 'wryly', 'yacht', 'yearn', 'yeast', 'yield', 'young', 'youth', 'zebra', 'zesty', 'zonal'
    ]

print(f'{len(wordle_word_list) = }')


len(wordle_word_list) = 2317


In [ ]:

wordle_word_dict: Dict[str, Tuple[int, int, int, int]] = {

'aback':  (60, 96, 31, 96), 'abase':  (82, 96, 46, 96), 'abate':  (82, 96, 44, 96), 'abbey':  (81, 96, 39, 96), 'abbot':  (77, 96, 26, 96),
'abhor':  (84, 96, 22, 96), 'abide':  (87, 96, 40, 96), 'abled':  (84, 96, 30, 96), 'abode':  (87, 96, 39, 96), 'abort':  (86, 96, 35, 96),
'about':  (84, 96, 32, 96), 'above':  (86, 96, 38, 96), 'abuse':  (88, 96, 40, 96), 'abyss':  (70, 96, 17, 96), 'acorn':  (86, 96, 31, 96),
'acrid':  (85, 96, 27, 96), 'actor':  (87, 96, 27, 96), 'acute':  (90, 96, 39, 96), 'adage':  (78, 96, 42, 96), 'adapt':  (73, 96, 33, 96),
'adept':  (87, 96, 28, 96), 'admin':  (78, 96, 22, 96), 'admit':  (81, 96, 27, 96), 'adobe':  (87, 96, 37, 96), 'adopt':  (83, 96, 31, 96),
'adore':  (90, 96, 42, 96), 'adorn':  (86, 96, 30, 96), 'adult':  (83, 96, 32, 96), 'affix':  (65, 96, 15, 96), 'afire':  (90, 96, 43, 96),
'afoot':  (76, 96, 34, 96), 'afoul':  (84, 96, 27, 96), 'after':  (87, 96, 34, 96), 'again':  (74, 96, 32, 96), 'agape':  (79, 96, 40, 96),
'agate':  (83, 96, 44, 96), 'agent':  (87, 96, 33, 96), 'agile':  (89, 96, 43, 96), 'aging':  (74, 96, 28, 96), 'aglow':  (79, 96, 18, 96),
'agony':  (80, 96, 41, 96), 'agora':  (80, 96, 26, 96), 'agree':  (82, 96, 46, 96), 'ahead':  (81, 96, 32, 96), 'aider':  (90, 96, 41, 96),
'aisle':  (92, 97, 44, 97), 'alarm':  (78, 96, 36, 96), 'album':  (74, 96, 23, 96), 'alert':  (92, 97, 40, 97), 'algae':  (81, 96, 43, 96),
'alibi':  (74, 96, 28, 96), 'alien':  (91, 96, 46, 97), 'align':  (81, 96, 35, 96), 'alike':  (88, 96, 47, 96), 'alive':  (87, 96, 47, 96),
'allay':  (63, 96, 42, 96), 'alley':  (84, 96, 49, 96), 'allot':  (82, 96, 36, 96), 'allow':  (75, 96, 26, 96), 'alloy':  (78, 96, 41, 96),
'aloft':  (84, 96, 38, 96), 'alone':  (92, 97, 51, 97), 'along':  (83, 96, 35, 96), 'aloof':  (75, 96, 32, 96), 'aloud':  (85, 96, 34, 96),
'alpha':  (71, 96, 21, 96), 'altar':  (84, 96, 36, 96), 'alter':  (92, 97, 42, 97), 'amass':  (62, 96, 30, 96), 'amaze':  (74, 96, 40, 96),
'amber':  (84, 96, 33, 96), 'amble':  (84, 96, 36, 96), 'amend':  (85, 96, 28, 96), 'amiss':  (77, 96, 28, 96), 'amity':  (82, 96, 41, 96),
'among':  (78, 96, 28, 96), 'ample':  (84, 96, 36, 96), 'amply':  (73, 96, 33, 96), 'amuse':  (88, 96, 41, 96), 'angel':  (87, 96, 33, 96),
'anger':  (87, 96, 36, 96), 'angle':  (87, 96, 38, 96), 'angry':  (83, 96, 35, 96), 'angst':  (82, 96, 31, 96), 'anime':  (89, 96, 43, 96),
'ankle':  (86, 96, 36, 96), 'annex':  (78, 96, 30, 96), 'annoy':  (78, 96, 37, 96), 'annul':  (77, 96, 26, 96), 'anode':  (90, 96, 42, 96),
'antic':  (84, 96, 23, 96), 'anvil':  (80, 96, 26, 96), 'aorta':  (83, 96, 34, 96), 'apart':  (78, 96, 39, 96), 'aphid':  (79, 96, 21, 96),
'aping':  (79, 96, 30, 96), 'apnea':  (82, 96, 31, 96), 'apple':  (82, 96, 37, 96), 'apply':  (69, 96, 34, 96), 'apron':  (86, 96, 27, 96),
'aptly':  (81, 96, 36, 96), 'arbor':  (78, 96, 35, 96), 'ardor':  (80, 96, 36, 96), 'arena':  (85, 96, 36, 96), 'argue':  (89, 96, 42, 96),
'arise':  (93, 97, 55, 97), 'armor':  (79, 96, 35, 96), 'aroma':  (79, 96, 34, 96), 'arose':  (92, 97, 54, 97), 'array':  (71, 96, 47, 96),
'arrow':  (78, 96, 31, 96), 'arson':  (89, 96, 32, 96), 'artsy':  (87, 96, 46, 96), 'ascot':  (86, 96, 26, 96), 'ashen':  (89, 96, 26, 96),
'aside':  (90, 96, 40, 96), 'askew':  (83, 96, 22, 96), 'assay':  (66, 96, 33, 96), 'asset':  (85, 96, 35, 96), 'atoll':  (82, 96, 34, 96),
'atone':  (92, 96, 46, 96), 'attic':  (78, 96, 22, 96), 'audio':  (92, 96, 27, 96), 'audit':  (85, 96, 35, 96), 'augur':  (77, 96, 30, 96),
'aunty':  (83, 96, 42, 96), 'avail':  (73, 96, 34, 96), 'avert':  (86, 96, 32, 96), 'avian':  (72, 96, 31, 96), 'avoid':  (84, 96, 29, 96),
'await':  (76, 96, 39, 96), 'awake':  (76, 96, 42, 96), 'award':  (71, 96, 33, 96), 'aware':  (81, 96, 46, 96), 'awash':  (68, 96, 35, 96),
'awful':  (74, 96, 19, 96), 'awoke':  (87, 96, 39, 96), 'axial':  (71, 96, 32, 96), 'axiom':  (83, 96, 26, 96), 'axion':  (85, 96, 30, 96),
'azure':  (87, 96, 38, 96), 'bacon':  (80, 96, 34, 96), 'badge':  (81, 96, 45, 96), 'badly':  (73, 96, 47, 96), 'bagel':  (83, 96, 44, 96),
'baggy':  (63, 96, 42, 96), 'baker':  (84, 96, 44, 96), 'baler':  (87, 96, 48, 96), 'balmy':  (71, 96, 44, 96), 'banal':  (71, 96, 40, 96),
'banjo':  (75, 96, 29, 96), 'barge':  (85, 96, 49, 96), 'baron':  (85, 96, 39, 96), 'basal':  (73, 96, 38, 96), 'basic':  (82, 96, 32, 96),
'basil':  (83, 96, 38, 96), 'basin':  (83, 96, 36, 96), 'basis':  (77, 96, 32, 96), 'baste':  (87, 96, 48, 96), 'batch':  (75, 96, 38, 96),
'bathe':  (86, 96, 45, 96), 'baton':  (83, 96, 37, 96), 'batty':  (71, 96, 47, 96), 'bawdy':  (68, 96, 40, 96), 'bayou':  (82, 96, 28, 96),
'beach':  (84, 96, 44, 96), 'beady':  (84, 96, 50, 96), 'beard':  (84, 96, 43, 96), 'beast':  (87, 96, 49, 96), 'beech':  (69, 96, 38, 96),
'beefy':  (69, 96, 43, 96), 'befit':  (80, 96, 37, 96), 'began':  (83, 96, 33, 96), 'begat':  (85, 96, 39, 96), 'beget':  (73, 96, 45, 96),
'begin':  (79, 96, 33, 96), 'begun':  (76, 96, 30, 96), 'being':  (79, 96, 39, 96), 'belch':  (79, 96, 35, 96), 'belie':  (78, 96, 48, 96),
'belle':  (66, 96, 48, 96), 'belly':  (75, 96, 45, 96), 'below':  (81, 96, 29, 96), 'bench':  (79, 96, 36, 96), 'beret':  (78, 96, 50, 96),
'berry':  (77, 96, 47, 96), 'berth':  (83, 96, 37, 96), 'beset':  (77, 96, 46, 96), 'betel':  (79, 96, 43, 96), 'bevel':  (67, 96, 40, 96),
'bezel':  (66, 96, 39, 96), 'bible':  (78, 96, 44, 96), 'bicep':  (80, 96, 35, 96), 'biddy':  (56, 96, 38, 96), 'bigot':  (76, 96, 36, 96),
'bilge':  (81, 96, 43, 96), 'billy':  (64, 96, 44, 96), 'binge':  (79, 96, 44, 96), 'bingo':  (71, 96, 28, 96), 'biome':  (86, 96, 48, 96),
'birch':  (75, 96, 36, 96), 'birth':  (79, 96, 35, 96), 'bison':  (79, 96, 31, 96), 'bitty':  (65, 96, 43, 96), 'black':  (70, 96, 41, 96),
'blade':  (84, 96, 51, 96), 'blame':  (84, 96, 51, 96), 'bland':  (77, 96, 42, 96), 'blank':  (74, 96, 42, 96), 'blare':  (87, 96, 54, 96),
'blast':  (81, 96, 48, 96), 'blaze':  (80, 96, 49, 96), 'bleak':  (83, 96, 36, 96), 'bleat':  (88, 96, 42, 96), 'bleed':  (72, 96, 43, 96),
'bleep':  (72, 96, 40, 96), 'blend':  (81, 96, 37, 96), 'bless':  (77, 96, 33, 96), 'blimp':  (68, 96, 33, 96), 'blind':  (71, 96, 41, 96),
'blink':  (69, 96, 40, 96), 'bliss':  (70, 96, 37, 96), 'blitz':  (70, 96, 34, 96), 'bloat':  (84, 96, 45, 96), 'block':  (68, 96, 38, 96),
'bloke':  (82, 96, 47, 96), 'blond':  (73, 96, 40, 96), 'blood':  (63, 96, 37, 96), 'bloom':  (62, 96, 34, 96), 'blown':  (71, 96, 33, 96),
'bluer':  (85, 96, 46, 96), 'bluff':  (53, 96, 26, 96), 'blunt':  (77, 96, 42, 96), 'blurb':  (72, 96, 30, 96), 'blurt':  (83, 96, 41, 96),
'blush':  (72, 96, 37, 96), 'board':  (82, 96, 44, 96), 'boast':  (84, 96, 51, 96), 'bobby':  (50, 96, 39, 96), 'boney':  (85, 96, 55, 96),
'bongo':  (58, 96, 31, 96), 'bonus':  (74, 96, 31, 96), 'booby':  (50, 96, 47, 96), 'boost':  (69, 96, 48, 96), 'booth':  (64, 96, 42, 96),
'booty':  (66, 96, 52, 96), 'booze':  (70, 96, 49, 96), 'boozy':  (50, 96, 47, 96), 'borax':  (79, 96, 34, 96), 'borne':  (85, 96, 53, 96),
'bosom':  (62, 96, 30, 96), 'bossy':  (65, 96, 46, 96), 'botch':  (71, 96, 37, 96), 'bough':  (66, 96, 36, 96), 'boule':  (85, 96, 52, 96),
'bound':  (69, 96, 40, 96), 'bowel':  (81, 96, 41, 96), 'boxer':  (79, 96, 43, 96), 'brace':  (86, 96, 57, 96), 'braid':  (84, 96, 44, 96),
'brain':  (86, 96, 45, 96), 'brake':  (84, 96, 53, 96), 'brand':  (81, 96, 45, 96), 'brash':  (83, 96, 46, 96), 'brass':  (78, 96, 41, 96),
'brave':  (82, 96, 53, 96), 'bravo':  (79, 96, 37, 96), 'brawl':  (79, 96, 40, 96), 'brawn':  (79, 96, 39, 96), 'bread':  (84, 96, 39, 96),
'break':  (84, 96, 39, 96), 'breed':  (73, 96, 45, 96), 'briar':  (80, 96, 47, 96), 'bribe':  (80, 96, 50, 96), 'brick':  (73, 96, 42, 96),
'bride':  (83, 96, 52, 96), 'brief':  (82, 96, 45, 96), 'brine':  (85, 96, 57, 96), 'bring':  (75, 96, 40, 96), 'brink':  (75, 96, 43, 96),
'briny':  (79, 96, 54, 96), 'brisk':  (78, 96, 43, 96), 'broad':  (82, 96, 42, 96), 'broil':  (84, 96, 43, 96), 'broke':  (82, 96, 50, 96),
'brood':  (66, 96, 40, 96), 'brook':  (64, 96, 40, 96), 'broom':  (65, 96, 37, 96), 'broth':  (78, 96, 42, 96), 'brown':  (74, 96, 36, 96),
'brunt':  (80, 96, 45, 96), 'brush':  (76, 96, 40, 96), 'brute':  (84, 96, 50, 96), 'buddy':  (51, 96, 37, 96), 'budge':  (74, 96, 40, 96),
'buggy':  (50, 96, 37, 96), 'bugle':  (78, 96, 44, 96), 'build':  (71, 96, 39, 96), 'built':  (77, 96, 45, 96), 'bulge':  (78, 96, 42, 96),
'bulky':  (63, 96, 38, 96), 'bully':  (59, 96, 43, 96), 'bunch':  (65, 96, 34, 96), 'bunny':  (57, 96, 45, 96), 'burly':  (79, 96, 45, 96),
'burnt':  (80, 96, 41, 96), 'burst':  (79, 96, 41, 96), 'bused':  (80, 96, 38, 96), 'bushy':  (67, 96, 36, 96), 'butch':  (66, 96, 33, 96),
'butte':  (75, 96, 45, 96), 'buxom':  (58, 96, 24, 96), 'buyer':  (84, 96, 40, 96), 'bylaw':  (71, 96, 21, 96), 'cabal':  (68, 96, 38, 96),
'cabby':  (67, 96, 41, 96), 'cabin':  (79, 96, 37, 96), 'cable':  (85, 96, 49, 96), 'cacao':  (69, 96, 34, 96), 'cache':  (81, 96, 44, 96),
'cacti':  (78, 96, 31, 96), 'caddy':  (68, 96, 44, 96), 'cadet':  (88, 96, 50, 96), 'cagey':  (87, 96, 54, 96), 'cairn':  (87, 96, 45, 96),
'camel':  (86, 96, 45, 96), 'cameo':  (89, 96, 41, 96), 'canal':  (75, 96, 41, 96), 'candy':  (77, 96, 46, 96), 'canny':  (72, 96, 51, 96),
'canoe':  (91, 96, 52, 96), 'canon':  (77, 96, 39, 96), 'caper':  (87, 96, 47, 96), 'caput':  (80, 96, 39, 96), 'carat':  (80, 96, 47, 96),
'cargo':  (84, 96, 34, 96), 'carol':  (88, 96, 41, 96), 'carry':  (78, 96, 51, 96), 'carve':  (84, 96, 49, 96), 'caste':  (89, 96, 49, 97),
'catch':  (70, 96, 39, 96), 'cater':  (89, 96, 49, 96), 'catty':  (75, 96, 48, 96), 'caulk':  (75, 96, 41, 96), 'cause':  (89, 96, 55, 96),
'cavil':  (78, 96, 37, 96), 'cease':  (84, 96, 58, 96), 'cedar':  (87, 96, 38, 96), 'cello':  (82, 96, 33, 96), 'chafe':  (83, 96, 48, 96),
'chaff':  (63, 96, 31, 96), 'chain':  (80, 96, 40, 96), 'chair':  (85, 96, 44, 96), 'chalk':  (72, 96, 40, 96), 'champ':  (71, 96, 33, 96),
'chant':  (79, 96, 47, 96), 'chaos':  (82, 96, 35, 96), 'chard':  (81, 96, 40, 96), 'charm':  (79, 96, 36, 96), 'chart':  (83, 96, 45, 96),
'chase':  (87, 96, 54, 96), 'chasm':  (75, 96, 37, 96), 'cheap':  (84, 96, 32, 96), 'cheat':  (87, 96, 40, 96), 'check':  (67, 96, 34, 96),
'cheek':  (67, 96, 41, 96), 'cheer':  (77, 96, 45, 96), 'chess':  (75, 96, 31, 96), 'chest':  (82, 96, 41, 96), 'chick':  (55, 96, 38, 96),
'chide':  (81, 96, 48, 96), 'chief':  (79, 96, 41, 96), 'child':  (73, 96, 38, 96), 'chili':  (67, 96, 34, 96), 'chill':  (67, 96, 40, 96),
'chime':  (81, 96, 48, 96), 'china':  (80, 96, 37, 96), 'chirp':  (76, 96, 35, 96), 'chock':  (55, 96, 37, 96), 'choir':  (81, 96, 41, 96),
'choke':  (81, 96, 46, 96), 'chord':  (76, 96, 37, 96), 'chore':  (85, 96, 50, 96), 'chose':  (85, 96, 51, 96), 'chuck':  (49, 96, 33, 96),
'chump':  (61, 96, 27, 96), 'chunk':  (62, 96, 35, 96), 'churn':  (77, 96, 34, 96), 'chute':  (81, 96, 46, 96), 'cider':  (85, 96, 43, 96),
'cigar':  (84, 96, 36, 96), 'cinch':  (63, 96, 36, 96), 'circa':  (82, 96, 34, 96), 'civic':  (46, 96, 28, 96), 'civil':  (62, 96, 33, 96),
'clack':  (66, 96, 42, 96), 'claim':  (80, 96, 39, 96), 'clamp':  (74, 96, 36, 96), 'clang':  (79, 96, 40, 96), 'clank':  (76, 96, 43, 96),
'clash':  (80, 96, 44, 96), 'clasp':  (80, 96, 40, 96), 'class':  (76, 96, 39, 96), 'clean':  (88, 96, 38, 96), 'clear':  (89, 96, 41, 96),
'cleat':  (90, 96, 43, 96), 'cleft':  (84, 96, 37, 96), 'clerk':  (84, 96, 36, 96), 'click':  (63, 96, 40, 96), 'cliff':  (63, 96, 31, 96),
'climb':  (70, 96, 32, 96), 'cling':  (75, 96, 38, 96), 'clink':  (72, 96, 41, 96), 'cloak':  (80, 96, 40, 96), 'clock':  (64, 96, 39, 96),
'clone':  (88, 96, 54, 96), 'close':  (89, 96, 53, 96), 'cloth':  (79, 96, 40, 96), 'cloud':  (76, 96, 36, 96), 'clout':  (81, 96, 42, 96),
'clove':  (83, 96, 48, 96), 'clown':  (74, 96, 34, 96), 'cluck':  (58, 96, 36, 96), 'clued':  (82, 96, 43, 96), 'clump':  (68, 96, 30, 96),
'clung':  (72, 96, 34, 96), 'coach':  (74, 96, 46, 96), 'coast':  (86, 96, 52, 96), 'cobra':  (83, 96, 32, 96), 'cocoa':  (69, 96, 32, 96),
'colon':  (72, 96, 37, 96), 'color':  (78, 96, 40, 96), 'comet':  (86, 96, 48, 96), 'comfy':  (65, 96, 40, 96), 'comic':  (68, 96, 31, 96),
'comma':  (73, 96, 29, 96), 'conch':  (64, 96, 39, 96), 'condo':  (65, 96, 32, 96), 'conic':  (71, 96, 35, 96), 'copse':  (86, 96, 49, 96),
'coral':  (88, 96, 41, 96), 'corer':  (82, 96, 50, 96), 'corny':  (81, 96, 51, 96), 'couch':  (62, 96, 40, 96), 'cough':  (68, 96, 37, 96),
'could':  (76, 96, 40, 96), 'count':  (78, 96, 47, 96), 'coupe':  (85, 96, 48, 96), 'court':  (82, 96, 45, 96), 'coven':  (82, 96, 42, 96),
'cover':  (83, 96, 46, 96), 'covet':  (83, 96, 47, 96), 'covey':  (83, 96, 52, 96), 'cower':  (83, 96, 45, 96), 'coyly':  (70, 96, 45, 96),
'crack':  (72, 96, 45, 96), 'craft':  (82, 96, 46, 96), 'cramp':  (79, 96, 39, 96), 'crane':  (89, 97, 60, 97), 'crank':  (80, 96, 46, 96),
'crash':  (83, 96, 47, 96), 'crass':  (80, 96, 42, 96), 'crate':  (89, 97, 58, 97), 'crave':  (84, 96, 54, 96), 'crawl':  (82, 96, 41, 96),
'craze':  (84, 96, 53, 96), 'crazy':  (78, 96, 50, 96), 'creak':  (85, 96, 40, 96), 'cream':  (86, 96, 37, 96), 'credo':  (85, 96, 33, 96),
'creed':  (77, 96, 47, 96), 'creek':  (74, 96, 46, 96), 'creep':  (77, 96, 44, 96), 'creme':  (77, 96, 49, 96), 'crepe':  (77, 96, 48, 96),
'crept':  (85, 96, 41, 96), 'cress':  (82, 96, 37, 96), 'crest':  (88, 96, 46, 96), 'crick':  (68, 96, 43, 96), 'cried':  (85, 96, 50, 96),
'crier':  (82, 96, 55, 96), 'crime':  (85, 96, 53, 96), 'crimp':  (75, 96, 37, 96), 'crisp':  (81, 96, 41, 96), 'croak':  (82, 96, 43, 96),
'crock':  (67, 96, 42, 96), 'crone':  (88, 96, 57, 96), 'crony':  (81, 96, 54, 96), 'crook':  (67, 96, 41, 96), 'cross':  (77, 96, 40, 96),
'croup':  (77, 96, 37, 96), 'crowd':  (73, 96, 37, 96), 'crown':  (76, 96, 37, 96), 'crude':  (83, 96, 49, 96), 'cruel':  (87, 96, 48, 96),
'crumb':  (71, 96, 31, 96), 'crump':  (72, 96, 33, 96), 'crush':  (77, 96, 41, 96), 'crust':  (81, 96, 46, 96), 'crypt':  (80, 96, 34, 96),
'cubic':  (62, 96, 27, 96), 'cumin':  (71, 96, 32, 96), 'curio':  (84, 96, 33, 96), 'curly':  (83, 96, 46, 96), 'curry':  (70, 96, 46, 96),
'curse':  (87, 96, 49, 96), 'curve':  (81, 96, 44, 96), 'curvy':  (73, 96, 41, 96), 'cutie':  (87, 96, 46, 96), 'cyber':  (84, 96, 35, 96),
'cycle':  (79, 96, 37, 96), 'cynic':  (66, 96, 24, 96), 'daddy':  (59, 96, 40, 96), 'daily':  (81, 96, 52, 96), 'dairy':  (86, 96, 52, 96),
'daisy':  (84, 96, 53, 96), 'dally':  (70, 96, 45, 96), 'dance':  (86, 96, 49, 96), 'dandy':  (70, 96, 43, 96), 'datum':  (78, 96, 28, 96),
'daunt':  (83, 96, 44, 96), 'dealt':  (89, 96, 46, 96), 'death':  (87, 96, 40, 96), 'debar':  (84, 96, 34, 96), 'debit':  (82, 96, 35, 96),
'debug':  (74, 96, 23, 96), 'debut':  (80, 96, 32, 96), 'decal':  (86, 96, 31, 96), 'decay':  (87, 96, 40, 96), 'decor':  (85, 96, 32, 96),
'decoy':  (85, 96, 39, 96), 'decry':  (84, 96, 40, 96), 'defer':  (72, 96, 39, 96), 'deign':  (79, 96, 36, 96), 'deity':  (85, 96, 48, 96),
'delay':  (86, 96, 43, 96), 'delta':  (89, 96, 29, 96), 'delve':  (69, 96, 40, 96), 'demon':  (83, 96, 29, 96), 'demur':  (80, 96, 31, 96),
'denim':  (81, 96, 30, 96), 'dense':  (78, 96, 47, 96), 'depot':  (84, 96, 34, 96), 'depth':  (80, 96, 30, 96), 'derby':  (80, 96, 39, 96),
'deter':  (80, 96, 43, 96), 'detox':  (80, 96, 26, 96), 'deuce':  (74, 96, 47, 96), 'devil':  (79, 96, 31, 96), 'diary':  (86, 96, 49, 96),
'dicey':  (84, 96, 45, 96), 'digit':  (64, 96, 34, 96), 'dilly':  (65, 96, 41, 96), 'dimly':  (71, 96, 39, 96), 'diner':  (85, 96, 42, 96),
'dingo':  (71, 96, 25, 96), 'dingy':  (66, 96, 38, 96), 'diode':  (83, 96, 45, 96), 'dirge':  (83, 96, 42, 96), 'dirty':  (81, 96, 42, 96),
'disco':  (80, 96, 26, 96), 'ditch':  (73, 96, 31, 96), 'ditto':  (72, 96, 27, 96), 'ditty':  (67, 96, 40, 96), 'diver':  (80, 96, 38, 96),
'dizzy':  (51, 96, 31, 96), 'dodge':  (74, 96, 42, 96), 'dodgy':  (58, 96, 39, 96), 'dogma':  (76, 96, 26, 96), 'doing':  (71, 96, 38, 96),
'dolly':  (67, 96, 44, 96), 'donor':  (73, 96, 38, 96), 'donut':  (77, 96, 37, 96), 'dopey':  (82, 96, 49, 96), 'doubt':  (72, 96, 36, 96),
'dough':  (68, 96, 33, 96), 'dowdy':  (55, 96, 37, 96), 'dowel':  (82, 96, 38, 96), 'downy':  (67, 96, 42, 96), 'dowry':  (72, 96, 40, 96),
'dozen':  (79, 96, 37, 96), 'draft':  (81, 96, 42, 96), 'drain':  (86, 96, 42, 96), 'drake':  (85, 96, 50, 96), 'drama':  (73, 96, 35, 96),
'drank':  (81, 96, 42, 96), 'drape':  (86, 96, 50, 96), 'drawl':  (81, 96, 37, 96), 'drawn':  (80, 96, 36, 96), 'dread':  (82, 96, 36, 96),
'dream':  (85, 96, 33, 96), 'dress':  (80, 96, 33, 96), 'dried':  (80, 96, 47, 96), 'drier':  (80, 96, 51, 96), 'drift':  (77, 96, 40, 96),
'drill':  (78, 96, 42, 96), 'drink':  (75, 96, 41, 96), 'drive':  (80, 96, 48, 96), 'droit':  (83, 96, 45, 96), 'droll':  (77, 96, 41, 96),
'drone':  (86, 96, 53, 96), 'drool':  (77, 96, 39, 96), 'droop':  (68, 96, 35, 96), 'dross':  (75, 96, 36, 96), 'drove':  (80, 96, 47, 96),
'drown':  (75, 96, 34, 96), 'druid':  (72, 96, 35, 96), 'drunk':  (74, 96, 36, 96), 'dryer':  (77, 96, 40, 96), 'dryly':  (74, 96, 40, 96),
'duchy':  (65, 96, 32, 96), 'dully':  (61, 96, 40, 96), 'dummy':  (52, 96, 34, 96), 'dumpy':  (60, 96, 33, 96), 'dunce':  (81, 96, 44, 96),
'dusky':  (65, 96, 34, 96), 'dusty':  (74, 96, 38, 96), 'dutch':  (69, 96, 30, 96), 'duvet':  (78, 96, 40, 96), 'dwarf':  (75, 96, 28, 96),
'dwell':  (71, 96, 28, 96), 'dwelt':  (82, 96, 32, 96), 'dying':  (66, 96, 27, 96), 'eager':  (82, 96, 42, 96), 'eagle':  (81, 96, 44, 96),
'early':  (91, 96, 46, 96), 'earth':  (89, 96, 35, 96), 'easel':  (85, 96, 40, 96), 'eaten':  (85, 96, 40, 96), 'eater':  (86, 96, 44, 96),
'ebony':  (85, 96, 38, 96), 'eclat':  (90, 96, 28, 96), 'edict':  (85, 96, 33, 96), 'edify':  (79, 96, 33, 96), 'eerie':  (77, 96, 46, 96),
'egret':  (79, 96, 35, 96), 'eight':  (83, 96, 27, 96), 'eject':  (72, 96, 28, 96), 'eking':  (78, 96, 25, 96), 'elate':  (86, 96, 49, 96),
'elbow':  (81, 96, 21, 96), 'elder':  (80, 96, 38, 96), 'elect':  (82, 96, 37, 96), 'elegy':  (76, 96, 38, 96), 'elfin':  (83, 96, 25, 96),
'elide':  (78, 96, 45, 96), 'elite':  (83, 96, 48, 96), 'elope':  (81, 96, 43, 96), 'elude':  (76, 96, 40, 96), 'email':  (89, 96, 32, 96),
'embed':  (66, 96, 26, 96), 'ember':  (73, 96, 30, 96), 'emcee':  (64, 96, 39, 96), 'empty':  (82, 96, 29, 96), 'enact':  (89, 96, 38, 96),
'endow':  (80, 96, 17, 96), 'enema':  (81, 96, 20, 96), 'enemy':  (76, 96, 33, 96), 'enjoy':  (83, 96, 28, 96), 'ennui':  (81, 96, 17, 96),
'ensue':  (79, 96, 32, 96), 'enter':  (83, 96, 35, 96), 'entry':  (90, 96, 34, 96), 'envoy':  (84, 96, 30, 96), 'epoch':  (82, 96, 29, 96),
'epoxy':  (80, 96, 32, 96), 'equal':  (86, 96, 24, 96), 'equip':  (79, 96, 20, 96), 'erase':  (86, 96, 54, 96), 'erect':  (82, 96, 40, 96),
'erode':  (79, 96, 46, 96), 'error':  (76, 96, 37, 96), 'erupt':  (85, 96, 35, 96), 'essay':  (85, 96, 30, 96), 'ester':  (83, 96, 31, 96),
'ether':  (80, 96, 30, 96), 'ethic':  (84, 96, 15, 96), 'ethos':  (87, 96, 14, 96), 'etude':  (77, 96, 35, 96), 'evade':  (75, 96, 38, 96),
'event':  (75, 96, 30, 96), 'every':  (75, 96, 34, 96), 'evict':  (81, 96, 33, 96), 'evoke':  (72, 96, 35, 96), 'exact':  (85, 96, 34, 96),
'exalt':  (86, 96, 35, 96), 'excel':  (71, 96, 27, 96), 'exert':  (76, 96, 29, 96), 'exile':  (75, 96, 41, 96), 'exist':  (82, 96, 34, 96),
'expel':  (68, 96, 27, 96), 'extol':  (85, 96, 21, 96), 'extra':  (86, 96, 18, 96), 'exult':  (82, 96, 29, 96), 'eying':  (83, 96, 25, 96),
'fable':  (83, 96, 47, 96), 'facet':  (86, 96, 46, 96), 'faint':  (81, 96, 49, 96), 'fairy':  (85, 96, 53, 96), 'faith':  (80, 96, 43, 96),
'false':  (87, 96, 50, 96), 'fancy':  (75, 96, 47, 96), 'fanny':  (68, 96, 49, 96), 'farce':  (86, 96, 51, 96), 'fatal':  (73, 96, 38, 96),
'fatty':  (70, 96, 45, 96), 'fault':  (80, 96, 44, 96), 'fauna':  (71, 96, 37, 96), 'favor':  (79, 96, 36, 96), 'feast':  (87, 96, 48, 96),
'fecal':  (85, 96, 32, 96), 'feign':  (78, 96, 37, 96), 'fella':  (80, 96, 31, 96), 'felon':  (85, 96, 32, 96), 'femme':  (59, 96, 40, 96),
'femur':  (80, 96, 32, 96), 'fence':  (73, 96, 47, 96), 'feral':  (87, 96, 37, 96), 'ferry':  (77, 96, 46, 96), 'fetal':  (88, 96, 35, 96),
'fetch':  (79, 96, 34, 96), 'fetid':  (81, 96, 33, 96), 'fetus':  (81, 96, 26, 96), 'fever':  (69, 96, 41, 96), 'fewer':  (70, 96, 40, 96),
'fiber':  (82, 96, 40, 96), 'fibre':  (82, 96, 42, 96), 'ficus':  (72, 96, 22, 96), 'field':  (80, 96, 34, 96), 'fiend':  (79, 96, 35, 96),
'fiery':  (86, 96, 44, 96), 'fifth':  (61, 96, 28, 96), 'fifty':  (63, 96, 37, 96), 'fight':  (68, 96, 30, 96), 'filer':  (87, 96, 42, 96),
'filet':  (85, 96, 44, 96), 'filly':  (62, 96, 42, 96), 'filmy':  (68, 96, 38, 96), 'filth':  (74, 96, 31, 96), 'final':  (80, 96, 34, 96),
'finch':  (66, 96, 33, 96), 'finer':  (85, 96, 43, 96), 'first':  (81, 96, 40, 96), 'fishy':  (70, 96, 35, 96), 'fixer':  (79, 96, 38, 96),
'fizzy':  (48, 96, 32, 96), 'fjord':  (66, 96, 28, 96), 'flack':  (70, 96, 39, 96), 'flail':  (74, 96, 41, 96), 'flair':  (86, 96, 44, 96),
'flake':  (82, 96, 48, 96), 'flaky':  (70, 96, 46, 96), 'flame':  (83, 96, 49, 96), 'flank':  (74, 96, 41, 96), 'flare':  (87, 96, 53, 96),
'flash':  (78, 96, 41, 96), 'flask':  (74, 96, 40, 96), 'fleck':  (76, 96, 34, 96), 'fleet':  (78, 96, 47, 96), 'flesh':  (82, 96, 36, 96),
'flick':  (66, 96, 38, 96), 'flier':  (87, 96, 49, 96), 'fling':  (69, 96, 36, 96), 'flint':  (77, 96, 45, 96), 'flirt':  (84, 96, 44, 96),
'float':  (84, 96, 43, 96), 'flock':  (68, 96, 37, 96), 'flood':  (63, 96, 36, 96), 'floor':  (74, 96, 40, 96), 'flora':  (86, 96, 34, 96),
'floss':  (69, 96, 34, 96), 'flour':  (81, 96, 38, 96), 'flout':  (77, 96, 40, 96), 'flown':  (70, 96, 32, 96), 'fluff':  (48, 96, 24, 96),
'fluid':  (70, 96, 34, 96), 'fluke':  (77, 96, 42, 96), 'flume':  (78, 96, 43, 96), 'flung':  (66, 96, 31, 96), 'flunk':  (66, 96, 34, 96),
'flush':  (70, 96, 35, 96), 'flute':  (83, 96, 46, 96), 'flyer':  (85, 96, 39, 96), 'foamy':  (77, 96, 50, 96), 'focal':  (81, 96, 34, 96),
'focus':  (73, 96, 25, 96), 'foggy':  (56, 96, 40, 96), 'foist':  (79, 96, 48, 96), 'folio':  (70, 96, 32, 96), 'folly':  (64, 96, 45, 96),
'foray':  (84, 96, 48, 96), 'force':  (84, 96, 50, 96), 'forge':  (82, 96, 47, 96), 'forgo':  (65, 96, 31, 96), 'forte':  (85, 96, 49, 96),
'forth':  (77, 96, 37, 96), 'forty':  (79, 96, 47, 96), 'forum':  (73, 96, 30, 96), 'found':  (69, 96, 38, 96), 'foyer':  (86, 96, 42, 96),
'frail':  (86, 96, 44, 96), 'frame':  (84, 96, 52, 96), 'frank':  (78, 96, 43, 96), 'fraud':  (80, 96, 39, 96), 'freak':  (84, 96, 37, 96),
'freed':  (72, 96, 44, 96), 'freer':  (67, 96, 48, 96), 'fresh':  (83, 96, 38, 96), 'friar':  (79, 96, 45, 96), 'fried':  (82, 96, 48, 96),
'frill':  (74, 96, 43, 96), 'frisk':  (76, 96, 41, 96), 'fritz':  (73, 96, 35, 96), 'frock':  (70, 96, 39, 96), 'frond':  (76, 96, 41, 96),
'front':  (80, 96, 47, 96), 'frost':  (81, 96, 46, 96), 'froth':  (77, 96, 40, 96), 'frown':  (73, 96, 35, 96), 'froze':  (79, 96, 47, 96),
'fruit':  (79, 96, 42, 96), 'fudge':  (74, 96, 39, 96), 'fugue':  (68, 96, 39, 96), 'fully':  (58, 96, 41, 96), 'fungi':  (65, 96, 24, 96),
'funky':  (60, 96, 38, 96), 'funny':  (56, 96, 43, 96), 'furor':  (68, 96, 36, 96), 'furry':  (64, 96, 43, 96), 'fussy':  (58, 96, 40, 96),
'fuzzy':  (42, 96, 31, 96), 'gaffe':  (77, 96, 39, 96), 'gaily':  (80, 96, 52, 96), 'gamer':  (85, 96, 44, 96), 'gamma':  (54, 96, 26, 96),
'gamut':  (76, 96, 35, 96), 'gassy':  (71, 96, 45, 96), 'gaudy':  (74, 96, 44, 96), 'gauge':  (84, 96, 47, 96), 'gaunt':  (80, 96, 44, 96),
'gauze':  (84, 96, 44, 96), 'gavel':  (82, 96, 41, 96), 'gawky':  (66, 96, 37, 96), 'gayer':  (88, 96, 42, 96), 'gayly':  (69, 96, 42, 96),
'gazer':  (83, 96, 41, 96), 'gecko':  (81, 96, 23, 96), 'geeky':  (70, 96, 41, 96), 'geese':  (67, 96, 49, 96), 'genie':  (75, 96, 47, 96),
'genre':  (77, 96, 46, 96), 'ghost':  (75, 96, 40, 96), 'ghoul':  (74, 96, 32, 96), 'giant':  (82, 96, 46, 96), 'giddy':  (56, 96, 36, 96),
'gipsy':  (70, 96, 39, 96), 'girly':  (83, 96, 43, 96), 'girth':  (80, 96, 33, 96), 'given':  (76, 96, 35, 96), 'giver':  (80, 96, 39, 96),
'glade':  (84, 96, 48, 96), 'gland':  (77, 96, 40, 96), 'glare':  (88, 96, 52, 96), 'glass':  (74, 96, 36, 96), 'glaze':  (81, 96, 46, 96),
'gleam':  (85, 96, 30, 96), 'glean':  (87, 96, 34, 96), 'glide':  (81, 96, 46, 96), 'glint':  (79, 96, 44, 96), 'gloat':  (85, 96, 42, 96),
'globe':  (82, 96, 44, 96), 'gloom':  (63, 96, 32, 96), 'glory':  (82, 96, 46, 96), 'gloss':  (72, 96, 33, 96), 'glove':  (80, 96, 44, 96),
'glyph':  (68, 96, 23, 96), 'gnash':  (78, 96, 35, 96), 'gnome':  (81, 96, 41, 96), 'godly':  (72, 96, 43, 96), 'going':  (67, 96, 38, 96),
'golem':  (83, 96, 37, 96), 'golly':  (66, 96, 45, 96), 'gonad':  (79, 96, 35, 96), 'goner':  (85, 96, 46, 96), 'goody':  (58, 96, 46, 96),
'gooey':  (79, 96, 57, 96), 'goofy':  (56, 96, 45, 96), 'goose':  (80, 96, 53, 96), 'gorge':  (79, 96, 46, 96), 'gouge':  (79, 96, 46, 96),
'gourd':  (75, 96, 36, 96), 'grace':  (87, 96, 55, 96), 'grade':  (85, 96, 51, 96), 'graft':  (80, 96, 42, 96), 'grail':  (87, 96, 43, 96),
'grain':  (86, 96, 42, 96), 'grand':  (81, 96, 43, 96), 'grant':  (85, 96, 49, 96), 'grape':  (86, 96, 50, 96), 'graph':  (80, 96, 38, 96),
'grasp':  (82, 96, 40, 96), 'grass':  (79, 96, 39, 96), 'grate':  (88, 96, 54, 96), 'grave':  (83, 96, 50, 96), 'gravy':  (77, 96, 47, 96),
'graze':  (83, 96, 49, 96), 'great':  (88, 96, 42, 96), 'greed':  (73, 96, 43, 96), 'green':  (77, 96, 44, 96), 'greet':  (79, 96, 49, 96),
'grief':  (82, 96, 43, 96), 'grill':  (77, 96, 42, 96), 'grime':  (83, 96, 49, 96), 'grimy':  (75, 96, 47, 96), 'grind':  (75, 96, 41, 96),
'gripe':  (83, 96, 49, 96), 'groan':  (85, 96, 40, 96), 'groin':  (80, 96, 39, 96), 'groom':  (66, 96, 35, 96), 'grope':  (83, 96, 48, 96),
'gross':  (74, 96, 36, 96), 'group':  (75, 96, 33, 96), 'grout':  (80, 96, 42, 96), 'grove':  (80, 96, 47, 96), 'growl':  (78, 96, 35, 96),
'grown':  (73, 96, 34, 96), 'gruel':  (86, 96, 44, 96), 'gruff':  (60, 96, 26, 96), 'grunt':  (80, 96, 42, 96), 'guard':  (81, 96, 38, 96),
'guava':  (64, 96, 31, 96), 'guess':  (75, 96, 30, 96), 'guest':  (83, 96, 39, 96), 'guide':  (81, 96, 46, 96), 'guild':  (71, 96, 37, 96),
'guile':  (85, 96, 50, 96), 'guilt':  (77, 96, 42, 96), 'guise':  (85, 96, 50, 96), 'gulch':  (69, 96, 30, 96), 'gully':  (60, 96, 41, 96),
'gumbo':  (62, 96, 19, 96), 'gummy':  (50, 96, 34, 96), 'guppy':  (52, 96, 33, 96), 'gusto':  (76, 96, 25, 96), 'gusty':  (72, 96, 38, 96),
'gypsy':  (58, 96, 32, 96), 'habit':  (80, 96, 36, 96), 'hairy':  (87, 96, 50, 96), 'halve':  (84, 96, 41, 96), 'handy':  (77, 96, 41, 96),
'happy':  (67, 96, 36, 96), 'hardy':  (82, 96, 42, 96), 'harem':  (87, 96, 39, 96), 'harpy':  (81, 96, 41, 96), 'harry':  (78, 96, 45, 96),
'harsh':  (80, 96, 37, 96), 'haste':  (88, 96, 44, 96), 'hasty':  (80, 96, 41, 96), 'hatch':  (70, 96, 33, 96), 'hater':  (89, 96, 44, 96),
'haunt':  (81, 96, 42, 96), 'haute':  (89, 96, 48, 96), 'haven':  (84, 96, 38, 96), 'havoc':  (76, 96, 25, 96), 'hazel':  (84, 96, 37, 96),
'heady':  (87, 96, 45, 96), 'heard':  (87, 96, 38, 96), 'heart':  (89, 96, 44, 97), 'heath':  (83, 96, 39, 96), 'heave':  (77, 96, 47, 96),
'heavy':  (84, 96, 44, 96), 'hedge':  (69, 96, 38, 96), 'hefty':  (81, 96, 36, 96), 'heist':  (85, 96, 43, 96), 'helix':  (81, 96, 25, 96),
'hello':  (82, 96, 28, 96), 'hence':  (75, 96, 44, 96), 'heron':  (88, 96, 32, 96), 'hilly':  (68, 96, 39, 96), 'hinge':  (81, 96, 39, 96),
'hippo':  (68, 96, 19, 96), 'hippy':  (60, 96, 32, 96), 'hitch':  (65, 96, 29, 96), 'hoard':  (85, 96, 40, 96), 'hobby':  (59, 96, 34, 96),
'hoist':  (80, 96, 45, 96), 'holly':  (69, 96, 43, 96), 'homer':  (85, 96, 41, 96), 'honey':  (88, 96, 50, 96), 'honor':  (74, 96, 36, 96),
'horde':  (84, 96, 43, 96), 'horny':  (81, 96, 46, 96), 'horse':  (88, 96, 48, 96), 'hotel':  (89, 96, 40, 96), 'hotly':  (79, 96, 43, 96),
'hound':  (72, 96, 35, 96), 'house':  (87, 96, 48, 96), 'hovel':  (83, 96, 38, 96), 'hover':  (82, 96, 40, 96), 'howdy':  (64, 96, 35, 96),
'human':  (77, 96, 26, 96), 'humid':  (67, 96, 26, 96), 'humor':  (77, 96, 28, 96), 'humph':  (52, 96, 22, 96), 'humus':  (57, 96, 19, 96),
'hunch':  (59, 96, 30, 96), 'hunky':  (65, 96, 35, 96), 'hurry':  (69, 96, 40, 96), 'husky':  (64, 96, 33, 96), 'hussy':  (61, 96, 38, 96),
'hutch':  (60, 96, 28, 96), 'hydro':  (77, 96, 16, 96), 'hyena':  (89, 96, 22, 96), 'hymen':  (82, 96, 26, 96), 'hyper':  (83, 96, 29, 96),
'icily':  (69, 96, 37, 96), 'icing':  (61, 96, 24, 96), 'ideal':  (89, 96, 24, 96), 'idiom':  (65, 96, 21, 96), 'idiot':  (72, 96, 30, 96),
'idler':  (88, 96, 30, 96), 'idyll':  (65, 96, 17, 96), 'igloo':  (72, 96, 15, 96), 'iliac':  (77, 96, 30, 96), 'image':  (87, 96, 38, 96),
'imbue':  (81, 96, 27, 96), 'impel':  (82, 96, 26, 96), 'imply':  (71, 96, 28, 96), 'inane':  (86, 96, 45, 96), 'inbox':  (68, 96, 14, 96),
'incur':  (81, 96, 20, 96), 'index':  (76, 96, 23, 96), 'inept':  (84, 96, 26, 96), 'inert':  (88, 96, 30, 96), 'infer':  (85, 96, 29, 96),
'ingot':  (78, 96, 25, 96), 'inlay':  (84, 96, 33, 96), 'inlet':  (88, 96, 35, 96), 'inner':  (83, 96, 34, 96), 'input':  (75, 96, 22, 96),
'inter':  (88, 96, 33, 96), 'intro':  (85, 96, 19, 96), 'ionic':  (71, 96, 28, 96), 'irate':  (92, 97, 51, 97), 'irony':  (84, 96, 47, 96),
'islet':  (88, 96, 32, 96), 'issue':  (83, 96, 27, 96), 'itchy':  (74, 96, 24, 96), 'ivory':  (79, 96, 35, 96), 'jaunt':  (78, 96, 40, 96),
'jazzy':  (52, 96, 31, 96), 'jelly':  (72, 96, 39, 96), 'jerky':  (78, 96, 36, 96), 'jetty':  (74, 96, 38, 96), 'jewel':  (66, 96, 33, 96),
'jiffy':  (48, 96, 28, 96), 'joint':  (74, 96, 43, 96), 'joist':  (77, 96, 43, 96), 'joker':  (80, 96, 36, 96), 'jolly':  (61, 96, 40, 96),
'joust':  (72, 96, 38, 96), 'judge':  (71, 96, 34, 96), 'juice':  (81, 96, 45, 96), 'juicy':  (67, 96, 43, 96), 'jumbo':  (57, 96, 15, 96),
'jumpy':  (52, 96, 29, 96), 'junta':  (78, 96, 24, 96), 'junto':  (71, 96, 23, 96), 'juror':  (65, 96, 31, 96), 'kappa':  (54, 96, 21, 96),
'karma':  (70, 96, 27, 96), 'kayak':  (55, 96, 27, 96), 'kebab':  (76, 96, 21, 96), 'khaki':  (72, 96, 23, 96), 'kinky':  (59, 96, 34, 96),
'kiosk':  (71, 96, 32, 96), 'kitty':  (65, 96, 36, 96), 'knack':  (66, 96, 29, 96), 'knave':  (81, 96, 38, 96), 'knead':  (84, 96, 24, 96),
'kneed':  (70, 96, 31, 96), 'kneel':  (76, 96, 33, 96), 'knelt':  (86, 96, 30, 96), 'knife':  (78, 96, 36, 96), 'knock':  (60, 96, 27, 96),
'knoll':  (67, 96, 29, 96), 'known':  (54, 96, 22, 96), 'koala':  (75, 96, 36, 96), 'krill':  (76, 96, 38, 96), 'label':  (80, 96, 40, 96),
'labor':  (86, 96, 34, 96), 'laden':  (87, 96, 39, 96), 'ladle':  (81, 96, 45, 96), 'lager':  (88, 96, 43, 96), 'lance':  (88, 96, 48, 96),
'lanky':  (76, 96, 41, 96), 'lapel':  (82, 96, 40, 96), 'lapse':  (88, 96, 45, 96), 'large':  (88, 96, 46, 96), 'larva':  (76, 96, 29, 96),
'lasso':  (81, 96, 30, 96), 'latch':  (80, 96, 34, 96), 'later':  (92, 96, 45, 96), 'lathe':  (89, 96, 41, 96), 'latte':  (86, 96, 46, 96),
'laugh':  (78, 96, 33, 96), 'layer':  (91, 96, 41, 96), 'leach':  (87, 96, 40, 96), 'leafy':  (85, 96, 45, 96), 'leaky':  (86, 96, 46, 96),
'leant':  (91, 96, 46, 96), 'leapt':  (89, 96, 41, 96), 'learn':  (91, 96, 40, 96), 'lease':  (85, 96, 53, 96), 'leash':  (90, 96, 41, 96),
'least':  (90, 96, 46, 96), 'leave':  (78, 96, 48, 96), 'ledge':  (73, 96, 39, 96), 'leech':  (76, 96, 34, 96), 'leery':  (84, 96, 44, 96),
'lefty':  (84, 96, 37, 96), 'legal':  (81, 96, 31, 96), 'leggy':  (76, 96, 36, 96), 'lemon':  (87, 96, 28, 96), 'lemur':  (86, 96, 30, 96),
'leper':  (81, 96, 40, 96), 'level':  (63, 96, 37, 96), 'lever':  (77, 96, 39, 96), 'libel':  (78, 96, 35, 96), 'liege':  (78, 96, 42, 96),
'light':  (77, 96, 28, 96), 'liken':  (83, 96, 32, 96), 'lilac':  (77, 96, 26, 96), 'limbo':  (75, 96, 19, 96), 'limit':  (71, 96, 33, 96),
'linen':  (81, 96, 38, 96), 'liner':  (90, 96, 41, 96), 'lingo':  (78, 96, 24, 96), 'lipid':  (64, 96, 27, 96), 'lithe':  (87, 96, 37, 96),
'liver':  (86, 96, 38, 96), 'livid':  (60, 96, 27, 96), 'llama':  (61, 96, 32, 96), 'loamy':  (81, 96, 48, 96), 'loath':  (85, 96, 41, 96),
'lobby':  (65, 96, 35, 96), 'local':  (78, 96, 32, 96), 'locus':  (80, 96, 23, 96), 'lodge':  (83, 96, 41, 96), 'lofty':  (77, 96, 39, 96),
'logic':  (79, 96, 27, 96), 'login':  (78, 96, 31, 96), 'loopy':  (67, 96, 44, 96), 'loose':  (85, 96, 52, 96), 'lorry':  (79, 96, 45, 96),
'loser':  (91, 96, 42, 96), 'louse':  (89, 96, 49, 96), 'lousy':  (79, 96, 46, 96), 'lover':  (86, 96, 41, 96), 'lower':  (87, 96, 40, 96),
'lowly':  (64, 96, 40, 96), 'loyal':  (78, 96, 31, 96), 'lucid':  (75, 96, 26, 96), 'lucky':  (68, 96, 32, 96), 'lumen':  (84, 96, 34, 96),
'lumpy':  (67, 96, 32, 96), 'lunar':  (89, 96, 34, 96), 'lunch':  (74, 96, 30, 96), 'lunge':  (82, 96, 39, 96), 'lupus':  (65, 96, 19, 96),
'lurch':  (81, 96, 31, 96), 'lurid':  (84, 96, 31, 96), 'lusty':  (78, 96, 37, 96), 'lying':  (73, 96, 26, 96), 'lymph':  (67, 96, 16, 96),
'lynch':  (75, 96, 23, 96), 'lyric':  (86, 96, 20, 96), 'macaw':  (62, 96, 28, 96), 'macho':  (77, 96, 24, 96), 'macro':  (83, 96, 29, 96),
'madam':  (56, 96, 30, 96), 'madly':  (74, 96, 44, 96), 'mafia':  (69, 96, 28, 96), 'magic':  (77, 96, 29, 96), 'magma':  (54, 96, 26, 96),
'maize':  (85, 96, 48, 96), 'major':  (79, 96, 33, 96), 'maker':  (85, 96, 41, 96), 'mambo':  (70, 96, 24, 96), 'mamma':  (47, 96, 26, 96),
'mammy':  (57, 96, 39, 96), 'manga':  (66, 96, 30, 96), 'mange':  (84, 96, 45, 96), 'mango':  (78, 96, 30, 96), 'mangy':  (73, 96, 43, 96),
'mania':  (74, 96, 33, 96), 'manic':  (79, 96, 32, 96), 'manly':  (78, 96, 46, 96), 'manor':  (86, 96, 39, 96), 'maple':  (84, 96, 46, 96),
'march':  (79, 96, 37, 96), 'marry':  (74, 96, 47, 96), 'marsh':  (83, 96, 38, 96), 'mason':  (83, 96, 32, 96), 'masse':  (82, 96, 47, 96),
'match':  (75, 96, 35, 96), 'matey':  (89, 96, 52, 96), 'mauve':  (83, 96, 45, 96), 'maxim':  (67, 96, 27, 96), 'maybe':  (84, 96, 38, 96),
'mayor':  (84, 96, 34, 96), 'mealy':  (86, 96, 51, 96), 'meant':  (88, 96, 47, 96), 'meaty':  (89, 96, 50, 96), 'mecca':  (80, 96, 27, 96),
'medal':  (84, 96, 32, 96), 'media':  (87, 96, 28, 96), 'medic':  (81, 96, 26, 96), 'melee':  (68, 96, 52, 96), 'melon':  (87, 96, 31, 96),
'mercy':  (85, 96, 44, 96), 'merge':  (74, 96, 44, 96), 'merit':  (87, 96, 40, 96), 'merry':  (78, 96, 44, 96), 'metal':  (89, 96, 34, 96),
'meter':  (80, 96, 43, 96), 'metro':  (87, 96, 29, 96), 'micro':  (80, 96, 25, 96), 'midge':  (77, 96, 38, 96), 'midst':  (76, 96, 35, 96),
'might':  (70, 96, 28, 96), 'milky':  (69, 96, 36, 96), 'mimic':  (50, 96, 24, 96), 'mince':  (83, 96, 44, 96), 'miner':  (86, 96, 42, 96),
'minim':  (52, 96, 28, 96), 'minor':  (82, 96, 34, 96), 'minty':  (75, 96, 41, 96), 'minus':  (74, 96, 24, 96), 'mirth':  (80, 96, 32, 96),
'miser':  (88, 96, 40, 96), 'missy':  (65, 96, 40, 96), 'mocha':  (77, 96, 23, 96), 'modal':  (81, 96, 34, 96), 'model':  (84, 96, 40, 96),
'modem':  (75, 96, 35, 96), 'mogul':  (71, 96, 30, 96), 'moist':  (81, 96, 46, 96), 'molar':  (87, 96, 38, 96), 'moldy':  (72, 96, 40, 96),
'money':  (86, 96, 52, 96), 'month':  (74, 96, 35, 96), 'moody':  (58, 96, 46, 96), 'moose':  (80, 96, 53, 96), 'moral':  (87, 96, 37, 96),
'moron':  (72, 96, 35, 96), 'morph':  (74, 96, 32, 96), 'mossy':  (65, 96, 43, 96), 'motel':  (88, 96, 42, 96), 'motif':  (75, 96, 29, 96),
'motor':  (74, 96, 36, 96), 'motto':  (57, 96, 30, 96), 'moult':  (79, 96, 42, 96), 'mound':  (70, 96, 37, 96), 'mount':  (76, 96, 43, 96),
'mourn':  (78, 96, 36, 96), 'mouse':  (86, 96, 50, 96), 'mouth':  (72, 96, 36, 96), 'mover':  (81, 96, 42, 96), 'movie':  (84, 96, 44, 96),
'mower':  (82, 96, 41, 96), 'mucky':  (59, 96, 33, 96), 'mucus':  (60, 96, 20, 96), 'muddy':  (52, 96, 35, 96), 'mulch':  (68, 96, 30, 96),
'mummy':  (43, 96, 34, 96), 'munch':  (65, 96, 31, 96), 'mural':  (85, 96, 33, 96), 'murky':  (70, 96, 38, 96), 'mushy':  (67, 96, 33, 96),
'music':  (74, 96, 24, 96), 'musky':  (63, 96, 34, 96), 'musty':  (72, 96, 38, 96), 'myrrh':  (66, 96, 25, 96), 'nadir':  (86, 96, 34, 96),
'naive':  (86, 96, 47, 96), 'nanny':  (64, 96, 44, 96), 'nasal':  (78, 96, 32, 96), 'nasty':  (84, 96, 40, 96), 'natal':  (80, 96, 33, 96),
'naval':  (70, 96, 31, 96), 'navel':  (85, 96, 37, 96), 'needy':  (75, 96, 38, 96), 'neigh':  (81, 96, 33, 96), 'nerdy':  (85, 96, 38, 96),
'nerve':  (76, 96, 39, 96), 'never':  (76, 96, 37, 96), 'newer':  (76, 96, 36, 96), 'newly':  (82, 96, 36, 96), 'nicer':  (87, 96, 36, 96),
'niche':  (83, 96, 32, 96), 'niece':  (79, 96, 43, 96), 'night':  (73, 96, 25, 96), 'ninja':  (70, 96, 19, 96), 'ninny':  (55, 96, 40, 96),
'ninth':  (68, 96, 28, 96), 'noble':  (85, 96, 41, 96), 'nobly':  (75, 96, 39, 96), 'noise':  (91, 96, 51, 96), 'noisy':  (81, 96, 48, 96),
'nomad':  (80, 96, 28, 96), 'noose':  (84, 96, 50, 96), 'north':  (83, 96, 33, 96), 'nosey':  (89, 96, 47, 96), 'notch':  (75, 96, 31, 96),
'novel':  (84, 96, 36, 96), 'nudge':  (77, 96, 34, 96), 'nurse':  (88, 96, 42, 96), 'nutty':  (67, 96, 36, 96), 'nylon':  (71, 96, 19, 96),
'nymph':  (65, 96, 13, 96), 'oaken':  (90, 96, 35, 96), 'obese':  (80, 96, 36, 96), 'occur':  (72, 96, 19, 96), 'ocean':  (91, 96, 24, 96),
'octal':  (86, 96, 22, 96), 'octet':  (83, 96, 33, 96), 'odder':  (79, 96, 29, 96), 'oddly':  (67, 96, 29, 96), 'offal':  (75, 96, 17, 96),
'offer':  (79, 96, 26, 96), 'often':  (84, 96, 26, 96), 'olden':  (86, 96, 33, 96), 'older':  (87, 96, 37, 96), 'olive':  (87, 96, 42, 96),
'ombre':  (82, 96, 31, 96), 'omega':  (89, 96, 17, 96), 'onion':  (62, 96, 28, 96), 'onset':  (89, 96, 34, 96), 'opera':  (90, 96, 21, 96),
'opine':  (88, 96, 42, 96), 'opium':  (75, 96, 21, 96), 'optic':  (79, 96, 17, 96), 'orbit':  (83, 96, 34, 96), 'order':  (79, 96, 39, 96),
'organ':  (85, 96, 29, 96), 'other':  (87, 96, 28, 96), 'otter':  (84, 96, 33, 96), 'ought':  (73, 96, 25, 96), 'ounce':  (86, 96, 41, 96),
'outdo':  (69, 96, 20, 96), 'outer':  (89, 96, 37, 96), 'outgo':  (67, 96, 20, 96), 'ovary':  (83, 96, 38, 96), 'ovate':  (89, 96, 40, 96),
'overt':  (84, 96, 28, 96), 'ovine':  (85, 96, 40, 96), 'ovoid':  (62, 96, 25, 96), 'owing':  (69, 96, 25, 96), 'owner':  (85, 96, 33, 96),
'oxide':  (83, 96, 35, 96), 'ozone':  (75, 96, 39, 96), 'paddy':  (66, 96, 41, 96), 'pagan':  (68, 96, 35, 96), 'paint':  (83, 96, 50, 96),
'paler':  (89, 96, 47, 96), 'palsy':  (79, 96, 47, 96), 'panel':  (88, 96, 46, 96), 'panic':  (80, 96, 33, 96), 'pansy':  (79, 96, 48, 96),
'papal':  (63, 96, 36, 96), 'paper':  (83, 96, 45, 96), 'parer':  (83, 96, 49, 96), 'parka':  (71, 96, 32, 96), 'parry':  (75, 96, 49, 96),
'parse':  (88, 96, 52, 96), 'party':  (84, 96, 48, 96), 'pasta':  (73, 96, 32, 96), 'paste':  (88, 96, 47, 96), 'pasty':  (80, 96, 44, 96),
'patch':  (76, 96, 37, 96), 'patio':  (89, 96, 33, 96), 'patsy':  (80, 96, 47, 96), 'patty':  (73, 96, 46, 96), 'pause':  (88, 96, 52, 96),
'payee':  (82, 96, 53, 96), 'payer':  (88, 96, 43, 96), 'peace':  (80, 96, 55, 96), 'peach':  (84, 96, 42, 96), 'pearl':  (89, 96, 43, 96),
'pecan':  (86, 96, 32, 96), 'pedal':  (85, 96, 34, 96), 'penal':  (88, 96, 36, 96), 'pence':  (75, 96, 47, 96), 'penne':  (67, 96, 49, 96),
'penny':  (76, 96, 46, 96), 'perch':  (81, 96, 36, 96), 'peril':  (88, 96, 37, 96), 'perky':  (81, 96, 42, 96), 'pesky':  (78, 96, 38, 96),
'pesto':  (87, 96, 29, 96), 'petal':  (89, 96, 35, 96), 'petty':  (78, 96, 43, 96), 'phase':  (86, 96, 51, 96), 'phone':  (85, 96, 49, 96),
'phony':  (72, 96, 46, 96), 'photo':  (66, 96, 31, 96), 'piano':  (88, 96, 38, 96), 'picky':  (65, 96, 35, 96), 'piece':  (77, 96, 47, 96),
'piety':  (86, 96, 44, 96), 'piggy':  (58, 96, 37, 96), 'pilot':  (83, 96, 36, 96), 'pinch':  (70, 96, 33, 96), 'piney':  (84, 96, 50, 96),
'pinky':  (66, 96, 39, 96), 'pinto':  (79, 96, 29, 96), 'piper':  (80, 96, 40, 96), 'pique':  (79, 96, 37, 96), 'pitch':  (71, 96, 32, 96),
'pithy':  (73, 96, 37, 96), 'pivot':  (75, 96, 34, 96), 'pixel':  (79, 96, 36, 96), 'pixie':  (69, 96, 41, 96), 'pizza':  (68, 96, 19, 96),
'place':  (86, 96, 53, 96), 'plaid':  (81, 96, 40, 96), 'plain':  (83, 96, 41, 96), 'plait':  (84, 96, 46, 96), 'plane':  (88, 96, 54, 96),
'plank':  (77, 96, 41, 96), 'plant':  (85, 96, 47, 96), 'plate':  (89, 96, 52, 97), 'plaza':  (63, 96, 32, 96), 'plead':  (85, 96, 35, 96),
'pleat':  (89, 96, 40, 96), 'plied':  (82, 96, 45, 96), 'plier':  (88, 96, 49, 96), 'pluck':  (65, 96, 33, 96), 'plumb':  (64, 96, 25, 96),
'plume':  (80, 96, 43, 96), 'plump':  (59, 96, 27, 96), 'plunk':  (69, 96, 35, 96), 'plush':  (72, 96, 35, 96), 'poesy':  (86, 96, 49, 96),
'point':  (79, 96, 49, 96), 'poise':  (90, 96, 55, 96), 'poker':  (83, 96, 42, 96), 'polar':  (87, 96, 39, 96), 'polka':  (80, 96, 28, 96),
'polyp':  (67, 96, 26, 96), 'pooch':  (60, 96, 41, 96), 'poppy':  (52, 96, 39, 96), 'porch':  (75, 96, 38, 96), 'poser':  (87, 96, 45, 96),
'posit':  (81, 96, 39, 96), 'posse':  (81, 96, 47, 96), 'pouch':  (69, 96, 38, 96), 'pound':  (71, 96, 38, 96), 'pouty':  (75, 96, 47, 96),
'power':  (82, 96, 42, 96), 'prank':  (81, 96, 44, 96), 'prawn':  (80, 96, 38, 96), 'preen':  (79, 96, 45, 96), 'press':  (79, 96, 34, 96),
'price':  (85, 96, 54, 96), 'prick':  (74, 96, 41, 96), 'pride':  (84, 96, 51, 96), 'pried':  (84, 96, 48, 96), 'prime':  (83, 96, 51, 96),
'primo':  (79, 96, 35, 96), 'primp':  (68, 96, 35, 96), 'print':  (82, 96, 48, 96), 'prior':  (76, 96, 44, 96), 'prism':  (79, 96, 38, 96),
'privy':  (74, 96, 47, 96), 'prize':  (81, 96, 48, 96), 'probe':  (83, 96, 48, 96), 'prone':  (87, 96, 54, 96), 'prong':  (77, 96, 38, 96),
'proof':  (66, 96, 35, 96), 'prose':  (87, 96, 54, 96), 'proud':  (76, 96, 37, 96), 'prove':  (81, 96, 49, 96), 'prowl':  (79, 96, 36, 96),
'proxy':  (71, 96, 44, 96), 'prude':  (81, 96, 46, 96), 'prune':  (85, 96, 51, 96), 'psalm':  (78, 96, 29, 96), 'pubic':  (69, 96, 25, 96),
'pudgy':  (60, 96, 36, 96), 'puffy':  (50, 96, 32, 96), 'pulpy':  (61, 96, 37, 96), 'pulse':  (83, 96, 45, 96), 'punch':  (67, 96, 33, 96),
'pupal':  (73, 96, 30, 96), 'pupil':  (68, 96, 30, 96), 'puppy':  (44, 96, 35, 96), 'puree':  (78, 96, 53, 96), 'purer':  (78, 96, 44, 96),
'purge':  (81, 96, 43, 96), 'purse':  (85, 96, 47, 96), 'pushy':  (66, 96, 35, 96), 'putty':  (62, 96, 41, 96), 'pygmy':  (48, 96, 29, 96),
'quack':  (66, 96, 34, 96), 'quail':  (82, 96, 36, 96), 'quake':  (83, 96, 43, 96), 'qualm':  (71, 96, 31, 96), 'quark':  (75, 96, 34, 96),
'quart':  (81, 96, 40, 96), 'quash':  (74, 96, 36, 96), 'quasi':  (82, 96, 30, 96), 'queen':  (70, 96, 36, 96), 'queer':  (73, 96, 40, 96),
'quell':  (71, 96, 30, 96), 'query':  (82, 96, 39, 96), 'quest':  (79, 96, 35, 96), 'queue':  (59, 96, 38, 96), 'quick':  (60, 96, 32, 96),
'quiet':  (82, 96, 45, 96), 'quill':  (62, 96, 34, 96), 'quilt':  (74, 96, 38, 96), 'quirk':  (71, 96, 32, 96), 'quite':  (82, 96, 45, 96),
'quota':  (82, 96, 28, 96), 'quote':  (84, 96, 44, 96), 'quoth':  (68, 96, 32, 96), 'rabbi':  (80, 96, 22, 96), 'rabid':  (84, 96, 32, 96),
'racer':  (84, 96, 43, 96), 'radar':  (69, 96, 37, 96), 'radii':  (81, 96, 28, 96), 'radio':  (90, 96, 30, 96), 'rainy':  (88, 96, 53, 96),
'raise':  (93, 97, 55, 97), 'rajah':  (71, 96, 31, 96), 'rally':  (81, 96, 45, 96), 'ralph':  (85, 96, 31, 96), 'ramen':  (88, 96, 40, 96),
'ranch':  (83, 96, 36, 96), 'randy':  (84, 96, 42, 96), 'range':  (87, 96, 45, 96), 'rapid':  (85, 96, 32, 96), 'rarer':  (79, 96, 48, 96),
'raspy':  (83, 96, 39, 96), 'ratio':  (92, 96, 32, 96), 'ratty':  (81, 96, 44, 96), 'raven':  (86, 96, 39, 96), 'rayon':  (88, 96, 30, 96),
'razor':  (76, 96, 33, 96), 'reach':  (87, 96, 41, 96), 'react':  (89, 97, 46, 97), 'ready':  (88, 96, 47, 96), 'realm':  (88, 96, 37, 96),
'rearm':  (82, 96, 37, 96), 'rebar':  (81, 96, 34, 96), 'rebel':  (79, 96, 38, 96), 'rebus':  (84, 96, 23, 96), 'rebut':  (84, 96, 32, 96),
'recap':  (87, 96, 27, 96), 'recur':  (80, 96, 30, 96), 'recut':  (87, 96, 32, 96), 'reedy':  (77, 96, 41, 96), 'refer':  (67, 96, 39, 96),
'refit':  (85, 96, 34, 96), 'regal':  (88, 96, 32, 96), 'rehab':  (86, 96, 23, 96), 'reign':  (85, 96, 35, 96), 'relax':  (86, 96, 27, 96),
'relay':  (91, 96, 43, 96), 'relic':  (89, 96, 28, 96), 'remit':  (87, 96, 35, 96), 'renal':  (91, 96, 35, 96), 'renew':  (76, 96, 35, 96),
'repay':  (88, 96, 40, 96), 'repel':  (81, 96, 38, 96), 'reply':  (86, 96, 40, 96), 'rerun':  (81, 96, 31, 96), 'reset':  (83, 96, 43, 96),
'resin':  (89, 96, 31, 96), 'retch':  (84, 96, 32, 96), 'retro':  (84, 96, 29, 96), 'retry':  (84, 96, 42, 96), 'reuse':  (82, 96, 48, 96),
'revel':  (77, 96, 38, 96), 'revue':  (75, 96, 39, 96), 'rhino':  (83, 96, 33, 96), 'rhyme':  (84, 96, 33, 96), 'rider':  (80, 96, 39, 96),
'ridge':  (83, 96, 38, 96), 'rifle':  (87, 96, 40, 96), 'right':  (80, 96, 28, 96), 'rigid':  (67, 96, 28, 96), 'rigor':  (74, 96, 31, 96),
'rinse':  (89, 96, 45, 96), 'ripen':  (86, 96, 35, 96), 'riper':  (80, 96, 39, 96), 'risen':  (89, 96, 36, 96), 'riser':  (85, 96, 40, 96),
'risky':  (80, 96, 35, 96), 'rival':  (86, 96, 29, 96), 'river':  (77, 96, 38, 96), 'rivet':  (84, 96, 40, 96), 'roach':  (84, 96, 42, 96),
'roast':  (89, 96, 48, 96), 'robin':  (81, 96, 31, 96), 'robot':  (73, 96, 36, 96), 'rocky':  (75, 96, 37, 96), 'rodeo':  (79, 96, 36, 96),
'roger':  (79, 96, 42, 96), 'rogue':  (86, 96, 41, 96), 'roomy':  (69, 96, 46, 96), 'roost':  (79, 96, 45, 96), 'rotor':  (69, 96, 36, 96),
'rouge':  (86, 96, 45, 96), 'rough':  (76, 96, 33, 96), 'round':  (79, 96, 37, 96), 'rouse':  (89, 96, 49, 96), 'route':  (89, 96, 48, 96),
'rover':  (77, 96, 42, 96), 'rowdy':  (72, 96, 36, 96), 'rower':  (78, 96, 41, 96), 'royal':  (89, 96, 32, 96), 'ruddy':  (67, 96, 34, 96),
'ruder':  (77, 96, 39, 96), 'rugby':  (70, 96, 32, 96), 'ruler':  (83, 96, 40, 96), 'rumba':  (79, 96, 19, 96), 'rumor':  (70, 96, 30, 96),
'rupee':  (78, 96, 47, 96), 'rural':  (82, 96, 33, 96), 'rusty':  (82, 96, 38, 96), 'sadly':  (81, 96, 55, 96), 'safer':  (88, 96, 53, 96),
'saint':  (85, 96, 59, 96), 'salad':  (76, 96, 46, 96), 'sally':  (75, 96, 56, 96), 'salon':  (87, 96, 45, 96), 'salsa':  (69, 96, 44, 96),
'salty':  (84, 96, 55, 96), 'salve':  (86, 96, 54, 96), 'salvo':  (82, 96, 38, 96), 'sandy':  (81, 96, 54, 96), 'saner':  (91, 96, 58, 96),
'sappy':  (70, 96, 49, 96), 'sassy':  (66, 96, 55, 96), 'satin':  (85, 96, 46, 96), 'satyr':  (87, 96, 43, 96), 'sauce':  (89, 96, 61, 96),
'saucy':  (81, 96, 58, 96), 'sauna':  (77, 96, 47, 96), 'saute':  (90, 96, 60, 96), 'savor':  (84, 96, 46, 96), 'savoy':  (81, 96, 52, 96),
'savvy':  (68, 96, 49, 96), 'scald':  (82, 96, 43, 96), 'scale':  (89, 96, 56, 96), 'scalp':  (80, 96, 40, 96), 'scaly':  (82, 96, 54, 96),
'scamp':  (75, 96, 36, 96), 'scant':  (83, 96, 50, 96), 'scare':  (90, 97, 56, 97), 'scarf':  (83, 96, 38, 96), 'scary':  (86, 96, 53, 96),
'scene':  (80, 96, 51, 96), 'scent':  (86, 96, 44, 96), 'scion':  (81, 96, 40, 96), 'scoff':  (65, 96, 31, 96), 'scold':  (81, 96, 40, 96),
'scone':  (88, 96, 54, 96), 'scoop':  (67, 96, 36, 96), 'scope':  (86, 96, 49, 96), 'score':  (89, 96, 53, 96), 'scorn':  (84, 96, 40, 96),
'scour':  (82, 96, 41, 96), 'scout':  (78, 96, 43, 96), 'scowl':  (77, 96, 36, 96), 'scram':  (83, 96, 33, 96), 'scrap':  (83, 96, 34, 96),
'scree':  (82, 96, 57, 96), 'screw':  (83, 96, 39, 96), 'scrub':  (78, 96, 29, 96), 'scrum':  (78, 96, 30, 96), 'scuba':  (78, 96, 29, 96),
'sedan':  (88, 96, 42, 96), 'seedy':  (76, 96, 53, 96), 'segue':  (75, 96, 51, 96), 'seize':  (75, 96, 57, 96), 'semen':  (78, 96, 48, 96),
'sense':  (72, 96, 58, 96), 'sepia':  (90, 96, 38, 96), 'serif':  (86, 96, 41, 96), 'serum':  (85, 96, 39, 96), 'serve':  (76, 96, 54, 96),
'setup':  (82, 96, 37, 96), 'seven':  (74, 96, 48, 96), 'sever':  (76, 96, 51, 96), 'sewer':  (77, 96, 50, 96), 'shack':  (72, 96, 47, 96),
'shade':  (88, 96, 57, 96), 'shady':  (79, 96, 54, 96), 'shaft':  (76, 96, 48, 96), 'shake':  (86, 96, 56, 96), 'shaky':  (74, 96, 53, 96),
'shale':  (90, 96, 61, 96), 'shall':  (75, 96, 49, 96), 'shalt':  (82, 96, 53, 96), 'shame':  (87, 96, 57, 96), 'shank':  (76, 96, 48, 96),
'shape':  (86, 96, 56, 96), 'shard':  (85, 96, 47, 96), 'share':  (90, 96, 60, 96), 'shark':  (81, 96, 47, 96), 'sharp':  (83, 96, 44, 96),
'shave':  (84, 96, 56, 96), 'shawl':  (78, 96, 43, 96), 'shear':  (90, 96, 46, 96), 'sheen':  (78, 96, 49, 96), 'sheep':  (73, 96, 46, 96),
'sheer':  (80, 96, 53, 96), 'sheet':  (77, 96, 54, 96), 'sheik':  (82, 96, 41, 96), 'shelf':  (82, 96, 38, 96), 'shell':  (80, 96, 43, 96),
'shied':  (84, 96, 52, 96), 'shift':  (71, 96, 46, 96), 'shine':  (86, 96, 60, 96), 'shiny':  (75, 96, 57, 96), 'shire':  (88, 96, 58, 96),
'shirk':  (79, 96, 45, 96), 'shirt':  (82, 96, 51, 96), 'shoal':  (85, 96, 46, 96), 'shock':  (68, 96, 44, 96), 'shone':  (87, 96, 59, 96),
'shook':  (61, 96, 43, 96), 'shoot':  (68, 96, 49, 96), 'shore':  (88, 96, 57, 96), 'shorn':  (83, 96, 45, 96), 'short':  (82, 96, 50, 96),
'shout':  (75, 96, 47, 96), 'shove':  (83, 96, 53, 96), 'shown':  (70, 96, 39, 96), 'showy':  (69, 96, 49, 96), 'shrew':  (82, 96, 44, 96),
'shrub':  (76, 96, 33, 96), 'shrug':  (77, 96, 34, 96), 'shuck':  (62, 96, 41, 96), 'shunt':  (73, 96, 48, 96), 'shush':  (50, 96, 43, 96),
'shyly':  (68, 96, 46, 96), 'siege':  (79, 96, 54, 96), 'sieve':  (76, 96, 52, 96), 'sight':  (73, 96, 40, 96), 'sigma':  (80, 96, 33, 96),
'silky':  (75, 96, 47, 96), 'silly':  (73, 96, 52, 96), 'since':  (86, 96, 55, 96), 'sinew':  (83, 96, 45, 96), 'singe':  (84, 96, 52, 96),
'siren':  (89, 96, 51, 96), 'sissy':  (59, 96, 51, 96), 'sixth':  (68, 96, 37, 96), 'sixty':  (72, 96, 47, 96), 'skate':  (87, 96, 54, 96),
'skier':  (87, 96, 51, 96), 'skiff':  (57, 96, 30, 96), 'skill':  (68, 96, 42, 96), 'skimp':  (66, 96, 33, 96), 'skirt':  (81, 96, 45, 96),
'skulk':  (63, 96, 35, 96), 'skull':  (63, 96, 37, 96), 'skunk':  (59, 96, 36, 96), 'slack':  (77, 96, 49, 96), 'slain':  (86, 96, 50, 96),
'slang':  (82, 96, 47, 96), 'slant':  (86, 96, 56, 96), 'slash':  (75, 96, 51, 96), 'slate':  (90, 97, 62, 97), 'slave':  (86, 96, 58, 96),
'sleek':  (76, 96, 51, 96), 'sleep':  (77, 96, 48, 96), 'sleet':  (82, 96, 57, 96), 'slept':  (85, 96, 45, 96), 'slice':  (87, 96, 61, 96),
'slick':  (75, 96, 47, 96), 'slide':  (86, 96, 57, 96), 'slime':  (86, 96, 57, 96), 'slimy':  (77, 96, 55, 96), 'sling':  (78, 96, 46, 96),
'slink':  (76, 96, 49, 96), 'sloop':  (71, 96, 43, 96), 'slope':  (87, 96, 55, 96), 'slosh':  (73, 96, 48, 96), 'sloth':  (81, 96, 47, 96),
'slump':  (70, 96, 37, 96), 'slung':  (75, 96, 41, 96), 'slunk':  (74, 96, 44, 96), 'slurp':  (83, 96, 41, 96), 'slush':  (67, 96, 45, 96),
'slyly':  (59, 96, 48, 96), 'smack':  (72, 96, 42, 96), 'small':  (73, 96, 44, 96), 'smart':  (85, 96, 48, 96), 'smash':  (69, 96, 44, 96),
'smear':  (88, 96, 41, 96), 'smell':  (78, 96, 39, 96), 'smelt':  (86, 96, 43, 96), 'smile':  (86, 96, 54, 96), 'smirk':  (78, 96, 40, 96),
'smite':  (86, 96, 53, 96), 'smith':  (73, 96, 41, 96), 'smock':  (69, 96, 39, 96), 'smoke':  (83, 96, 49, 96), 'smoky':  (68, 96, 46, 96),
'smote':  (87, 96, 52, 96), 'snack':  (76, 96, 44, 96), 'snail':  (86, 96, 46, 96), 'snake':  (86, 96, 53, 96), 'snaky':  (77, 96, 51, 96),
'snare':  (91, 97, 58, 97), 'snarl':  (90, 96, 46, 96), 'sneak':  (86, 96, 39, 96), 'sneer':  (83, 96, 50, 96), 'snide':  (85, 96, 52, 96),
'sniff':  (65, 96, 34, 96), 'snipe':  (85, 96, 52, 96), 'snoop':  (68, 96, 38, 96), 'snore':  (90, 96, 55, 96), 'snort':  (86, 96, 48, 96),
'snout':  (79, 96, 45, 96), 'snowy':  (73, 96, 47, 96), 'snuck':  (68, 96, 38, 96), 'snuff':  (61, 96, 29, 96), 'soapy':  (82, 96, 59, 96),
'sober':  (86, 96, 53, 96), 'soggy':  (66, 96, 50, 96), 'solar':  (90, 96, 49, 96), 'solid':  (83, 96, 45, 96), 'solve':  (85, 96, 53, 96),
'sonar':  (89, 96, 50, 96), 'sonic':  (81, 96, 42, 96), 'sooth':  (68, 96, 50, 96), 'sooty':  (72, 96, 60, 96), 'sorry':  (76, 96, 57, 96),
'sound':  (76, 96, 48, 96), 'south':  (75, 96, 47, 96), 'sower':  (85, 96, 52, 96), 'space':  (87, 96, 57, 96), 'spade':  (86, 96, 53, 96),
'spank':  (75, 96, 44, 96), 'spare':  (88, 96, 57, 96), 'spark':  (80, 96, 43, 96), 'spasm':  (67, 96, 41, 96), 'spawn':  (76, 96, 38, 96),
'speak':  (84, 96, 38, 96), 'spear':  (88, 96, 42, 96), 'speck':  (76, 96, 38, 96), 'speed':  (73, 96, 45, 96), 'spell':  (77, 96, 40, 96),
'spelt':  (85, 96, 44, 96), 'spend':  (82, 96, 39, 96), 'spent':  (84, 96, 45, 96), 'sperm':  (82, 96, 34, 96), 'spice':  (84, 96, 55, 96),
'spicy':  (73, 96, 52, 96), 'spied':  (83, 96, 49, 96), 'spiel':  (85, 96, 50, 96), 'spike':  (81, 96, 51, 96), 'spiky':  (68, 96, 48, 96),
'spill':  (69, 96, 44, 96), 'spilt':  (79, 96, 48, 96), 'spine':  (85, 96, 56, 96), 'spiny':  (74, 96, 54, 96), 'spire':  (87, 96, 55, 96),
'spite':  (85, 96, 54, 96), 'splat':  (82, 96, 41, 96), 'split':  (79, 96, 41, 96), 'spoil':  (82, 96, 43, 96), 'spoke':  (83, 96, 50, 96),
'spoof':  (62, 96, 36, 96), 'spook':  (61, 96, 40, 96), 'spool':  (71, 96, 41, 96), 'spoon':  (68, 96, 40, 96), 'spore':  (87, 96, 54, 96),
'sport':  (82, 96, 46, 96), 'spout':  (76, 96, 43, 96), 'spray':  (83, 96, 48, 96), 'spree':  (79, 96, 57, 96), 'sprig':  (79, 96, 34, 96),
'spunk':  (66, 96, 38, 96), 'spurn':  (80, 96, 38, 96), 'spurt':  (79, 96, 43, 96), 'squad':  (75, 96, 35, 96), 'squat':  (77, 96, 41, 96),
'squib':  (66, 96, 30, 96), 'stack':  (77, 96, 44, 96), 'staff':  (72, 96, 35, 96), 'stage':  (88, 96, 54, 96), 'staid':  (85, 96, 44, 96),
'stain':  (85, 96, 45, 96), 'stair':  (89, 96, 48, 96), 'stake':  (87, 96, 53, 96), 'stale':  (90, 96, 58, 96), 'stalk':  (81, 96, 44, 96),
'stall':  (79, 96, 46, 96), 'stamp':  (77, 96, 38, 96), 'stand':  (84, 96, 45, 96), 'stank':  (80, 96, 45, 96), 'stare':  (90, 97, 57, 97),
'stark':  (83, 96, 44, 96), 'start':  (81, 96, 50, 96), 'stash':  (73, 96, 46, 96), 'state':  (85, 96, 57, 96), 'stave':  (86, 96, 53, 96),
'stead':  (89, 96, 39, 96), 'steak':  (87, 96, 39, 96), 'steal':  (90, 96, 41, 96), 'steam':  (88, 96, 36, 96), 'steed':  (79, 96, 46, 96),
'steel':  (82, 96, 47, 96), 'steep':  (77, 96, 43, 96), 'steer':  (83, 96, 50, 96), 'stein':  (87, 96, 39, 96), 'stern':  (89, 96, 39, 96),
'stick':  (73, 96, 42, 96), 'stiff':  (66, 96, 33, 96), 'still':  (75, 96, 44, 96), 'stilt':  (75, 96, 49, 96), 'sting':  (77, 96, 40, 96),
'stink':  (75, 96, 43, 96), 'stint':  (72, 96, 49, 96), 'stock':  (75, 96, 41, 96), 'stoic':  (82, 96, 38, 96), 'stoke':  (86, 96, 50, 96),
'stole':  (90, 96, 55, 96), 'stomp':  (74, 96, 35, 96), 'stone':  (89, 96, 56, 96), 'stony':  (80, 96, 53, 96), 'stood':  (72, 96, 40, 96),
'stool':  (77, 96, 42, 96), 'stoop':  (70, 96, 38, 96), 'store':  (89, 96, 55, 96), 'stork':  (81, 96, 41, 96), 'storm':  (82, 96, 38, 96),
'story':  (84, 96, 52, 96), 'stout':  (71, 96, 44, 96), 'stove':  (85, 96, 50, 96), 'strap':  (84, 96, 36, 96), 'straw':  (84, 96, 34, 96),
'stray':  (87, 96, 49, 96), 'strip':  (82, 96, 35, 96), 'strut':  (75, 96, 41, 96), 'stuck':  (69, 96, 38, 96), 'study':  (74, 96, 45, 96),
'stuff':  (61, 96, 29, 96), 'stump':  (68, 96, 32, 96), 'stung':  (74, 96, 36, 96), 'stunk':  (72, 96, 39, 96), 'stunt':  (69, 96, 45, 96),
'style':  (88, 96, 46, 96), 'suave':  (87, 96, 57, 96), 'sugar':  (85, 96, 43, 96), 'suing':  (73, 96, 45, 96), 'suite':  (87, 96, 60, 96),
'sulky':  (70, 96, 47, 96), 'sully':  (68, 96, 51, 96), 'sumac':  (78, 96, 35, 96), 'sunny':  (65, 96, 53, 96), 'super':  (85, 96, 49, 96),
'surer':  (82, 96, 54, 96), 'surge':  (85, 96, 52, 96), 'surly':  (85, 96, 54, 96), 'sushi':  (67, 96, 29, 96), 'swami':  (80, 96, 34, 96),
'swamp':  (71, 96, 36, 96), 'swarm':  (80, 96, 39, 96), 'swash':  (68, 96, 44, 96), 'swath':  (76, 96, 43, 96), 'swear':  (87, 96, 42, 96),
'sweat':  (87, 96, 43, 96), 'sweep':  (70, 96, 42, 96), 'sweet':  (75, 96, 50, 96), 'swell':  (76, 96, 39, 96), 'swept':  (79, 96, 38, 96),
'swift':  (70, 96, 42, 96), 'swill':  (68, 96, 43, 96), 'swine':  (83, 96, 55, 96), 'swing':  (69, 96, 39, 96), 'swirl':  (85, 96, 43, 96),
'swish':  (61, 96, 43, 96), 'swoon':  (65, 96, 40, 96), 'swoop':  (60, 96, 36, 96), 'sword':  (77, 96, 40, 96), 'swore':  (85, 96, 53, 96),
'sworn':  (80, 96, 40, 96), 'swung':  (66, 96, 34, 96), 'synod':  (76, 96, 34, 96), 'syrup':  (77, 96, 30, 96), 'tabby':  (71, 96, 39, 96),
'table':  (88, 96, 47, 96), 'taboo':  (77, 96, 30, 96), 'tacit':  (78, 96, 40, 96), 'tacky':  (77, 96, 40, 96), 'taffy':  (70, 96, 38, 96),
'taint':  (79, 96, 50, 96), 'taken':  (87, 96, 39, 96), 'taker':  (88, 96, 43, 96), 'tally':  (77, 96, 47, 96), 'talon':  (88, 96, 36, 96),
'tamer':  (88, 96, 45, 96), 'tango':  (83, 96, 31, 96), 'tangy':  (80, 96, 45, 96), 'taper':  (88, 96, 45, 96), 'tapir':  (87, 96, 38, 96),
'tardy':  (85, 96, 45, 96), 'tarot':  (83, 96, 43, 96), 'taste':  (85, 96, 47, 96), 'tasty':  (76, 96, 45, 96), 'tatty':  (67, 96, 46, 96),
'taunt':  (78, 96, 45, 96), 'tawny':  (79, 96, 44, 96), 'teach':  (87, 96, 43, 96), 'teary':  (91, 96, 52, 96), 'tease':  (85, 96, 56, 96),
'teddy':  (78, 96, 39, 96), 'teeth':  (69, 96, 37, 96), 'tempo':  (84, 96, 24, 96), 'tenet':  (73, 96, 48, 96), 'tenor':  (89, 96, 38, 96),
'tense':  (81, 96, 49, 96), 'tenth':  (78, 96, 35, 96), 'tepee':  (68, 96, 51, 96), 'tepid':  (83, 96, 31, 96), 'terra':  (86, 96, 33, 96),
'terse':  (83, 96, 50, 96), 'testy':  (81, 96, 42, 96), 'thank':  (77, 96, 39, 96), 'theft':  (72, 96, 33, 96), 'their':  (87, 96, 36, 96),
'theme':  (75, 96, 42, 96), 'there':  (80, 96, 45, 96), 'these':  (77, 96, 46, 96), 'theta':  (83, 96, 29, 96), 'thick':  (69, 96, 36, 96),
'thief':  (82, 96, 39, 96), 'thigh':  (64, 96, 33, 96), 'thing':  (73, 96, 34, 96), 'think':  (71, 96, 37, 96), 'third':  (80, 96, 36, 96),
'thong':  (74, 96, 33, 96), 'thorn':  (83, 96, 35, 96), 'those':  (87, 96, 49, 96), 'three':  (80, 96, 52, 96), 'threw':  (81, 96, 34, 96),
'throb':  (78, 96, 26, 96), 'throw':  (76, 96, 26, 96), 'thrum':  (77, 96, 25, 96), 'thumb':  (64, 96, 23, 96), 'thump':  (66, 96, 25, 96),
'thyme':  (83, 96, 35, 96), 'tiara':  (84, 96, 38, 96), 'tibia':  (76, 96, 27, 96), 'tidal':  (85, 96, 32, 96), 'tiger':  (86, 96, 41, 96),
'tight':  (64, 96, 30, 96), 'tilde':  (87, 96, 41, 96), 'timer':  (87, 96, 41, 96), 'timid':  (65, 96, 30, 96), 'tipsy':  (75, 96, 41, 96),
'titan':  (79, 96, 33, 96), 'tithe':  (79, 96, 39, 96), 'title':  (83, 96, 45, 96), 'toast':  (81, 96, 50, 96), 'today':  (84, 96, 44, 96),
'toddy':  (68, 96, 40, 96), 'token':  (85, 96, 38, 96), 'tonal':  (88, 96, 38, 96), 'tonga':  (83, 96, 31, 96), 'tonic':  (80, 96, 33, 96),
'tooth':  (58, 96, 41, 96), 'topaz':  (79, 96, 28, 96), 'topic':  (79, 96, 29, 96), 'torch':  (80, 96, 38, 96), 'torso':  (79, 96, 35, 96),
'torus':  (84, 96, 31, 96), 'total':  (82, 96, 37, 96), 'totem':  (81, 96, 39, 96), 'touch':  (74, 96, 38, 96), 'tough':  (73, 96, 35, 96),
'towel':  (86, 96, 40, 96), 'tower':  (85, 96, 42, 96), 'toxic':  (74, 96, 27, 96), 'toxin':  (74, 96, 31, 96), 'trace':  (89, 97, 56, 97),
'track':  (82, 96, 43, 96), 'tract':  (80, 96, 49, 96), 'trade':  (88, 96, 53, 96), 'trail':  (91, 96, 45, 96), 'train':  (89, 96, 44, 96),
'trait':  (84, 96, 49, 96), 'tramp':  (81, 96, 37, 96), 'trash':  (85, 96, 45, 96), 'trawl':  (86, 96, 39, 96), 'tread':  (88, 96, 38, 96),
'treat':  (86, 96, 44, 96), 'trend':  (86, 96, 39, 96), 'triad':  (87, 96, 42, 96), 'trial':  (91, 96, 43, 96), 'tribe':  (86, 96, 49, 96),
'trice':  (88, 96, 54, 96), 'trick':  (79, 96, 41, 96), 'tried':  (86, 96, 48, 96), 'tripe':  (87, 96, 50, 96), 'trite':  (84, 96, 54, 96),
'troll':  (83, 96, 42, 96), 'troop':  (75, 96, 37, 96), 'trope':  (87, 96, 49, 96), 'trout':  (77, 96, 43, 96), 'trove':  (84, 96, 49, 96),
'truce':  (87, 96, 50, 96), 'truck':  (76, 96, 37, 96), 'truer':  (82, 96, 48, 96), 'truly':  (86, 96, 48, 96), 'trump':  (76, 96, 31, 96),
'trunk':  (80, 96, 38, 96), 'truss':  (75, 96, 34, 96), 'trust':  (75, 96, 43, 96), 'truth':  (71, 96, 37, 96), 'tryst':  (77, 96, 38, 96),
'tubal':  (80, 96, 31, 96), 'tuber':  (84, 96, 40, 96), 'tulip':  (78, 96, 29, 96), 'tulle':  (82, 96, 45, 96), 'tumor':  (80, 96, 32, 96),
'tunic':  (77, 96, 29, 96), 'turbo':  (79, 96, 25, 96), 'tutor':  (77, 96, 34, 96), 'twang':  (76, 96, 31, 96), 'tweak':  (85, 96, 28, 96),
'tweed':  (72, 96, 35, 96), 'tweet':  (66, 96, 41, 96), 'twice':  (83, 96, 45, 96), 'twine':  (83, 96, 46, 96), 'twirl':  (85, 96, 33, 96),
'twist':  (66, 96, 38, 96), 'twixt':  (54, 96, 31, 96), 'tying':  (74, 96, 29, 96), 'udder':  (77, 96, 28, 96), 'ulcer':  (87, 96, 35, 96),
'ultra':  (89, 96, 24, 96), 'umbra':  (79, 96, 15, 96), 'uncle':  (85, 96, 33, 96), 'uncut':  (68, 96, 22, 96), 'under':  (84, 96, 31, 96),
'undid':  (63, 96, 20, 96), 'undue':  (75, 96, 30, 96), 'unfed':  (78, 96, 25, 96), 'unfit':  (73, 96, 24, 96), 'unify':  (69, 96, 34, 96),
'union':  (73, 96, 28, 96), 'unite':  (87, 96, 41, 96), 'unity':  (77, 96, 38, 96), 'unlit':  (81, 96, 28, 96), 'unmet':  (84, 96, 32, 96),
'unset':  (86, 96, 33, 96), 'untie':  (87, 96, 35, 96), 'until':  (81, 96, 24, 96), 'unwed':  (77, 96, 25, 96), 'unzip':  (65, 96, 15, 96),
'upper':  (78, 96, 29, 96), 'upset':  (82, 96, 32, 96), 'urban':  (83, 96, 28, 96), 'urine':  (89, 96, 51, 96), 'usage':  (89, 96, 37, 96),
'usher':  (86, 96, 25, 96), 'using':  (73, 96, 23, 96), 'usual':  (77, 96, 23, 96), 'usurp':  (70, 96, 18, 96), 'utile':  (88, 96, 42, 96),
'utter':  (82, 96, 32, 96), 'uvula':  (70, 96, 19, 96), 'vague':  (84, 96, 40, 96), 'valet':  (87, 96, 44, 96), 'valid':  (78, 96, 32, 96),
'valor':  (85, 96, 35, 96), 'value':  (86, 96, 42, 96), 'valve':  (78, 96, 40, 96), 'vapid':  (75, 96, 29, 96), 'vapor':  (81, 96, 32, 96),
'vault':  (80, 96, 40, 96), 'vaunt':  (80, 96, 41, 96), 'vegan':  (81, 96, 28, 96), 'venom':  (80, 96, 26, 96), 'venue':  (72, 96, 40, 96),
'verge':  (70, 96, 41, 96), 'verse':  (76, 96, 45, 96), 'verso':  (85, 96, 29, 96), 'verve':  (65, 96, 40, 96), 'vicar':  (82, 96, 29, 96),
'video':  (83, 96, 30, 96), 'vigil':  (58, 96, 27, 96), 'vigor':  (76, 96, 28, 96), 'villa':  (73, 96, 25, 96), 'vinyl':  (72, 96, 23, 96),
'viola':  (87, 96, 31, 96), 'viper':  (81, 96, 36, 96), 'viral':  (86, 96, 31, 96), 'virus':  (79, 96, 23, 96), 'visit':  (65, 96, 32, 96),
'visor':  (82, 96, 29, 96), 'vista':  (81, 96, 23, 96), 'vital':  (82, 96, 29, 96), 'vivid':  (43, 96, 25, 96), 'vixen':  (73, 96, 30, 96),
'vocal':  (79, 96, 30, 96), 'vodka':  (73, 96, 22, 96), 'vogue':  (80, 96, 39, 96), 'voice':  (85, 96, 50, 96), 'voila':  (87, 96, 35, 96),
'vomit':  (74, 96, 34, 96), 'voter':  (84, 96, 42, 96), 'vouch':  (65, 96, 34, 96), 'vowel':  (79, 96, 35, 96), 'vying':  (63, 96, 24, 96),
'wacky':  (68, 96, 37, 96), 'wafer':  (83, 96, 41, 96), 'wager':  (84, 96, 42, 96), 'wagon':  (77, 96, 31, 96), 'waist':  (82, 96, 47, 96),
'waive':  (84, 96, 49, 96), 'waltz':  (74, 96, 28, 96), 'warty':  (83, 96, 45, 96), 'waste':  (87, 96, 44, 96), 'watch':  (74, 96, 34, 96),
'water':  (87, 96, 44, 96), 'waver':  (81, 96, 42, 96), 'waxen':  (80, 96, 37, 96), 'weary':  (87, 96, 50, 96), 'weave':  (73, 96, 48, 96),
'wedge':  (63, 96, 39, 96), 'weedy':  (68, 96, 40, 96), 'weigh':  (77, 96, 35, 96), 'weird':  (81, 96, 37, 96), 'welch':  (79, 96, 31, 96),
'welsh':  (82, 96, 32, 96), 'wench':  (77, 96, 33, 96), 'whack':  (66, 96, 34, 96), 'whale':  (85, 96, 48, 96), 'wharf':  (75, 96, 31, 96),
'wheat':  (85, 96, 35, 96), 'wheel':  (73, 96, 38, 96), 'whelp':  (78, 96, 27, 96), 'where':  (73, 96, 42, 96), 'which':  (56, 96, 34, 96),
'whiff':  (51, 96, 24, 96), 'while':  (83, 96, 47, 96), 'whine':  (80, 96, 47, 96), 'whiny':  (68, 96, 45, 96), 'whirl':  (81, 96, 35, 96),
'whisk':  (65, 96, 34, 96), 'white':  (82, 96, 46, 96), 'whole':  (84, 96, 46, 96), 'whoop':  (55, 96, 28, 96), 'whose':  (83, 96, 46, 96),
'widen':  (78, 96, 35, 96), 'wider':  (81, 96, 38, 96), 'widow':  (63, 96, 22, 96), 'width':  (69, 96, 28, 96), 'wield':  (80, 96, 32, 96),
'wight':  (68, 96, 27, 96), 'willy':  (63, 96, 40, 96), 'wimpy':  (62, 96, 33, 96), 'wince':  (81, 96, 43, 96), 'winch':  (66, 96, 31, 96),
'windy':  (65, 96, 37, 96), 'wiser':  (86, 96, 39, 96), 'wispy':  (68, 96, 34, 96), 'witch':  (69, 96, 30, 96), 'witty':  (64, 96, 39, 96),
'woken':  (80, 96, 35, 96), 'woman':  (77, 96, 31, 96), 'women':  (81, 96, 38, 96), 'woody':  (55, 96, 45, 96), 'wooer':  (78, 96, 49, 96),
'wooly':  (64, 96, 49, 96), 'woozy':  (48, 96, 43, 96), 'wordy':  (72, 96, 41, 96), 'world':  (79, 96, 35, 96), 'worry':  (67, 96, 45, 96),
'worse':  (85, 96, 48, 96), 'worst':  (80, 96, 41, 96), 'worth':  (76, 96, 35, 96), 'would':  (72, 96, 35, 96), 'wound':  (68, 96, 36, 96),
'woven':  (78, 96, 37, 96), 'wrack':  (75, 96, 40, 96), 'wrath':  (81, 96, 40, 96), 'wreak':  (83, 96, 35, 96), 'wreck':  (77, 96, 34, 96),
'wrest':  (85, 96, 41, 96), 'wring':  (74, 96, 36, 96), 'wrist':  (81, 96, 45, 96), 'write':  (85, 96, 51, 96), 'wrong':  (73, 96, 35, 96),
'wrote':  (85, 96, 50, 96), 'wrung':  (71, 96, 32, 96), 'wryly':  (72, 96, 39, 96), 'yacht':  (78, 96, 28, 96), 'yearn':  (91, 96, 36, 96),
'yeast':  (91, 96, 42, 96), 'yield':  (84, 96, 29, 96), 'young':  (70, 96, 29, 96), 'youth':  (75, 96, 31, 96), 'zebra':  (82, 96, 22, 96),
'zesty':  (81, 96, 36, 96), 'zonal':  (81, 96, 32, 96),
}

print(f'{type(wordle_word_dict) = }  {len(wordle_word_dict) = }  {(wordle_word_list == list(wordle_word_dict.keys())) = }')




type(wordle_word_dict) = <class 'dict'>  len(wordle_word_dict) = 2317  (wordle_word_list == list(wordle_word_dict.keys())) = True


In [ ]:
def alpha_segments(wrds, line_length=12, multiple_lines_per_letter=None):
    """
    Convert a list of words into a single string for printing.
    The string is broken into lines by '\n' characters.
    - line_length is number of words per line.
    - multiple_lines_per_letter: may a line of words be split among multiple lines?
      If not, truncate lines with more than line_length words
    Order the lines of words alphabetically.
    """
    if wrds == [] or line_length <= 0: return ''

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    # lsts_of_wrds is lists of words, one list for words starting each letter of the alphabet.
    # Also, drop empy lists, i.e., lists corresponding to letters that start no words.
    lsts_of_wrds = [lst for lst in [[w for w in wrds if w[0] == ltr] for ltr in alphabet] if len(lst) > 0]
    lngth_of_longest_word_list = max([len(lst) for lst in lsts_of_wrds])

    # If multiple_lines_per_letter was set by the caller, keep it, otherwise set it
    # to depend on whether lngth_of_longest_word_list > line_length
    multiple_lines_per_letter = (multiple_lines_per_letter if multiple_lines_per_letter is not None
                                 else lngth_of_longest_word_list > line_length)

    if multiple_lines_per_letter:
        # For each list of words:
        # a) Add an extra '\n\t' in front of each list. This puts an empty line between the lines
        #    for each member of the alphabet--making it easier to read. Do this by adding '\n\t'
        #    as characters to the first word in each list.
        # b) Either break each list into multiple lists or truncate each list: the resulting lists
        #    will have length <= line_length.
        # bnded_lsts_of_wrds will be lists of words, each list of length <= line_length
        bnded_lsts_of_wrds = []
        for lst in lsts_of_wrds:
            # lst is originally a list of words starting with the same letter
            lst = ['\n\t' + lst[0]] + lst[1:]
            # Like bnded_lsts_of_wrds, lsts will be a list of lists of words,
            # each no lonter than line_length. Add those lists to bnded_lsts_of_wrds
            lsts = [lst[i:i + line_length] for i in range(0, len(lst), line_length)]
            bnded_lsts_of_wrds += lsts
    else:  # multiple_lines_per_letter = False
        # Since we are not splitting up long lists, truncate them and add '...' if too long.
        # Also, add no extra space between lines as we did with the extra '\n\t' additions.
        bnded_lsts_of_wrds = [(lst[:line_length] + ([] if len(lst) <= line_length else ['...']))
                                    for lst in lsts_of_wrds]

    # Join each list of words into a string with the words separated by ', '
    # Add \t in front of each string.
    # Add a comma at the end of each string if the end is not '...'.
    # Use the pack/unpack (*) operator to isolate the last string.
    *list_of_strings, last_string = [('\t' + ', '.join(lst) + (',' if lst[-1] != '...' else ''))
                                            for lst in bnded_lsts_of_wrds]
    # If the last_string ends with a ',', remove it.
    if last_string[-1] == ',':
        last_string = last_string[:-1]

    # Combine all the strings into one, separated by '\n'
    final_string = "\n".join(list_of_strings + [last_string])

    # If multiple_lines_per_letter, final_string will begin with an extraneous '\n\t'. Remove it.
    final_string = final_string[2:] if multiple_lines_per_letter else final_string

    return final_string


# print('alpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=True):')
# print(alpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=True))

# print()
# print('\nalpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=False):')
# print(alpha_segments(wordle_word_list[2200:], multiple_lines_per_letter=False))

# print('\nalpha_segments(wordle_word_list[2200:]):')
# print(alpha_segments(wordle_word_list[2200:]))

# print('\nalpha_segments(wordle_word_list[2295:]):')
# print(alpha_segments(wordle_word_list[2295:]))

# print('\nalpha_segments(wordle_word_list[2293:]):')
# print(alpha_segments(wordle_word_list[2293:]))


In [ ]:

class Filter():
    def __init__(self, word_length):

        self.word_length = word_length

        # The filter is the combination of self.greens, self.grays, self.yellows, and self.disallowed.
        # self.greens are the known letters; '_' if not known.
        # Initially none of the letters are known.
        self.greens = list('_____')

        # self.grays are letters known not to be in the word. These are lettters that were guessed
        # and returned gray, i.e., not in the word. Not included as grays are lettters that returned
        # gray because they are known greens or known to be unmatched from a yellow.
        self.grays = set()

        # self.disallowed are the letters that are known not to be at specific positions,
        # e.g., letters that have been guessed at a position and have been yellow
        self.disallowed: List[set] = [set() for _ in range(self.word_length)]

        # A Counter of Counters. A count of its still-hidden instances for each hidden letter.
        self.yellows = Counter()


    def __str__(self):
        return f'*** {self.greens = }\n*** {self.grays = }\n*** {self.disallowed = }\n*** {self.yellows = }'


    def edit_filter(self, guess, score, verbose=False) -> Self:   #  -> Filter
        """
        The score characters are interpreted as follows.
        'g': the letter is at its correct position in guess.
        'y': the letter is not at its correct position in guess but is elsewhere in the word
        '#': the letter is not at any position in the word that is neither green nor assigned

        Update the filter based on the guess and the score.
        """

        # Since we use the iterator enumerate(zip(self.guess, self.score))
        # multiple times, make it a concrete list to allow reuse.
        enumerated_guess_score = list(enumerate(zip(guess, score)))

        # Update the greens first.
        for index, (guess_ltr, score_letter) in enumerated_guess_score:

            # If this is a new green: self.greens[index] == '_'
            if score_letter == 'g' and self.greens[index] == '_':
                # Record the green value
                self.greens[index] = guess_ltr
                # This shouldn't be necessary.
                self.grays.discard(guess_ltr)

                # Decrement the yellow_counter for guess_ltr if there is one
                if self.yellows[guess_ltr] <= 1:
                    del self.yellows[guess_ltr]
                else: # self.yellows[guess_ltr] > 1:
                    self.yellows[guess_ltr] -= 1
            # If this is an existing green, i.e., self.greens[index] == guess_ltr, do nothing

        # Update the 'y's, i.e., update self.disallowed and self.yellows.

        # Update self.disallowed by adding the yellow letters to the disallowed sets for each yellow letter position
        for index, (guess_ltr, score_letter) in enumerated_guess_score:
            if score_letter == 'y':
                self.disallowed[index].add(guess_ltr)

        # Update self.yellows:
        # First, make a local yellow Counter for the y-letters in the score for this guess.
        # The following will count the number of occurrences of each yellow letter.
        yellows = Counter([guess_ltr for _, (guess_ltr, score_letter) in enumerated_guess_score
                                  if score_letter == 'y'])

        # Update yellows.
        # The new value is the max value from yellows and self.yellows
        for guess_ltr in yellows:
            empty_holes = [(green == guess_ltr and sc != 'g') for green, sc in zip(self.greens, score)].count(True)
            self.yellows[guess_ltr] = \
                                max(self.yellows[guess_ltr], yellows[guess_ltr] - empty_holes)

        # Update self.grays and self.disallowed.
        # Add guess_ltr to self.grays if it is marked # and is in neither self.greens nor self.yellows
        # If guess_ltr is in self.greens or self.yellows, add it to self.disallowed[index] but not to self.grays
        for index, (guess_ltr, score_ltr) in enumerated_guess_score:
            if score_ltr == '#':
                self.disallowed[index].add(guess_ltr)
                # Did it get a # because it is in the secret word but its
                # hole was already filled when this letter was scored?
                if guess_ltr in self.greens or guess_ltr in self.yellows:
                    # This shouldn't be necessary.
                    self.grays.discard(guess_ltr)
                else:
                    self.grays.add(guess_ltr)

        return self


    def edit_filter_from_profiles(self, position_profiles) -> List[str]:
        greens_profile_keys = zip(self.greens, [list(prfl.keys()) for prfl in position_profiles])
        new_greens = [(keys[0] if ltr == '_' and len(keys) == 1 else ltr) for ltr, keys in greens_profile_keys]

        # Identify the new green letters
        new_green_ltrs = [new_ltr for new_ltr, old_ltr in zip(new_greens, self.greens) if new_ltr != old_ltr]
        self.greens = new_greens

        # No need to test whether new_green_ltrs is empty.
        # If len(new_green_ltrs) == 0, the following loop will do nothing.
        for ltr in new_green_ltrs:
            self.yellows[ltr] -= 1
            if self.yellows[ltr] <= 0:
                del self.yellows[ltr]

        return new_green_ltrs


    def word_passes(self, word):
        '''
        A word fails the filter if:
        - It contains a gray letter
        - It fails to match a known (i.e., green) letter
        - It includes a letter in one of the disallowed sets
        - It is missing a required hidden yellow letter
        '''
        # This loop makes the first three tests.
        for index, ltr in enumerate(word):
            # print(index, ltr, self.disallowed[index], end = ';  ')
            if (ltr in self.grays or
                # If self.greens[index] not in {'_', ltr}, the secret word's
                # letter at index is known to be something other than ltr.
                self.greens[index] not in {'_', ltr} or
                ltr in self.disallowed[index]
                ): return False

        # The following makes the yellows test.

        # Count the instances of the letters in word
        letter_counter = Counter(word)

        # Count the instances of the letters in self.greens
        green_counters = Counter(self.greens)

        # self.yellows[ltr] is the known number of hidden positions in the secret word where ltr appears.
        # green_counters[ltr] is the number of known positions in the secret word where ltr appears.
        # I.e., the secret word requires at least self.yellows[ltr] + green_counters[ltr] ltr instances.
        # If letter_counter[ltr] is less than self.yellows[ltr] + green_counters[ltr], the word fails
        # because it has fewer instances of ltr than the secret word is known to require.
        for ltr in self.yellows:
            if letter_counter[ltr] < self.yellows[ltr] + green_counters[ltr]: return False

        return True



In [ ]:

class Utils():
    """
    Utility and I/O functions as static (class) methods
    """


    @staticmethod
    def lists_to_many_lists(lst, list_length):
        lsts = [lst[i:i + list_length] for i in range(0, len(lst), list_length)]
        return lsts


    @staticmethod
    def dict_to_row_strings(dct: Dict[str, Tuple[float, float]]) -> str:
        spacing = '    '
        rows = [ [f'{key}: ({avg:3}, {med:2}, )' for key, (avg, med) in list(dct.items())[i: i+5]]
                                                                            for i in range(0, len(dct), 5)]
        row_strings = [f',{spacing}'.join(row) for row in rows]
        full_string = spacing + f'\n{spacing}'.join(row_strings)
        return full_string


    @staticmethod
    def filter_guess_lists(guess_lists: List[List[str]]) -> List[List[str]]:

        guess_lists_and_sets: List[Tuple[List[str], Set[str]]] = []

        for guess_list in guess_lists:

            if len(guess_list) == 0:
                continue

            guess_set = set(guess_list)
            to_be_added = True
            for list_x, set_x in guess_lists_and_sets:
                if guess_set.issubset(set_x):
                    to_be_added = False
                    continue
                else:
                    if guess_set.issuperset(set_x):
                        guess_lists_and_sets.remove((list_x, set_x))
            if to_be_added:
                guess_lists_and_sets.append((guess_list, guess_set))

        filtered_list = [list for list, _ in guess_lists_and_sets]

        return filtered_list


    @staticmethod
    def get_score_from_user():
        while True:
            print('Please use the following letters to enter a 5-character score: g - Green; y - Yellow; # - gray')
            score = (input('    Score > ') + ' '*5)[:5].strip().lower()
            if len(score) == 5 and set(score).issubset(set('gy#')):
                return score
            elif len(score) != 5:
                print(f'\n==> Your score entry "{score}" has {len(score)} letters rather than the needed 5.')
            elif not set(score).issubset(set('gy#')):
                print(f'\n==> Your score entry "{score}" includes letters other than "g", "y", and "#".')
            else:
                print(f'\n==> Something is amiss with your score entry "{score}." Please try again.')


    @staticmethod
    def get_user_guess(current_step, guess_lists: List[str], hard_mode: bool=True, filter: Filter=Filter(3)):

        # menus_dicts: List[Dict[int, str]] = \
        #        [{nbr: guess for nbr, guess in list(enumerate(guess_dict, 1))[:8]} for guess_dict in guess_dicts]

        # labelled_menus =  {label: menu for label, menu in zip('ABCDEFG', menus_dicts)}

        menu = guess_lists
        menu_lines = Utils.lists_to_many_lists(menu, 10)

        while True:
            print("\nEnter either (a) your guess or (b) your selection from the following menus of suggested guesses. ")
            print("To select from the menu, enter the desired item number.\n")
            # for menu_label in labelled_menus:
            #     labelled_menu = labelled_menus[menu_label]
            #     print(f'{menu_label}: {", ".join([str(nbr)+". "+str(item) for nbr, item in labelled_menu.items()])}')

            for line_nbr in range(len(menu_lines)):
                menu_line = menu_lines[line_nbr]
                # f&quot;{my_num:03d}&quot;
                print(f'{", ".join([f"{nbr:2}"+". "+str(item) for nbr, item in enumerate(menu_line, 10*line_nbr + 1)])}')
                # print(f'{", ".join([(f&quot;{nbr:02d}&quot;)+". "+str(item) for nbr, item in enumerate(menu_line, 10*line_nbr + 1)])}')

            user_guess = (input(f'\nGuess {current_step}   > ') + ' '*5)[:5].strip().lower()
            # menu_label, item_number = user_guess[0], sub(r'D', ', user_guess[1:])

            if len(user_guess) <= 2:    # s1 = "sub(r'\D', ', '   ds3f  ')"

                item_number = sub(r'\D', '', user_guess)
                if item_number.isnumeric():
                    selection = menu[int(item_number)-1]
                    print(f'selection = {selection}')
                    return selection

            elif len(user_guess) == 5 and user_guess.isalpha():
                if hard_mode and not filter.word_passes(user_guess):
                    print(f'Hard-mode violation: "{user_guess}" fails one or more known constraints.')
                else:
                    return user_guess

            # No "else" in case the user enters a single non-digit character
            print(f'\nPlease enter either a number of one or two digits or a sequence of 5 alphabetic characters.')


    @staticmethod
    def get_user_secret_word(current_candidates, guess_words, word_length):

        while True:
            print('Please enter a 5 letter secret word.')
            print('Or enter "help" for the list of possible secret words.')
            print('Or enter "no" to have the system pick a secret word.\n')
            user_secret = input(' secret word > ').lower()

            # Do these cases first
            match user_secret:

                case 'no':
                    user_secret = choice(current_candidates)
                    print('A secret word has been selected.')
                    return user_secret

                case 'help':
                    print(alpha_segments(guess_words, line_length=19, multiple_lines_per_letter=True))
                    continue


            if len(user_secret) != word_length:
                    comparator = "more" if len(user_secret) > word_length else "fewer"
                    print(f'"{user_secret}" has {comparator} than 5 characters.')
                    continue

            if user_secret not in current_candidates:
                    print(f'"{user_secret}" isn\'t in the list of currently allowable secret words.')
                    continue

            # Since we found no reason to reject user_secret, return it as the secret word.
            return user_secret


    @staticmethod
    def nbr_str(n, w=2):
        st = f"{n:{w}}"
        return st


    @staticmethod
    def smallest_n(dict: Dict[str, Tuple], n=20) -> Dict[str, Tuple]:
        """
        Given a dictionary of type Dict[str, Tuple[int, int]], sort the keys with
        the smallest Tuple values.
        """
        sorted_elts: List[str] = sorted(dict, key=lambda guess: dict[guess])[:n]
        the_best = {guess: dict[guess] for guess in sorted_elts}
        return the_best


    @staticmethod
    def print_profiles(cndt_prfl, current_step, filter):
        init_or_rem = "" if current_step == 1 else "remaining "
        len_candidates = len(cndt_prfl.current_candidates)
        print('\n' + '-'*20 + f' The {"initial" if current_step == 1 else "current"} state of the game ' + '-'*20)


        init_or_rem = 'initial' if current_step == 1 else 'remaining'
        print(f'\n{len_candidates} {init_or_rem} secret-word candidate{"s" if len_candidates > 1 else ""}\n')
        if current_step > 1:
            print(alpha_segments(cndt_prfl.current_candidates))

        print('\n' + ("Initial letter position frequencies"
                      if current_step == 1 else
                      "Letter position frequencies in remaining words")
              )
        for (pos, counter) in enumerate(cndt_prfl.position_profiles):
            most_common = dict(counter.most_common()[:5])
            print(f"{pos + 1}. {most_common}")
        str_1 = "Letters that appear in the most words in the " + \
                ('initial' if current_step == 1 else 'remaining') + \
                " secret-word candidates:\n\t"
        str_2 = '' if len(cndt_prfl.letter_profiles) <= 9 else f' (out of {len(cndt_prfl.letter_profiles)})'
        print(f'\n{str_1}{dict(cndt_prfl.letter_profiles.most_common()[:9])} {str_2}')
        Utils.print_filter(cndt_prfl, filter)


    @staticmethod
    def print_filter(cndt_prfl, filter):
        print(f'\nPlaced letters: {"".join(filter.greens)}')
        unplaced = sorted(filter.yellows.keys())
        disalloweds = [(ltr, [str(n+1) for n in range(len(filter.disallowed)) if ltr in filter.disallowed[n]]) for ltr in unplaced]
        unplaced_strs = [f'"{ltr}" but not in position{"" if len(ns) == 1 else"s"} {", ".join(ns)}' for ltr, ns in disalloweds]

        print('Required but unplaced letters:' + \
              (('\n\t\t' + "\n\t\t".join(unplaced_strs)) if len(unplaced_strs) > 0 else "")
              )
        print(f'Excluded letters: {", ".join(sorted(cndt_prfl.excluded_letters))}')

        len_uncat = len(cndt_prfl.uncategorized_letters)
        str_1 = ('' if len_uncat <= 9 else f'(out of {len_uncat})') + '\n'
        min_len = len_uncat // 10
        uncat = Counter({ltr: cnt for ltr, cnt in cndt_prfl.uncategorized_letters.items() if cnt > min_len}).most_common()[:9]
        print(f'{len_uncat} uncategorized letter{"" if len_uncat == 1 else "s"}:' +
              ('\n\t' if len_uncat > 9 else ' ') +
              ("" if len_uncat == 0 else f'{dict(uncat)} {str_1}'))


    @staticmethod
    def raw_remaining_to_pct_removed(raw, total):
        return int(round(100*(total - raw)/total))


    @staticmethod
    def score_a_guess(secret, guess, verbose=False) -> str:
        '''
        Build a list with one char for each guess element.
        g = Letter in Correct position (green);
        y = Letter in Incorrect position (yellow);
        # = Letter not in secret word. (These are the gray results. Can't use 'g' since we use it for green)

        In the following example: secret    = 'hpaay'
                                  guess     = 'aaapz'
                                  -> score  = 'y#gy#'

        The third letter in both the secret and the guess is "a".
            So the third letter in the score must be "g".
            (The matching letters must be determined first.)
        The first letter in the guess, 'a', is associated with the fourth letter in the secret, 'a'.
            So the first letter in the score is "y".
        The second letter in the guess, 'a', has nothing in secret to be associated with.
            So the second letter in the score is "#".
        The other two letter in the score should be self explanatory.
        '''

        # Generate the score list with the letters in the correct position marked 'g'.
        # All non-green elements are marked '#'. These will be changed to 'y' later if appropriate.
        # score = [('g' if guess[i] == secret[i] else '#') for i in range(word_length)]
        score = [('g' if sec_ltr == guess_ltr else '#') for sec_ltr, guess_ltr in zip(secret, guess)]

        # Generate a list of the "holes," i.e., secret letters that do not have correct corresponding guess letters.
        # secret_holes = [secret[i] for i in range(word_length) if score[i] != 'g']
        secret_holes = [sec_ltr for sec_ltr, score_ltr in zip(secret, score) if score_ltr != 'g']

        if verbose: print(f'{secret_holes = }')

        # For each available secret hole find a matching guess letters. Ignore guess letters that are paired with
        # their secret letters. Score the first match as 'y' and don't look further.
        for ltr in secret_holes:
            for i in range(len(score)):
                # Have we found a hole-filling letter in guess that is not otherwise committed?
                # If so, mark it 'y'. Since we have now filled this hole, go on to the next hole.
                if guess[i] == ltr and score[i] == '#':
                    score[i] = 'y'
                    break

        # Return score as a string
        score_string = ''.join(score)
        if verbose:
            print(f'\n\t{score_string = }\n\n')
        return score_string



In [ ]:
from re import sub
print(f"{'sdf'.isnumeric() = }")
s1 = "sub(r'\\D', '', '   ds3f  ')"
print(f"{s1} -> {eval(s1)}")
s2 = "sub(r'\\d', '', '   ds3f  ')"
print(f"{s2} -> {eval(s2)}")
print(f"{'b3'.upper() = }")


'sdf'.isnumeric() = False
sub(r'\D', '', '   ds3f  ') -> 3
sub(r'\d', '', '   ds3f  ') ->    dsf  
'b3'.upper() = 'B3'


In [ ]:
# guess = 'abcde'
# candt = 'bcfaa'
w_1 = 'apaza'
w_2 = 'phaav'
current_candidates = [w_2]
uncategorized_letters = {'g', 'c', 'e', 'd'}
hits1 = [int(w_1[pos] in w_2 and w_1[pos] not in uncategorized_letters) for pos in range(len(w_1)) for cndt in current_candidates]
print(f'{hits1 = }')
hits2 = [int(w_2[pos] in w_1 and w_2[pos] not in uncategorized_letters) for pos in range(len(w_1)) for cndt in current_candidates]
print(f'{hits2 = }\n')

print(f'{Utils.score_a_guess(w_1, w_2) = }')
print(f'{Utils.score_a_guess(w_2, w_1) = }')

hits1 = [1, 1, 1, 0, 1]
hits2 = [1, 0, 1, 1, 0]

Utils.score_a_guess(w_1, w_2) = 'y#gy#'
Utils.score_a_guess(w_2, w_1) = 'yyg##'


In [ ]:
# step_1 = {'raise': (93, 97, 43, 97), 'arise': (93, 97, 51, 97), 'irate': (92, 97, 53, 97), 'arose': (92, 97, 49, 97),
#           'alter': (92, 97, 60, 97), 'stare': (90, 97, 51, 97), 'snare': (91, 97, 54, 97), 'slate': (90, 97, 51, 97),
#           'alert': (92, 97, 52, 97), 'crate': (89, 97, 49, 97), 'trace': (89, 97, 49, 97), 'aisle': (92, 97, 57, 97),
#           'alone': (92, 97, 56, 97), 'crane': (89, 97, 51, 97), 'react': (89, 97, 48, 97), 'scare': (90, 97, 53, 97),
#           'alien': (91, 96, 51, 97), 'caste': (89, 96, 56, 97), 'heart': (89, 96, 55, 97), 'plate': (89, 96, 61, 97)}

# step_f = {'raise': (93, 43, 97, 97), 'arise': (93, 51, 97, 97), 'irate': (92, 53, 97, 97), 'arose': (92, 49, 97, 97),
#           'alter': (92, 60, 97, 97), 'stare': (90, 51, 97, 97), 'snare': (91, 54, 97, 97), 'slate': (90, 51, 97, 97),
#           'alert': (92, 52, 97, 97), 'crate': (89, 49, 97, 97), 'trace': (89, 49, 97, 97), 'aisle': (92, 57, 97, 97),
#           'alone': (92, 56, 97, 97), 'crane': (89, 51, 97, 97), 'react': (89, 48, 97, 97), 'scare': (90, 53, 97, 97),
#           'alien': (91, 51, 96, 97), 'caste': (89, 56, 96, 97), 'heart': (89, 55, 96, 97), 'plate': (89, 61, 96, 97)}

step_1 = {'raise': (93, 43, 61, 70), 'arise': (93, 51, 64, 70), 'irate': (92, 53, 64, 70), 'arose': (92, 49, 66, 70),
          'alter': (92, 60, 70, 70), 'stare': (90, 51, 71, 70), 'snare': (91, 54, 71, 70), 'slate': (90, 51, 72, 70),
          'alert': (92, 52, 72, 70), 'crate': (89, 49, 73, 70), 'trace': (89, 49, 74, 70), 'aisle': (92, 57, 76, 70),
          'alone': (92, 56, 77, 70), 'crane': (89, 51, 79, 70), 'react': (89, 48, 80, 70), 'scare': (90, 53, 81, 70),
          'alien': (91, 51, 84, 70), 'caste': (89, 56, 85, 70), 'heart': (89, 55, 89, 70), 'plate': (89, 61, 89, 70)}

# first_letter = ''

class Guess():

    """
    Instances are guesses along with metrics for them.
    """

    first_letter = ''
    printed_intro = False

    def __init__(self, guess, current_candidates=wordle_word_list, included_letters=set(), current_step=1):

        # if current_step == 1 and not Guess.printed_intro:
        #     print('Examining the guess words: ')
        #     Guess.printed_intro = True

        # if current_step == 1 and guess[0] != Guess.first_letter:
        #     Guess.first_letter = guess[0]
        #     print(f'{guess[0]}', end='')
        #     if guess[0] == 'z':
        #         print()

        self.guess = guess
        self.current_candidates = current_candidates

        self.pct_words_hit = Guess.pct_of_words_hit(guess, current_candidates, included_letters)

        self.pct_pos_matched = Guess.pct_current_cndt_pos_matched(guess, current_candidates, included_letters)

        self.nbr_remaining_mean, self.nbr_remaining_median = \
                step_1[guess][2:] if current_step == 1 and guess in step_1 else \
                (100, 100)        if current_step == 1 and guess not in step_1 else \
                Guess.fwst_remaining(current_candidates, guess)


    def __str__(self) -> str:
        # return f'{self.guess}[{self.nbr_remaining_mean}, {self.pct_pos_matched}, {self.nbr_remaining_median}]'
        return f"'{self.guess}':  {self.metrics()}"


    @staticmethod
    def fwst_remaining(current_candidates, guess) -> Tuple[int, int]:
        """
        Determine the mean (and median) number of remaining candidates for each possible secret word.
        """
        nbr_curr_cndts = len(current_candidates)
        # Keep track of the number of remaining candidates for each possible secret word.
        nbr_remaining_lst: List[float] = []
        for secret_word in current_candidates:
            # Score the guess with secret_word as the secret word
            score = Utils.score_a_guess(guess, secret_word)
            # Create a filter for that combination
            filter = Filter(len(guess)).edit_filter(guess, score)
            # Use the created filter to filter current_candidates
            nbr_remaining_candidates = len([word for word in current_candidates if filter.word_passes(word)])
            # nbr_remaining = nbr_curr_cndts - len([word for word in current_candidates if filter.word_passes(word)])
            nbr_remaining_lst.append(nbr_remaining_candidates)

        # Use an arithmetic average over all the number-remaining values.
        # Use statistics.fmean (float mean), which is faster than statistics.mean.
        nbr_remaining_mean = int(round(fmean(nbr_remaining_lst)))

        # min_remaining: float = int(100*round(min(nbr_remaining_lst), 2))

        # Use the median remaining candidates number as the metric for guess.
        nbr_remaining_median = int(round(median(nbr_remaining_lst)))

        return nbr_remaining_mean, nbr_remaining_median


    def metrics(self) -> Tuple:
        len_cc = len(self.current_candidates)
        # if Utils.raw_to_pct(self.nbr_remaining_mean, len_cc) < 0:
        # print(f'{self.guess = }: {self.nbr_remaining_mean = } {len_cc = } {Utils.raw_to_pct(self.nbr_remaining_mean, len_cc) = }')
        return (self.pct_words_hit,   Utils.raw_remaining_to_pct_removed(self.nbr_remaining_mean, len_cc),
                self.pct_pos_matched, Utils.raw_remaining_to_pct_removed(self.nbr_remaining_median, len_cc)
                )


    @staticmethod
    def pct_current_cndt_pos_matched(guess, current_candidates, included_letters):
        """
        Determine the percentage of candidate positions--excluding included_letters--
        that guess matches over all candidates.
        """
        # guess_set = set(guess) - set(included_letters)
        # list_nbr_uncat: List[int]= [len(guess_set.intersection(set(cndt))) for cndt in current_candidates]
        list_nbr_uncat: List[int]= [int(guess[pos] == cndt[pos] and cndt[pos] not in included_letters)
                                    for pos in range(5)
                                    for cndt in current_candidates]
        avg_interscts_pct = 100*(sum(list_nbr_uncat)/len(current_candidates))
        avg_interscts = int(round(avg_interscts_pct))
        return avg_interscts


    @staticmethod
    def pct_of_words_hit(guess, current_candidates, included_letters) -> int:
        trgt_ltrs: set[str] = set(guess) - set(included_letters)
        words_hit: List[str]= [cndt for cndt in current_candidates if trgt_ltrs.intersection(set(cndt))]
        # print(f'{self.guess}: {len(words_hit)}', end=', ')
        return int(round(100*len(words_hit)/len(current_candidates)))


# def sorted_guesses(guesses: List[Guess]) -> List[Guess]:
#     sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:8]
#     return sorted_guesses


# def sorted_guesses_from_words(current_candidates: List[str],
#                               guess_words: List[str],
#                               included_letters: Set[str]=set(),
#                               current_step=1) -> List[Guess]:
#     guesses = [Guess(guess, current_candidates, included_letters) for guess in guess_words]
#     sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:8]
#     return sorted_guesses


# list_guesses = [g1, g2] = [Guess('raise'), Guess('arise')]
# print([g1, g2])
# print(f"[{g1}, {g2}]")
# for g in list_guesses:
#     print(g, end=' ')
# print()

# Utils.update_forward_refs()

In [ ]:
g1 = Guess('asdfe')
print(g1)
g2 = Guess('gfihj')
print(g2)
g3 = Guess('jklmn')
print(g3)
# # gs = [f"'{g.guess}': {g.metrics()}" for g in [g1, g2, g3]]
gs = {g.guess: g.metrics() for g in [g1, g2, g3]}
print(f'\n{type(gs) = },   {gs = }')
# gs = {str(g) for g in [g1, g2, g3]}
# print(f'\n{type(gs) = },   {gs = }')
# gs = dict(sorted(gs.items(), key=lambda item: item[1], reverse=True))
# print(f'{gs = }, {type(gs) =
# gs_str = ', '.join(gs) + ','
# print(gs_str)



'asdfe':  (85, 96, 30, 96)
'gfihj':  (54, 96, 18, 96)
'jklmn':  (61, 96, 15, 96)

type(gs) = <class 'dict'>,   gs = {'asdfe': (85, 96, 30, 96), 'gfihj': (54, 96, 18, 96), 'jklmn': (61, 96, 15, 96)}


In [ ]:
fwst_rem =  {'raise': (61, 43, 70), 'arise': (64, 51, 70), 'irate': (64, 53, 70), 'arose': (66, 49, 70),
             'alter': (70, 60, 70), 'stare': (71, 51, 70), 'snare': (71, 54, 70), 'slate': (72, 51, 70),
             'alert': (72, 52, 70), 'crate': (73, 49, 70), 'trace': (74, 49, 70), 'aisle': (76, 57, 70),
             'alone': (77, 56, 70), 'crane': (79, 51, 70), 'react': (80, 48, 70), 'scare': (81, 53, 70),
             'alien': (84, 51, 70), 'caste': (85, 56, 70), 'heart': (89, 55, 70), 'plate': (89, 61, 70)}

# broader_dict = {guess: (Guess.pct_of_words_hit(guess, current_candidates, []), a, b, c) for (guess, (a, b, c)) in fwst_rem.items()}

nbr_cndts = len(wordle_word_list)
def fwst_to_pct(x):
    return round(100*(nbr_cndts - x) / nbr_cndts, 1)

max_metrics = {guess :  (Guess.pct_of_words_hit(guess, wordle_word_list, []), b, a, c)  for (guess, (a, b, c)) in fwst_rem.items()}

print(f'\n{max_metrics}\n')



{'raise': (93, 43, 61, 70), 'arise': (93, 51, 64, 70), 'irate': (92, 53, 64, 70), 'arose': (92, 49, 66, 70), 'alter': (92, 60, 70, 70), 'stare': (90, 51, 71, 70), 'snare': (91, 54, 71, 70), 'slate': (90, 51, 72, 70), 'alert': (92, 52, 72, 70), 'crate': (89, 49, 73, 70), 'trace': (89, 49, 74, 70), 'aisle': (92, 57, 76, 70), 'alone': (92, 56, 77, 70), 'crane': (89, 51, 79, 70), 'react': (89, 48, 80, 70), 'scare': (90, 53, 81, 70), 'alien': (91, 51, 84, 70), 'caste': (89, 56, 85, 70), 'heart': (89, 55, 89, 70), 'plate': (89, 61, 89, 70)}



In [ ]:

class Secret_Candidates():
    """
    Given a set of possible Wordle answers, a profile of those words is:
    (a) a list of 5 Counters, one for each letter position in the 5-character Wordle words.
        Each counter has the English letters as keys. The values are the number of times that letter
        appears at the Counter's position in the set of words.
    (b) a counter for the number of words in which the letters appear at least once.

    After these Counters are calculated, the words are evaluated against them. The result can be used
    to select a Wordle guess
    """

    W = TypeVar("W", bound="Wordle")

    # def __init__(self, current_candidates, guess_words, filter=Filter(5), wordle: Union[W, Any]=None, current_step=0):
    def __init__(self, current_candidates, guess_words, filter=Filter(5), current_step=0):
                    #  interactive: Union[bool, Any]=False, is_long_test: Union[bool, Any]=False, current_step=0):
        """
        Extract scores for the positions and letters from words. (These are the remaining
        possible secret words.)
        self.position_profiles and self.letter_profiles characterize the collection of words.
        a) position_profiles are profiles for each letter position. For each position, position_profiles
           counts the number of occurrences of the various letters in that position in all the words.
        b) letter_profiles is a single Counter for the letters in all the words. It counts the
           number of occurrences of the various letters in all the words.

        First build the position_profiles and the letter_profile and then evaluate the words
        against them.
        """

        if current_candidates == []:
            raise ValueError(f'current_candidates is empty.')

        # self.wordle = wordle

        self.current_candidates = current_candidates

        self.word_length = 5

        # ===============================================================

        # Generate the Profiles and letter-sets based on the candidates.

        self.position_profiles: List[Counter[str]]
        self.letter_profiles: Counter[str]

        self.position_profiles, self.letter_profiles = self.generate_candidates_profiles(current_candidates)

        sets: Tuple[Set[str], Set[str], Set[str], Counter[str]] = self.generate_letter_sets(filter)

        (self.included_letters, self.excluded_letters, self.known_letters, self.uncategorized_letters) = sets

        # Letters that are known to be included in the secret word
        self.included_letters:  Set[str]

        # Letters that are known to be excluded from the secret word
        self.excluded_letters:  Set[str]

        # The union of the included and excluded sets
        self.known_letters:     Set[str] # The union of the included and excluded sets

        # Letters in the candidate secret words that are not (yet) in known letters
        self.uncategorized_letters: Counter[str]

        # ==============  Update the Filter and then update included_letters and uncategorized_letters ===============

        self.filtr = filter

        new_green_ltrs: List[str] = filter.edit_filter_from_profiles(self.position_profiles)

        self.included_letters = self.included_letters.union(new_green_ltrs)
        self.uncategorized_letters = Counter({key: self.uncategorized_letters[key] for key in
                                              (set(self.uncategorized_letters.keys()) - set(new_green_ltrs))})

        # ======================== Build the guess lists =======================================

        self.fl_guesses: Set[str] = self.guesses_with_frequent_letters(self.position_profiles, guess_words, verbose=False)

        self.best_position_guesses: Set[str] = self.best_positn_guesses(guess_words)
        self.best_uncertain_position_guesses: Set[str] = self.best_positn_guesses(guess_words, discount_knowns=True)

        self.guesses_with_most_hits: Counter[str] = self.guesses_with_the_most_hits(guess_words)
        self.guesses_with_most_unc_hits: Counter[str] = self.guesses_with_the_most_unc_hits(guess_words)

        self.intersections: Dict[str, float] = (
            {'alert': 1.78, 'alter': 1.78, 'irate': 1.78, 'later': 1.78, 'arose': 1.77,
             'stare': 1.77, 'arise': 1.76, 'raise': 1.76, 'learn': 1.73, 'renal': 1.73,
             'saner': 1.71, 'snare': 1.71, 'cater': 1.69, 'crate': 1.69, 'react': 1.69,
             'trace': 1.69, 'aisle': 1.68, 'clear': 1.68, 'least': 1.68, 'slate': 1.68}

            if current_step == 1 else self.avg_of_intersections(guess_words, current_candidates)
                                                )

        self.max_cndts_reduction: Dict[str, Tuple] = (

            {'raise': (97, 43, 97), 'arise': (97, 51, 97), 'irate': (97, 53, 97), 'arose': (97, 49, 97),
             'alter': (97, 60, 97), 'stare': (97, 51, 97), 'snare': (97, 54, 97), 'slate': (97, 51, 97),
             'alert': (97, 52, 97), 'crate': (97, 49, 97), 'trace': (97, 49, 97), 'aisle': (97, 57, 97),
             'alone': (97, 56, 97), 'crane': (97, 51, 97), 'react': (97, 48, 97), 'scare': (97, 53, 97),
             'alien': (96, 51, 97), 'caste': (96, 56, 97), 'heart': (96, 55, 97), 'plate': (96, 61, 97)}

            if current_step == 1 else self.max_candidates_reduction(current_candidates=current_candidates,
                                                                    guess_words=guess_words,
                                                                    current_step=current_step,
                                                                    verbose=False)
                                                            )

    # ================================== Profile generation ===========================================

    def generate_candidates_profiles(self, candidates) -> Tuple[List[Counter[str]], Counter[str]]:
        '''
        Calculate scores for the positions and letters of the words.
        candidates are the candidate secret words
        '''
        # position_profiles are profiles for each letter position. For each position, a profile counts
        # the number of occurrences of the various letters in that position in the remaining words.
        position_profiles: List[Counter[str]] = [Counter(), Counter(), Counter(), Counter(), Counter()]

        # letter_profiles counts the number of occurrences of the various letters in all the words.
        letter_profiles: Counter[str]  = Counter()

        # Build the position_profiles and the letter_profiles
        for cndt in candidates:
            # Use zip to pair the word positions with its associated position Counter
            used_ltrs = set()
            for (ltr, position_profile) in zip(cndt, position_profiles):
                position_profile[ltr] += 1
                if ltr not in used_ltrs:
                    letter_profiles[ltr] += 1
                    used_ltrs.add(ltr)
        return position_profiles, letter_profiles


    def generate_letter_sets(self, filter) -> Tuple[Set[str], Set[str], Set[str], Counter[str]]:

        # The letters known to be included in the secret word
        included_letters: Set[str] = set(filter.greens).difference("_").union(filter.yellows.keys())

        # The letters known to be excluded in the secret word
        excluded_letters: Set[str] = set(filter.grays)

        # The union of the preceding two sets
        known_letters: Set[str] = included_letters.union(excluded_letters)

        # Letters in the secret word candidates that are not (yet) known to be either included or excluded
        uncategorized_letters: Counter[str] = Counter({ltr: self.letter_profiles[ltr] for ltr in (set(self.letter_profiles) - known_letters)})

        return included_letters, excluded_letters, known_letters, uncategorized_letters


    def avg_of_intersections(self, guess_words, secret_word_candidates) -> Dict[str, float]:
        """
        Compare each possible guess (in guess_words) with each secret-word candidate by determining how
        many letters each pair has in common. (Don't count self.included_letters since each possible guess and each
        candidate secret word has them all.) For each possible guess, average the number of letters in common with all
        the candidate secret words and return a dictionary with that average as the value for each possible guess.

        We want to know how much of each secret-word candidate, on average, (other than the included letters)
        a guess would expose to the secret word were it to be used as the guess.
        """
        len_cndts = len(secret_word_candidates)
        soi: Dict[str, float] = dict(Counter(
            {guess: Guess.pct_current_cndt_pos_matched(guess, secret_word_candidates, self.included_letters)
                                                                    for guess in guess_words}).most_common()[:20])
        return soi


    def best_positn_guesses(self, guess_words, discount_knowns=False) -> Set[str]:
        """
        Find and return the best position-based guess.
        If discount_knowns=True, discount known letters in scoring.
        """

        highest_ltrs: List[str] = list([cntr.most_common()[0][0] for cntr in self.position_profiles])
        if len(highest_ltrs) < 5:
            raise ValueError(f'In best_position_guesses: {highest_ltrs = }')
        tent_guess: str = ''.join(highest_ltrs)
        if tent_guess in guess_words:
            return {tent_guess}

        # If we've narrowed the remaining candidates down to 3 or fewer, don't use discount_knowns
        if len(self.current_candidates) <= 3:
            discount_knowns = False

        eval_fn = self.eval_guess_against_profiles

        guess_scores: Counter[str] = Counter({guess: eval_fn(guess, discount_knowns) for guess in guess_words})

        best_guesses = set(dict(guess_scores.most_common()[:5]).keys())

        return best_guesses


    def compute_entropy(self, guess, score_a_guess) -> float:
        hashes_per_cand: Dict[str, int] = {candidate: score_a_guess(guess, candidate).count('#')
                                                      for candidate in self.current_candidates}
        hash_classes: Dict[int, Set[str]] = {n: {cand for cand in hashes_per_cand if hashes_per_cand[cand] == n}
                                                for n in sorted(set(hashes_per_cand.values()))}
        len_candidates = len(self.current_candidates) - 1 if len(self.current_candidates) > 1 else 1
        dist_value = sum([(5-n)*len(hash_classes[n]) for n in hash_classes])

        entropy = round(sum([len(cls)/len_candidates * log2(len_candidates/len(cls))
                                                                         for cls in hash_classes.values()]), 2)
        return entropy


    def eval_guess_against_profiles(self, guess, discount_knowns) -> int:
        '''
        Given a guess, find (a) the sum of its word-position-vals according to the position_profiles
        and (b) its letters-val according to the letter_profiles
        A word gets credit for its letters in proportion to the frequency with which those
        letters correspond the the overall letter frequency at each position.
        '''
        # positions_score is the score for guess based on its letters at their positions
        # letters_score is the score for guess based on its letters at any position
        positions_score, letters_score = 0, 0

        # Only count the letter score once even if a letter appears multiple times in a guess
        used_ltrs = set()

        # pos_counter is a Counter of frequencies of letters in a specific position.
        # We want to know for each ltr in a guess, how often that ltr appears in its position in
        # all remaining candidate secret words.
        # The zip expression pairs the letters with the Counter for that position
        for (ltr, pos_counter) in zip(guess, self.position_profiles):
            positions_score += pos_counter[ltr] if not discount_knowns or ltr in self.uncategorized_letters else 0
            if ltr not in used_ltrs:
                letters_score += self.letter_profiles[ltr] - 1
                used_ltrs.add(ltr)

        # A word's score is its position score + 1/4 * its letter score
        score = positions_score + int(round(letters_score/4))
        return score


    def max_candidates_reduction(self,
                                 current_candidates,
                                 guess_words,
                                #  guess_words_start_nbr=0,
                                 current_step=-1,
                                #  saved_dict: Union[Dict[str, Tuple[float, float]], Any]=None,
                                 verbose=None) -> Dict[str, Tuple]:
        """
        The objective is to determine which of the words in guess_words would eliminate the most words
        from current_candidates. Since we don't know which word in current_candidates is the secret word,
        try them all and score each guess_word as the mean/median number of current_candidates remaining.

        Test each guess (gw) in guess_words against each word in current_candidates as the unknown secret word.
        Use the resulting "score" (pattern) to determine how many words in current_candidates would remain
        viable given gw and the presumed secret word. (The smaller the better.) This determines
        how many of the current_candidates, gw would eliminate. Return a dictionary with the guess words
        as key and a tuple (mean remaining, median remaining) as the value for each key. The guess_word
        with the smallest mean is the best guess.
        """

        # if verbose:
        #     # print(f'1. {guess_words = }, {guess_words_start_nbr = }, {guess_words[guess_words_start_nbr:] = }')

        #     # if saved_dict is not None and guess_words_start_nbr > 0:
        #     #     print(f'Resuming at {guess_words_start_nbr} ({guess_words[guess_words_start_nbr]})\n' +
        #     #           f'{Utils.dict_to_row_strings(saved_dict)}\n')

        #     print(f'Testing the current {len(current_candidates)}-long NYT Wordle words (from "{current_candidates[0]}" to ' +
        #           f'"{current_candidates[len(current_candidates)-1]}") for the best first guess against\n' +
        #           f'all possible secret words from that same list. "Best" means fewest remaining words after the guess.\n\t')

        rem_dict: Dict[str, Tuple[float, float]] = dict() # if saved_dict is None else saved_dict

        for guess in guess_words:  # [guess_words_start_nbr:]:
            mean_remaining, median_remaining = Guess.fwst_remaining(current_candidates, guess)
            rem_dict[guess] = (mean_remaining, median_remaining)

        fwst_remaining: Dict[str, Tuple] = Utils.smallest_n(rem_dict, n=20)
        return fwst_remaining


    def guesses_with_the_most_hits(self, guess_words):
        hits_cntr = Counter({guess: sum([self.letter_profiles[ltr] for ltr in set(guess)]) for guess in guess_words})
        best_guesses = Counter(dict(hits_cntr.most_common()[:5]))
        return best_guesses


    def guesses_with_the_most_unc_hits(self, guess_words):
        hits_cntr = Counter({guess: sum([self.uncategorized_letters[ltr] for ltr in set(guess)]) for guess in guess_words})
        best_guesses =  Counter(dict(hits_cntr.most_common()[:5]).keys())
        return best_guesses


    def guesses_with_frequent_letters(self, pos_prfls, guess_words, verbose=False):
        """
        Find guesses made from top letters in each position.
        """
        tmp_highest_cntrs: List[Tuple[Counter, int]] = [(ctr, ceil(max(ctr.values())*0.75)) for ctr in pos_prfls]
        highest_cntrs = [(ctr, min(lb, max(ctr.values())-1)) for (ctr, lb) in tmp_highest_cntrs]

        highest_ltr_sets: List[Set[str]] = [set(sorted({ltr for ltr in cntr if cntr[ltr] >= cntr_mx}))
                                                                         for cntr, cntr_mx in highest_cntrs]
        if verbose: print(f'{highest_ltr_sets = }')

        cand_guesses_0: List[str] = sorted([''.join(elmt) for elmt in product(*highest_ltr_sets)])

        cand_guesses = set(sorted([guess for guess in cand_guesses_0 if guess in guess_words]))

        return cand_guesses


    # def select_a_guess(self, greens, current_candidates, guess_words,
    #                    current_step, hard_mode, filter, interactive) -> str:
    #     """
    #     Suggest the next guess. There are a couple of issues.
    #     - The range of possible words from which to select. The two
    #       most widely used approaches are:
    #         a) the remaining candidates. This list automatically
    #            satisfies the constraints derived from previous guesses.
    #         b) a separate list of allowable guess words. This list is
    #            generally larger than the list of words from which the
    #            secret word is selected. This list must be filtered to
    #            eliminate words that don't satisfy the constraints derived
    #            from previous guesses.
    #     - The system used to score possible guesses to determine which has
    #       the best chance of being the best guess.
    #     """

    #     the_guess: str     # Union[str, Any] = None

    #     # Is only one candidate left?
    #     if '_' not in greens or len(current_candidates) == 1:
    #         the_guess = ''.join(greens) if '_' not in greens else current_candidates[0]

    #         if interactive:
    #             print(f'\nSince "{the_guess}" is the only remaining secret-word candidate, it will be the next guess.')
    #             print(f'Guess {current_step}   > {the_guess}')
    #         return the_guess

    #     else:
    #         guesses = [Guess(guess, current_candidates, self.included_letters, current_step) for guess in guess_words]
    #         # if current_step == 1:
    #         #     # print('Starting list of Guesses')
    #         #     list_of_lists = Utils.lists_to_many_lists(guesses, 5)
    #         #     for lst in list_of_lists:
    #         #         lst_of_strs = [str(g) for g in lst]
    #         #         strng = ', '.join(lst_of_strs) + ','
    #         #         print(strng)
    #         #     # print(f'{len(guesses) = }\n{alpha_segments(guesses, line_length=5, multiple_lines_per_letter=None)}')
    #         sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:30]
    #         if interactive:
    #             lsts_of_sorted_guesses = Utils.lists_to_many_lists(sorted_guesses, 5)
    #             print(f'\n** sorted_guesses ')
    #             for sub_sorted_guesses in lsts_of_sorted_guesses:
    #                 print(f'  {", ".join([str(g) for g in sub_sorted_guesses])}')
    #         # words = []
    #         # if current_step == 1:
    #         #     words = list(self.max_cndts_reduction.keys())[:8]
    #         # else:
    #         #     # sorted_guesses: List[Guess] = sorted_guesses_from_words(current_candidates,
    #         #     #                                                         guess_words,
    #         #     #                                                         included_letters=self.included_letters,
    #         #     #                                                         current_step=current_step)
    #         #     # guesses = [Guess(guess, current_candidates, self.included_letters, current_step) for guess in guess_words]
    #         #     # sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:30]
    #         #     # if self.wordle.interactive:
    #         #     #     lsts_of_sorted_guesses = Utils.lists_to_many_lists(sorted_guesses, 5)
    #         #     #     print(f'\n** sorted_guesses ')
    #         #     #     for sub_sorted_guesses in lsts_of_sorted_guesses:
    #         #     #         print(f'  {", ".join([str(g) for g in sub_sorted_guesses])}')
    #         words: List[str] = [g.guess for g in sorted_guesses]

    #         if interactive:
    #             user_guess = Utils.get_user_guess(current_step, words, hard_mode, filter)
    #             print(f'Guess {current_step}   > {user_guess}')
    #             return user_guess

    #         else:
    #             the_guess = words[0]
    #             if interactive:
    #                 print(f'Guess {current_step}   > {the_guess}')
    #             return the_guess


# scs = Secret_Candidates(wordle_word_list, guess_words)
# ranking: Dict[str, Tuple[int, int]] = scs.max_candidates_reduction(wordle_word_list, guess_words, guess_words_start_nbr=0,
                                                                    #   current_step=-1, saved_dict=None, verbose=False)

In [ ]:
from re import sub
s = 'asdf234h'
s2 = sub(r'\D','',s)
s3 = sub(r'\d','',s)
print(f'{s = }   {s2 = }   {s3 = }')


s = 'asdf234h'   s2 = '234'   s3 = 'asdfh'


In [ ]:
class Wordle():

    def __init__(self, is_long_test=False, candidates_are_guess_words=False):

        # Are we performing a long test?
        self.is_long_test: bool = is_long_test

        # If true, the guess_words and the current_candidates are the same ofject.
        self.candidates_are_guess_words: bool = candidates_are_guess_words

        # self.guess_strategy: Union[Callable, Any] = None

        # In hard_mode, guesses must be acceptable secret-word candidates
        # self.hard_mode: bool = True

        # If interactive is True, the user enters guesses
        self.interactive: Optional[bool] = None

        # If online is True, the user enters both the guesses and the score of the guess--
        # as when playing the NYT Wordle. The user serves as a communication channel that
        # allows the system to play the online game.
        self.online: Optional[bool] = None


        # self.verbose: bool = False

        # filter = Filter(5)
        # self.placed_ltrs = ["".join(filter.greens)]
        # self.unplaced_ltrs = []
        # self.guesses = []
        # self.scores  = []
        # self.words_left = ['2317', ]


    def play_a_game(self,
                    current_candidates,
                    guess_words,
                    secret_word=None,
                    hard_mode=True,
                    interactive=False,
                    online=False,
                    verbose=False):

        # If self.candidates_are_guess_words, no need to recompute the guess_words at each turn.
        # Can use the same list of words.
        self.candidates_are_guess_words = current_candidates == guess_words
        if not self.candidates_are_guess_words:
            warn('\n *** Warning: current_candidates and guess_words are not the same. ***\n')

        self.hard_mode = hard_mode

        # Using either "str or None" or "Optional[str]" as the type and assigning None generates an error
        # when we apply len() to self.secret_word. Also, "Sized" as the type generates an error
        # when None is assigned. But Union[str, Any] works. None can be assigned, and len() can be applied.

        self.secret_word: Union[str, Any] = None if self.online else \
                                            secret_word if secret_word else \
                                            choice(current_candidates)

        # If online is True, the user enters both the guesses and the score of the guess--
        # as when playing the NYT Wordle. The user serves as a communication channel that
        # allows the system to play the online game.
        self.online: Optional[bool] = online

        # If interactive is True, the user enters guesses
        self.interactive: Optional[bool] = True if self.online else interactive

        self.verbose = interactive or verbose

        self.filter = Filter(5)

        # self.placed_ltrs = ["".join(filter.greens)]
        self.placed_ltrs = []
        # self.unplaced_ltrs = ["".join(sorted(filter.yellows.keys()))]
        self.unplaced_ltrs = []

        # The following keep track of the guesses, scores, etc. for each game to be reported at the end
        self.guesses = []
        self.scores  = []
        self.words_left = [str(len(current_candidates))]

        return self.finish_the_game(current_candidates, guess_words, self.filter)


    def finish_the_game(self, current_candidates, guess_words, filter, current_step=1):
        """
        Use recursion instead of looping to go through the guesses.
        """

        candidates_object = Secret_Candidates(current_candidates, guess_words, self.filter, current_step=current_step)
        if self.verbose:
            Utils.print_profiles(candidates_object, current_step, filter)

        guess: str = self.select_a_guess(self.filter.greens, current_candidates, guess_words,
                                         current_step, candidates_object.included_letters, filter)

        # if self.hard_mode and not self.filter.word_passes(guess):
        #     raise Exception(f'Hard-mode violation: {guess = } not in {guess_words = }.')

        score: str = Utils.get_score_from_user() if self.online else Utils.score_a_guess(self.secret_word, guess)

        self.placed_ltrs.append("".join(self.filter.greens))
        sorted_keys = sorted(self.filter.yellows.keys())
        joined_keys = ".".join(sorted_keys)
        yellows = '     ' if not sorted_keys else ((' '*(5-len(joined_keys))) + joined_keys)
        # self.unplaced_ltrs.append("".join([('    ' + key) for key in sorted(filter.yellows.keys())]))
        self.unplaced_ltrs.append(yellows)
        self.guesses.append(guess)
        self.scores.append(score)

        if self.is_long_test:
            print(f'{current_step}. {guess} ({score}); ', end='')
        elif self.verbose:
            # print(f'{current_step}.\t{guess = }')
            print(f'     Score: {score}')

        # Stop if we've found the secret word.
        if set(score) == {'g'}:
            if self.verbose:
                steps = len(self.placed_ltrs)
                step_markers = 'Steps:                   ' + "__1______2______3______4______5______6___"[:6*steps]
                plural = "s" if current_step > 1 else ""
                print(f'\nYou found the secret word "{guess}" in {current_step} step{plural}.')
                print('\nYour guesses and scores were as follows.\n')
                # print('Steps:                   __1______2______3______4______5______6___')
                print(step_markers)
                print(' -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - ')
                print(f'Placed letters:          {", ".join(self.placed_ltrs)}')
                print(f'Unplaced letters:        {", ".join(self.unplaced_ltrs)}')
                print(' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -')
                print(f'Guesses:                 {", ".join(self.guesses)}')
                print(f'Scores:                  {", ".join(self.scores)}')
                self.words_left.append(f'{0:5}')
                print(f'Words left:        {", ".join(self.words_left)}')
            return current_step

        # Update the filter.
        new_filter = self.filter.edit_filter(guess, score)

        # Use the updated filter to filter the current_candidates and the guess_words
        remaining_current_candidates = [word for word in current_candidates if new_filter.word_passes(word)]

        self.words_left.append(f'{len(remaining_current_candidates):5}')

        if not self.candidates_are_guess_words:
            warn('\n *** Warning: current_candidates and guess_words are not the same. ***\n')

        if self.secret_word is not None and self.secret_word not in remaining_current_candidates:
            raise Exception(f'The secret_word "{self.secret_word}" is not in {remaining_current_candidates = }')

        remaining_guess_words = remaining_current_candidates if self.candidates_are_guess_words else \
                                                        [word for word in guess_words if new_filter.word_passes(word)]

        if remaining_current_candidates != remaining_guess_words:
            warn(f'\n *** Warning: current_candidates and guess_words are not the same. ***\n')
            warn(f'\n *** {remaining_current_candidates = } {remaining_guess_words = } ***\n')

        # Recursive call for the remaining guesses
        return self.finish_the_game(remaining_current_candidates, remaining_guess_words, new_filter, current_step + 1)


    @staticmethod
    def long_test():
        guess_words=wordle_word_list
        verbose=False
        total_guesses = 0
        print_counter = 0
        nbr_of_games = len(wordle_word_list)
        for game_nbr in range(1, nbr_of_games + 1):
            secret_word = wordle_word_list[-game_nbr]
            print(f'{game_nbr:4}. secret word: {secret_word}', end=';  ')
            wordle = Wordle(is_long_test=True)
            guess_count = wordle.play_a_game(wordle_word_list, guess_words, secret_word=secret_word)
            total_guesses += guess_count
            print(f'Avg guesses: {round(total_guesses/game_nbr, 2)}')


    def select_a_guess(self, greens, current_candidates, guess_words, current_step, included_letters, filter) -> str:
        """
        Suggest the next guess. There are a couple of issues.
        - The range of possible words from which to select. The two
          most widely used approaches are:
            a) the remaining candidates. This list automatically
               satisfies the constraints derived from previous guesses.
            b) a separate list of allowable guess words. This list is
               generally larger than the list of words from which the
               secret word is selected. This list must be filtered to
               eliminate words that don't satisfy the constraints derived
               from previous guesses.
        - The system used to score possible guesses to determine which has
          the best chance of being the best guess.
        """

        the_guess: str     # Union[str, Any] = None

        # Is only one candidate left?
        if '_' not in greens or len(current_candidates) == 1:
            the_guess = ''.join(greens) if '_' not in greens else current_candidates[0]

            if self.interactive:
                print(f'\nSince "{the_guess}" is the only remaining secret-word candidate, it will be the next guess.')
                print(f'Guess {current_step}   > {the_guess}')
            return the_guess

        else:

            guesses = [Guess(guess, current_candidates, included_letters, current_step) for guess in guess_words]

            # if current_step == 1:
            #     # print('Starting list of Guesses')
            #     list_of_lists = Utils.lists_to_many_lists(guesses, 5)
            #     for lst in list_of_lists:
            #         lst_of_strs = [str(g) for g in lst]
            #         strng = ', '.join(lst_of_strs) + ','
            #         print(strng)
            #     # print(f'{len(guesses) = }\n{alpha_segments(guesses, line_length=5, multiple_lines_per_letter=None)}')
            sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:30]
            if self.interactive:
                lsts_of_sorted_guesses = Utils.lists_to_many_lists(sorted_guesses, 5)
                print(f'\n** sorted_guesses ')
                for sub_sorted_guesses in lsts_of_sorted_guesses:
                    print(f'  {", ".join([str(g) for g in sub_sorted_guesses])}')
            # words = []
            # if current_step == 1:
            #     words = list(self.max_cndts_reduction.keys())[:8]
            # else:
            #     # sorted_guesses: List[Guess] = sorted_guesses_from_words(current_candidates,
            #     #                                                         guess_words,
            #     #                                                         included_letters=self.included_letters,
            #     #                                                         current_step=current_step)
            #     # guesses = [Guess(guess, current_candidates, self.included_letters, current_step) for guess in guess_words]
            #     # sorted_guesses: List[Guess] = sorted(guesses, reverse=True, key=lambda g: g.metrics())[:30]
            #     # if self.wordle.interactive:
            #     #     lsts_of_sorted_guesses = Utils.lists_to_many_lists(sorted_guesses, 5)
            #     #     print(f'\n** sorted_guesses ')
            #     #     for sub_sorted_guesses in lsts_of_sorted_guesses:
            #     #         print(f'  {", ".join([str(g) for g in sub_sorted_guesses])}')
            words: List[str] = [g.guess for g in sorted_guesses]

            if self.interactive:
                user_guess = Utils.get_user_guess(current_step, words, self.hard_mode, filter)
                print(f'Guess {current_step}   > {user_guess}')
                return user_guess

            else:
                the_guess = words[0]
                if self.interactive:
                    print(f'Guess {current_step}   > {the_guess}')
                return the_guess


wwl = list(wordle_word_dict.keys())

# Make it an assignment so that the result is not printed when the game completes.
_ = Wordle().play_a_game(wwl, wwl, secret_word=None, interactive=True, online=False)
# _ = Wordle.long_test()




-------------------- The initial state of the game --------------------

2317 initial secret-word candidates


Initial letter position frequencies
1. {'s': 366, 'c': 198, 'b': 173, 't': 149, 'p': 143}
2. {'a': 304, 'o': 279, 'r': 268, 'e': 242, 'i': 202}
3. {'a': 307, 'i': 267, 'o': 244, 'e': 177, 'u': 166}
4. {'e': 318, 'n': 182, 's': 171, 'l': 163, 'a': 163}
5. {'e': 424, 'y': 364, 't': 253, 'r': 212, 'l': 156}

Letters that appear in the most words in the initial secret-word candidates:
	{'e': 1056, 'a': 910, 'r': 838, 'o': 673, 't': 667, 'l': 649, 'i': 648, 's': 618, 'n': 550}  (out of 26)

Placed letters: _____
Required but unplaced letters:
Excluded letters: 
26 uncategorized letters:
	{'e': 1056, 'a': 910, 'r': 838, 'o': 673, 't': 667, 'l': 649, 'i': 648, 's': 618, 'n': 550} (out of 26)


** sorted_guesses 
  'arise':  (93, 97, 55, 97), 'raise':  (93, 97, 55, 97), 'arose':  (92, 97, 54, 97), 'alone':  (92, 97, 51, 97), 'irate':  (92, 97, 51, 97)
  'aisle':  (92, 97, 44, 97), 'a